<a href="https://colab.research.google.com/github/eisbetterthanpi/Autoencoder/blob/main/blt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## shelf

In [1]:
# @title hf stream dataset me
!pip install -qU datasets # restart?
from datasets import load_dataset
import torch
from torch.utils.data import Dataset
import tiktoken # https://github.com/openai/tiktoken/tree/main

class StreamDataset(Dataset):
    def __init__(self, dataset, seq_len, buffer_size):
        self.enc = tiktoken.get_encoding("gpt2") # https://github.com/openai/tiktoken/blob/main/tiktoken/core.py
        self.vocab_size = self.enc.n_vocab # gpt2:50257
        self.dataset = dataset
        self.data = iter(dataset)
        self.seq_len = seq_len
        self.buffer_size = buffer_size  # must be ≥ seq_len
        self.buffer = []  # token buffer
        self.fill_buffer()

    def fill_buffer(self):
        while len(self.buffer) < self.buffer_size:
            x = next(self.data)
            tokens = self.enc.encode(x["text"]) # tiktoken
            self.buffer.extend(tokens)

    def __len__(self):
        # /4.5/(4/3)
        return 128000000
        # return self.length

    def __getitem__(self, idx):
        # print('get', idx)
        if idx == 0: self.data = iter(self.dataset)
        if len(self.buffer) < self.seq_len: self.fill_buffer()
        if len(self.buffer) < self.seq_len:
            raise StopIteration
        x = self.buffer[:self.seq_len]
        self.buffer = self.buffer[self.seq_len:]
        return torch.tensor(x)

def collate_fn(batch):
    # print(batch)
    return torch.stack(batch)

# dataset = load_dataset("stas/openwebtext-10k", split="train", streaming=True, cache_dir="/content/hf")
# dataset = load_dataset("maxtli/OpenWebText-2M", split="train", streaming=True, cache_dir="/content/hf") # 8.7,3.8
dataset = load_dataset("Skylion007/openwebtext", trust_remote_code=True, split="train", streaming=True, cache_dir="/content/hf") # 8.7,3.8

seq_len = 128
buffer_size = seq_len*1
train_data = StreamDataset(dataset, seq_len, buffer_size) # train_data = StreamDataset(dataset["train"], seq_len, buffer_size)
del dataset

from torch.utils.data.dataloader import DataLoader
batch_size = 64 #512
train_loader = DataLoader(train_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=True, pin_memory=True, num_workers=2)
del train_data
def encode(context):
    if type(context) == str: return torch.tensor([train_loader.dataset.enc.encode(context)], device=device)
    elif type(context) == list: return train_loader.dataset.enc.encode_batch(context)
    else: raise Exception
def decode(x): return train_loader.dataset.enc.decode(list(x))
# for x,y in train_loader:
#     break
# print(train_data.vocab_size)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.35k [00:00<?, ?B/s]

openwebtext.py:   0%|          | 0.00/2.73k [00:00<?, ?B/s]

In [2]:
# @title -1,1 code quant
import torch
import torch.nn as nn
import torch.nn.functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"

def ste_round(x): return x.round().detach() + x - x.detach()

def soft_clamp_relu(x, lower=None, upper=None):
    if lower != None: x = lower+F.relu(x-lower)
    if upper != None: x = upper-F.relu(upper-x)
    return x

class codequant(nn.Module):
    def __init__(self, levels):
        super().__init__()
        self.levels = torch.tensor(levels, device=device)
        self.basis = torch.cumprod(torch.tensor([*levels[1:], 1], device=device).flip(-1), dim=0).flip(-1)
        self.half_width = (self.levels-1)/2
        self.codebook_size = torch.prod(self.levels).item()
        # self.codebook = self.indexes_to_codes(torch.arange(self.codebook_size, device=device))
    @property
    def codebook(self): return self.indexes_to_codes(torch.arange(self.codebook_size, device=device))

    def codes_to_indexes(self, zhat):
        zhat = soft_clamp_relu(zhat,-1,1)
        zhat = (zhat*torch.pi/2).sin() # cos to unif
        zhat = (zhat + 1) * self.half_width
        return (zhat * self.basis).sum(axis=-1).round().int()

    def indexes_to_codes(self, indices):
        indices = indices.unsqueeze(-1)
        codes = torch.remainder(indices//self.basis, self.levels)
        # return codes / self.half_width - 1
        codes = codes / self.half_width - 1
        return soft_clamp_relu(codes,-1,1).asin()/torch.pi*2 # unif to cos

    def forward(self, z): # round code to nearest code
        offset = (self.levels+1) % 2 /2 # .5 if even, 0 if odd
        z = soft_clamp_relu(z,-1,1)
        z = (z*torch.pi/2).sin() # cos to unif
        # print('codequant fwd', z)
        bound = z * self.half_width + offset
        # print('codequant fwd', bound)
        quantized = ste_round(bound)
        # return (quantized-offset) / self.half_width # split [-1,1]
        zhat = (quantized-offset) / self.half_width # split [-1,1]
        return soft_clamp_relu(zhat,-1,1).asin()/torch.pi*2 # unif to cos

# cq = codequant(levels = [128,4,3,2])
# # print(cq.codebook)
# batch_size, seq_len = 2, 4
# # x = torch.linspace(-1.2,1.2,23).repeat(4,1).T
# x = torch.linspace(-1.2,1.2,29).repeat(4,1).T
# # x=la
# la = cq(x)
# print(la)
# lact = cq.codes_to_indexes(la)
# print(lact)
# la = cq.indexes_to_codes(lact)
# print(la)


In [3]:
# @title TTLinear
# Tensor Train embedding https://arxiv.org/pdf/1901.10787
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def make_einsum(num_tensors):
    a = 97
    R = chr(a+25) # 'z'
    lhs = [chr(a)+R]
    for i in range(1, num_tensors-1):lhs.append(R+chr(a+i)+R)
    lhs.append(R+chr(a+num_tensors-1))
    return ','.join(lhs) + '->' + ''.join([chr(a+i) for i in range(num_tensors)]) # az,zbz,zcz,zd->abcd

class TTLinear(nn.Module):
    def __init__(self, in_features=None, out_features=None, rank=256, std=1):
        super().__init__()
        self.lfeat = len(in_features)
        if self.lfeat==1: lst = in_features + out_features
        elif self.lfeat>=2: lst = [i*j for i, j in zip(in_features, out_features)]
        last = len(lst)
        var = last/rank**(1/(2*(std**.5)*last))
        c=1/last
        self.params = nn.ParameterList([nn.Parameter(torch.randn(lst[0], rank).clamp(-c,c)*var),
            *[nn.Parameter(torch.randn(rank, ij, rank).clamp(-c,c)*var) for ij in lst[1:-1]],
            nn.Parameter(torch.randn(rank, lst[-1]).clamp(-c,c)*var)])
        self.einsum_str = make_einsum(last)
        self.shape = [p for ij in zip(in_features, out_features) for p in ij]
        self.permute = list(range(0, 2*self.lfeat - 1, 2)) + list(range(1, 2*self.lfeat, 2))
    def weight(self): return torch.einsum(self.einsum_str, *self.params).reshape(self.shape).permute(self.permute).flatten(0,self.lfeat-1).flatten(1)

    def forward(self, x):
        weight = self.weight()
        return x.to(weight.dtype) @ weight


import math
class TTEmbedding(nn.Module):
    def __init__(self, in_dim, d_model, rank=256):
        super().__init__()
        self.ttlin = TTLinear(in_dim, d_model, rank, std=1) # https://docs.pytorch.org/docs/stable/generated/torch.nn.Embedding.html
        self.weight = self.ttlin.weight
        self.num_classes = math.prod(in_dim)

    def forward(self, x):
        return self.ttlin(F.one_hot(x, self.num_classes))
# self.out = lambda x: x @ self.tok_emb.weight().T  # weight tying

# in_features=(3,4,5,6); out_features=(2,3,4,5)
# in_features=[120]; out_features=[300]
# rank=16
# std=.5
# lin = TTLinear(in_features, out_features, rank, std).to(device)
# # x = torch.rand(4,math.prod((3,4,5,6)))
# x = torch.rand(4,7,math.prod(in_features), device=device)
# print(lin.params[0].device)
# out = lin(x)
# print(out.shape)
# print(lin.ttlin.params[0].device)

# emb = TTEmbedding(in_features, out_features, rank).to(device)
# x = torch.randint(0, math.prod(in_features), (2, 5), device=device)
# out = emb(x)
# print(out.shape)

# o=lin.weight
# print(o.mean().item(), o.std().item(), o.min().item(), o.max().item())

# import matplotlib.pyplot as plt
# plt.rcParams["figure.figsize"] = (4,4)
# # plt.hist(o.flatten().tolist(), bins=20, alpha=.5, label='context mask')
# # plt.hist(o[:100,:100].flatten().tolist(), bins=20, alpha=.5, label='context mask')
# x = torch.randn(100,100)*std
# # plt.hist(x.flatten().tolist(), bins=20, alpha=.5, label='context mask')
# plt.hist([o[:100,:100].flatten().tolist(), x.flatten().tolist()], bins=20, alpha=.5, label='context mask')
# plt.show()


In [4]:
# @title RoPE
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class RoPE(nn.Module): # Rotary Positional Embeddings
    def __init__(self, dim, seq_len=512, base=10000):
        super().__init__()
        self.dim, self.base = dim, base
        theta = 1.0 / (base ** (torch.arange(0, dim, step=2) / dim))
        pos = torch.arange(seq_len).unsqueeze(-1)
        angles = (pos * theta)[None,...,None] # [seq_len, 1] * [dim // 2] -> [1, seq_len, dim // 2, 1]
        self.rot_emb = torch.cat([torch.sin(angles), torch.cos(angles)], dim=-1).flatten(-2).to(device) # [seq_len, dim // 2, 2] -> [1, seq_len, dim]

    def forward(self, x): # [b,t,d]
        seq_len = x.size(1)
        if self.rot_emb.shape[1] < seq_len: self.__init__(self.dim, seq_len, self.base)
        return x * self.rot_emb[:,:seq_len]


In [5]:
# @title PerceiverIO
# https://github.com/lucidrains/perceiver-pytorch/blob/main/perceiver_pytorch/perceiver_io.py
# https://arxiv.org/pdf/2107.14795
import torch
from torch import nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def zero_module(module):
    for p in module.parameters():
        p.detach().zero_()
    return module

class Attention(nn.Module):
    # def __init__(self, d_model, cond_dim=None, n_heads=None, d_head=8, dropout=0.): # .1
    def __init__(self, query_dim, cond_dim=None, n_heads=8, d_head=64, drop=0):
        super().__init__()
        d_model = d_head * n_heads
        self.d_head, self.n_heads = d_head, n_heads
        # self.n_heads = d_model // d_head
        # self.d_head = d_model // n_heads
        self.cond_dim = cond_dim
        self.q = nn.Linear(query_dim, d_model, bias=False)
        self.kv = nn.Linear(cond_dim or query_dim, 2*d_model, bias=False)
        # self.lin = nn.Linear(d_model, d_model)
        self.lin = zero_module(nn.Linear(d_model, d_model))
        self.drop = nn.Dropout(drop) # indp before q,k,v; after linout
        self.scale = self.d_head**-.5

    def forward(self, x, cond=None, mask=None): # [batch, T, d_model]=[batch, h*w, c], [batch, num_tok, cond_dim], [batch,T]
        if self.cond_dim==None: cond=x # is self attn
        q = self.q(x).unflatten(-1, (self.n_heads, self.d_head)).transpose(1, 2) # [batch, T, d_model] -> [batch, n_heads, T, d_head]
        k, v = self.kv(cond).unflatten(-1, (self.n_heads, 2*self.d_head)).transpose(1, 2).chunk(2, dim=-1) # [batch, n_heads, T/num_tok, d_head]
        # q @ k.T # [qlen,d]@[d,klen]=[qlen,klen]
        # q-> num_latent # k->seq_len
        if mask is not None: mask = mask.unsqueeze(1)
        # print('attn fwd',q.shape, k.shape, v.shape)
        out = F.scaled_dot_product_attention(q, k, v, attn_mask=mask, dropout_p=0) # F->mask # [batch, len_q, len_v]
        out = out.transpose(1, 2).flatten(2)
        return self.drop(self.lin(out)) # [batch, T, d_model]


class AttentionBlock(nn.Module):
    # def __init__(self, d_model, cond_dim=None, d_head, ff_dim=None, dropout=0.):
    def __init__(self, d_model, n_heads ,cond_dim=None, ff_dim=None, drop=0):
        super().__init__()
        self.d_model = d_model
        self.cond_dim = cond_dim
        self.norm1 = nn.RMSNorm(d_model) # LayerNorm RMSNorm
        if cond_dim!=None: self.norm2 = nn.RMSNorm(cond_dim)
        self.drop = nn.Dropout(drop)
        self.attn = Attention(d_model, cond_dim, n_heads=n_heads, d_head=d_model//n_heads)
        act = nn.ReLU()
        if ff_dim==None: ff_dim=d_model*4
        self.ff = nn.Sequential(
            nn.RMSNorm(d_model), nn.Linear(d_model, ff_dim), act,
            nn.RMSNorm(ff_dim), zero_module(nn.Linear(ff_dim, d_model))
            # nn.RMSNorm(d_model), act, nn.Linear(d_model, ff_dim),
            # nn.RMSNorm(ff_dim), act, zero_module(nn.Linear(ff_dim, d_model))
        )

    def forward(self, x, cond=None, mask=None): # [b,c,h,w], [batch, num_tok, cond_dim], [batch,T]
        # print('attblk fwd', x.shape, cond.shape if cond!=None else None, mask.shape if mask!=None else None)
        if self.cond_dim==None: x = x + self.attn(self.norm1(x), mask)
        else: x = x + self.attn(self.norm1(x), self.norm2(cond), mask) # maybe no res for decoder
        x = x + self.ff(x) # maybe no ff for decoder?
        return x


import inspect
class Seq(nn.Sequential):
    def __init__(self, *args):
        super().__init__(*args)
        for layer in self:
            params = inspect.signature(layer.forward).parameters.keys()
            layer._fwdparams = ','.join(params)

    def forward(self, x, cond=None, mask=None):
        for layer in self:
            args = [x]
            if 'cond' in layer._fwdparams: args.append(cond)
            if 'mask' in layer._fwdparams: args.append(mask)
            x = layer(*args)
        return x

model = Attention(64)



In [ ]:
# @title Attention with kvcache window
import torch
import torch.nn as nn
from torch.nn import functional as F

class Attention(nn.Module):
    # def __init__(self, d_model, cond_dim=None, n_heads=None, d_head=8, dropout=0.): # .1
    def __init__(self, query_dim, cond_dim=None, n_heads=8, d_head=64, drop=0, w=128):
        super().__init__()
        d_model = d_head * n_heads
        self.d_head, self.n_heads = d_head, n_heads
        # self.n_heads = d_model // d_head
        # self.d_head = d_model // n_heads
        self.cond_dim = cond_dim
        self.q = nn.Linear(query_dim, d_model, bias=False)
        self.kv = nn.Linear(cond_dim or query_dim, 2*d_model, bias=False)
        # self.lin = nn.Linear(d_model, d_model)
        self.lin = zero_module(nn.Linear(d_model, d_model))
        self.drop = nn.Dropout(drop) # indp before q,k,v; after linout
        self.scale = self.d_head**-.5
        # self.cache = {'x':None, 'xq':None, 'xk':None, 'xv':None}
        self.cache = {'x':None, 'q':None, 'kv':None}
# gen: xqkv, patch: xkv ~pq, gen: xqkv pvv
# so save xqkv
        self.w = w
        self.midx = torch.arange(w, dtype=torch.int32).repeat(t,1) + torch.arange(1-w, t-w+1, dtype=torch.int32).unsqueeze(-1) # [t,w,1]
        self.scale = self.d_head**-.5

    def forward(self, x, cond=None, mask=None): # [b,t,d], [batch, num_tok, cond_dim], [batch,T]
        if self.cond_dim==None: cond=x # is self attn
        # m = torch.rand_like(x)<.00001
        # if self.cache['x']!=None and x.shape==self.cache['x'].shape and (x[m]==self.cache['x'][m]).all():
        if self.cache['x']!=None and x.shape==self.cache['x'].shape and (x[0,0,0]==self.cache['x'][0,0,0]).all():
            # print('cache')
            q = self.cache['q']
            if cond==None: kv = self.cache['kv']
            else: kv = self.kv(cond).unflatten(-1, (self.n_heads, 2*self.d_head)).transpose(1, 2)#.chunk(2, dim=-1) # [batch, n_heads, T/num_tok, d_head]
        else:
            # print('no')
            q = self.q(x).unflatten(-1, (self.n_heads, self.d_head)).transpose(1, 2) # [batch, T, d_model] -> [batch, n_heads, T, d_head]
            kv = self.kv(cond).unflatten(-1, (self.n_heads, 2*self.d_head)).transpose(1, 2)#.chunk(2, dim=-1) # [batch, n_heads, T/num_tok, d_head]
            self.cache = {'x':x, 'q':q, 'kv':kv}
        # q @ k.T # [qlen,d]@[d,klen]=[qlen,klen]
        # q-> num_latent # k->seq_len
        if mask is not None: mask = mask.unsqueeze(1)
        # # if mask != None: attn = attn.masked_fill(mask[:, None, :, None], -torch.finfo(attn.dtype).max) # [batch,T]->[batch,1,T,1]
        # if mask != None: attn = attn.masked_fill(mask.unsqueeze(1), -torch.finfo(attn.dtype).max) # [b,t,t]->[b,1,t,t]

        # print('attn fwd',q.shape, k.shape, v.shape)
        b,t,d = x.shape
        # print(min(self.w,t))
        print(self.w,t)
        midx = self.midx[:t,-min(self.w,t):] # [t,w,1]
        kv = kv[torch.arange(b)[:,None,None,None], torch.arange(h)[None,:,None,None], midx[None,None,...]]
        mk, mv = kv.chunk(2, dim=-1) # [b,h,t,w,d]

        attn = q.unsqueeze(3) @ mk.transpose(-2,-1) * self.scale # [b,h,t,1,d] @ [b,h,t,d,w] = [b,h,t,1,w]
        attention = torch.softmax(attn, dim=-1) # [b,h,t,1,w]
        out = (self.drop(attention) @ mv).squeeze(-2) # [b,h,t,1,w]@[b,h,t,w,d]=[b,h,t,1,d]

        out = out.transpose(1, 2).flatten(2)
        return self.drop(self.lin(out)) # [batch, T, d_model]

dim = 512
model = Attention(dim) # 257 ms
x = torch.rand(64,100,dim)
out = model(x)
print(out.shape)


In [7]:
# @title entropy -> mask
# https://github.com/facebookresearch/blt/blob/main/bytelatent/data/patcher.py
import torch
from torch.nn import functional as F

def entropy(scores): # [batch, seq_len, vocab] # Shannon entropy; input is raw logits
    log_probs = F.log_softmax(scores, dim=-1) # log(softmax(x))
    probs = torch.exp(log_probs)
    # print('in etp', log_probs, probs, p_log_p)
    entropy = -(probs*log_probs).sum(dim=-1)
    return entropy # [batch, seq_len]

# def patch_start_mask_global_and_monotonicity(entropies, t, t_add=0): # [batch, seq_len]
def patchify(entropies, t, t_add=0): # [batch, seq_len]
    bs, seq_len = entropies.shape
    if seq_len == 0: return entropies > t # threshold
    mask = torch.zeros_like(entropies, dtype=torch.bool)
    mask[:, 0] = True
    differences = entropies[:, 1:] - entropies[:, :-1]
    # Calculate conditions for all elements except the first one in each sequence
    # condition = (differences > t_add) & (entropies[:, 1:] > t) & (~mask[:, :-1])
    # print('patchify',differences)
    condition = differences > t_add
    mask[:, 1:] = condition
    return mask # [batch, seq_len] # T at start of each patch

def padded_block_mask(mask): # [b,t]
    B, N = mask.shape
    block_id = mask.cumsum(dim=1) - 1          # (B, N)
    K = mask.sum(dim=1) # [b] # num blocks per sample
    N = max(K)
    offset = (N - K).unsqueeze(1) # (B,1)
    row_idx = block_id + offset # (B, N) # shift blocks to the end
    i = torch.arange(N, device=mask.device)[None,:,None] # [1,N,1] # broadcast compare: for each batch b, out[b,i,j] = (i == row_idx[b,j])
    out = i == row_idx.unsqueeze(1)                            # (B, N, N)
    return out # [batch ,num_patch, seq_len]

def block_diag(mask): # [b,t]
    """Convert a 1D bool mask to a block diagonal matrix, where each `True` starts a new block.
    mask = [1,0,1,1] ->
    [[1,1,0,0],
     [1,1,0,0],
     [0,0,1,0],
     [0,0,0,1]]"""
    block_ids = mask.cumsum(1) # Assign a unique group index to each block e.g., [0, 0, 1, 2]
    row_ids = block_ids.unsqueeze(2)  # [b,t,1]
    col_ids = block_ids.unsqueeze(1)  # [b,1,t]
    out = row_ids == col_ids # Compare row and col IDs — same block → True
    return out # [b,t,t]

x = torch.rand(2,3,4) # [b,t,d]
x = torch.tensor([-1,1,-1,-1]).float().repeat(2,3,1)*3 # 1.3863

print('max:',-torch.tensor(1/4).log())
# min=0

entropies = entropy(x)
print(entropies)
# mask = patchify(entropies, 1.) # [b, t]
# print(mask)
# # mask = padded_block_mask(mask) # [b ,num_patch, t]
# # print(mask)
# mask = block_diag(mask) # [b ,t, t]
# print(mask)
# xx = x.unsqueeze(1) *mask.unsqueeze(-1) # [b ,num_patch, t, d]
# print(xx.shape)
# print(xx.mean(2).shape)
# print(xx.mean(2)) # [b ,num_patch, d]

# [b,t,t]

# x = torch.rand(2,5,) # [b,t,d]
# mask=(x>.7)
# out = mask.unsqueeze(1) | mask.unsqueeze(2) # [b,t]->[b,t,t]
# print(out)


max: tensor(1.3863)
tensor([[0.0517, 0.0517, 0.0517],
        [0.0517, 0.0517, 0.0517]])


In [8]:
# @title BLTlayer transformer fsq noloss save
import torch
import torch.nn as nn
from torch.nn import functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"

class BLTlayer(nn.Module):
    def __init__(self, in_dim, d_model=None, quant=6, num_layers=1):
        super().__init__()
        d_model = d_model or in_dim
        self.pos_enc = RoPE(d_model, base=100) # 10000
        # self.latent = nn.Parameter(torch.randn(1,1,d_model)*.02) # init
        self.perceiver = Seq(*[AttentionBlock(d_model, n_heads=4, cond_dim=d_model) for _ in range(num_layers)])
        self.lin = nn.Linear(d_model, d_model)
        self.patch_enc = nn.Parameter(torch.randn(1,1,d_model)*.02)
        self.conv_emb = nn.Conv1d(d_model, d_model, 1)
        # self.conv_emb = nn.Conv1d(d_model, d_model, 7, 7//2)
        self.vq = codequant(levels = d_model*[quant])
        self.max_etp = 1 - torch.tensor([1-1/(quant-1)], device=device).asin()/torch.pi*2 # cos max etp
        # self.max_etp = self.max_etp**2
        # self.max_etp = 1/(quant-1) # unif max etp
        # self.max_etp = -torch.tensor(1/quant).log() # max shannon etp

    def forward(self, x, cond=None, mask=None): # [b,t,d], [b,t,t]
        x = self.pos_enc(x)
        b,t,d = x.shape
        # if mask==None: mask = torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(b,1,1) # F->mask # for F.scaled_dot_product_attention
        # print('BLTlayer',x.shape, cond.shape if cond!=None else None, mask.shape if mask!=None else None)
        x = self.perceiver(x, cond=cond if cond!=None else x, mask=mask) # [b,t,d], [nlyr,b,d]
        out = self.lin(x)
        return out # [b,t,d]

    def to_patch(self, x, out=None, entropies=None): # [b,t,d]
        if entropies==None: _, entropies = self.entropy(out) # [b,t] float
        mask = patchify(entropies, 1.) # [b,t] # T at start of each patch, F otherwise
        patch_mask = padded_block_mask(mask) # [b ,num_patch, t]
        # print('lyr to_patch', x.shape, out.shape, patch_mask.shape)

        latent = (x.unsqueeze(1) *patch_mask.unsqueeze(-1)).mean(2) # [b ,num_patch, t, d] -> [b ,num_patch, d]
        # latent = self.latent.repeat(1,mask.shape[1],1) # [b ,num_patch, d]
        # print('lyr to_patch', latent)
        latent = latent + self.patch_enc
        out = self.conv_emb(x.transpose(-2,-1)).transpose(-2,-1) # instead of hashing
        patches = self.forward(latent, cond=out, mask=patch_mask) # [batch, num_patch, latent_dim/d_model]
        # patches = self.vqp(patches)
        b,t = mask.shape
        s = mask.sum(-1)
        mask = torch.arange(t, device=device).repeat(b,1)>=(t-s).unsqueeze(-1)
        mask = mask[:,-s.max():] # [b,num_patch] # T-> can attend

        return patches, mask, patch_mask # [b, num_patch, out_dim], [b,t], [b ,num_patch, t]
# AE: mask->pos_patch?/conv1d? + mask+ latent(mean+*leant) -> patch; pos_patch?+ mask.T + patch(+*learnt?)-> og


# generate one tok unless uncertain ? then patch up?
# gen one only bec blt need to tok
# or blt feed only one tok? nope, should parallelise
# feed all toks bec topatch need out
    # def generate(self, x=None, cond=None, patch=None): # [1,t,d], [1,len_cond,d], [1,1,d]
    def generate(self, x=None, cond=None, patch=None, mask=None, patch_mask=None): # [b,t,d], [b,len_cond,d], [b,len_patch,d], [b,t], [b,t,len_patch]
        # print('lyr gen1 x cond patch mask patch_mask', x.shape, cond.shape if cond!=None else None, patch.shape if patch!=None else None, mask.shape if mask!=None else None, patch_mask.shape if patch_mask!=None else None)
        b,t,d = x.shape
        if cond==None:
            cond = x
            # mask = torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(b,1,1) # no cond => is causal
        # else:
        causal_mask = torch.tril(torch.ones((b,t,cond.shape[1]), dtype=bool, device=device)) # [1,t,n_cond+n_patch] # got cond, all can attend to cond
        if mask!=None: mask = causal_mask & mask.unsqueeze(1) # mask left side, tril is lower left
        # if mask!=None: mask = causal_mask & mask # mask left side, tril is lower left
        else: mask = causal_mask

        if patch!=None:
            cond = torch.cat([cond, patch], dim=1)
            if patch_mask==None:
                patch_mask = torch.zeros((b,t,patch.shape[1]), dtype=bool, device=device)
                patch_mask[:,-1] = True # patch is only for last (aka new) tok
                # patch_mask = torch.ones((b,t,patch.shape[1]), dtype=bool, device=device) # patch is for all toks

            # print('lyr gen1 x cond patch', x.shape, cond.shape, patch.shape, mask.shape, pmask.shape)
            mask = torch.cat([mask, patch_mask], dim=-1) # [b, t, t+len_patch]
        # print('lyr gen1', x.shape, cond.shape, mask.shape)
        out = self.forward(x, cond=cond, mask=mask)
        qout, etp = self.entropy(out)
        return qout, etp
# no need loss. gen one only; if etp high, blt patch_up lvl

    def entropy(self, out):
        qout = self.vq(out)
        etp = (out-qout).abs().mean(-1) # [b,1] float
        # etp = (out-qout).pow(2).mean(-1)
        # etp = entropy(out)
        etp /= self.max_etp # normalise to [0,1]
        return qout, etp


# bltlyr: btd->btd+bpd , gentrain: shift1, artrain:btd+bpd->btd

# in: qunatseq
# fwd: qunatx -> y_
# to_patch: x, y_ -> patches, mask
# quant at patch only? since tok emb quants for x
# need to quant out for to_patch
# for train, gen should mix in results from up, scaled by etp
# fwd, to_patch,


batch=2
d_model = 16
in_dim=128

model = BLTlayer(in_dim, d_model).to(device)
# # model = BLTlayer(d_model).to(device)
# print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683
# optim = torch.optim.AdamW(model.parameters(), 1e-3, (0.9, 0.95)) # 1e-4 #3e-4

x = torch.rand((batch, 7, d_model), device=device)
# out = model(x)
out, entropies = model.generate(x) #
# out, _ = model.generate(x, patch=patch) # patch cond from last tok onwards
# out, _ = model.generate(x[:,-1:], cond=x[:,:-1], patch=patch) # got cond no need to cond on all
# out, _ = model.generate(x[:,-1:], cond=x[:,:-1])
print(out.shape, entropies.shape)
# print('out', out.shape)

patches, mask, patch_mask = model.to_patch(x, entropies=entropies) # [b, num_patch, out_dim], [b,t], [b ,num_patch, t]

print(mask)

print('patches, mask',patches.shape, mask.shape, patch_mask.shape)


patches, _ = model.generate(patches, mask=mask) # upper lvl
# print(out.shape, entropies.shape)
# patches, mask, patch_mask = model.to_patch(x, entropies=entropies)
# print('patches, mask',patches.shape, mask.shape)
print(x[:,-1:].shape, x[:,:-1].shape, patches[:,-1:].shape)
# out_, _ = model.generate(x[:,-1:], cond=x[:,:-1], patch=patches[:,-1:]) # got cond no need to cond on all
out_, _ = model.generate(x, patch=patches, patch_mask=patch_mask.transpose(-2,-1))
# out_, _ = model.generate(x, patch=patches) # got cond no need to cond on all
# g = -entropies[:,-1:] # certainty
# print(g)
# out = g*out[:,-1:] + (1-g)*out_[:,-1:]

# patch_mask

# out, mask_ = left_pad_tensor(torch.cat([x,y],dim=1), torch.cat([mask, mask],dim=1))

# print(patches.shape)
# print(out)


torch.Size([2, 7, 16]) torch.Size([2, 7])
tensor([[False,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True]], device='cuda:0')
patches, mask torch.Size([2, 5, 16]) torch.Size([2, 5]) torch.Size([2, 5, 7])
torch.Size([2, 1, 16]) torch.Size([2, 6, 16]) torch.Size([2, 1, 16])


In [9]:
# @title BLT me fsq noloss
import torch
import torch.nn as nn
from torch.nn import functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"

class BLT(nn.Module):
    def __init__(self, d_model, num_layers=1):
        super().__init__()
        self.layer = nn.ModuleList([BLTlayer(d_model, d_model, quant=6*2**i) for i in range(num_layers)])

    def forward(self, x, mask=None): # [b,t,d], [b,t]
        blocks = []
        pmasks = []
        masks = []
        outs = []
        # etps = []
        for i, level in enumerate(self.layer[:-1]):
            blocks.append(x)
            masks.append(mask)
            out, entropies = level.generate(x, mask=mask)
            # out, entropies, kvcache = level.generate(x, mask=mask)
            outs.append(out)
            # etps.append(entropies)
            x, mask, patch_mask = level.to_patch(x, entropies=entropies)
            # x, mask, patch_mask = level.to_patch(x, entropies=entropies, kvcache)
            # print('blt_fwd_down', x.shape, mask.shape, patch_mask.shape)
            pmasks.append(patch_mask)
        out, _ = self.layer[-1].generate(x, mask=mask)
        out, _ = self.layer[-1].generate(x, mask=mask)
        x = torch.cat([x[:,:1], out], dim=1)
        for i, level in enumerate(reversed(self.layer[:-1])):


            # out[:,-1:] * entropies[:,-1:]
            v = blocks[-i-1] #+ outs
            # print('blt_fwd_up', v.shape, x.shape, masks[-i-1].shape, pmasks[-i-1].shape)
            # print('blt_fwd_up', pmasks[-i-1])

            patch_mask = pmasks[-i-1]
            b,p,t = patch_mask.shape
            add_mask = torch.zeros((b,1,t), dtype=bool, device=device)
            add_mask[...,-1] = True # patch is only for last (aka new) tok
            patch_mask = torch.cat([patch_mask, add_mask], dim=1)
            # print('blt_fwd_up', v.shape, x.shape, masks[-i-1].shape if masks[-i-1]!=None else None, patch_mask.shape)

            out, _ = level.generate(v, patch=x, mask=masks[-i-1], patch_mask=patch_mask.transpose(-2,-1))
            # out, _ = level.generate(v, patch=x, mask=masks[-i-1], patch_mask=patch_mask.transpose(-2,-1), kvcache)
            # x = torch.cat([v,x[:,-1:]], dim=1)
            # # x = torch.cat([x, blocks[-i-1]*2**-.5], dim=1)
            # x = torch.cat([x, blocks[-i-1]*self.skip_scale**(len(self.up_list)-i)], dim=1) # https://arxiv.org/pdf/2310.13545
            # g = -entropies[:,-1:] # certainty
            # out = g*out[:,-1:] + (1-g)*out_[:,-1:]
            # etp = -etps[-i-1].unsqueeze(-1) # certainty
            # # print(etp.shape, out.shape, outs[-i-1].shape)
            # x = etp*out + (1-etp)*outs[-i-1]
            # x = out + outs[-i-1]
            x = .5*(out + outs[-i-1])
            # x = out*2**-.5 + outs[-i-1]
            x = torch.cat([v[:,:1], x], dim=1)
        return x[:,1:]


# pos_all, entropy mask;
# AE: mask->pos_patch?/conv1d? + mask+ latent(mean+*leant) -> patch; pos_patch?+ mask.T + patch(+*learnt?)-> og
# gen train: mse(x[1:],out[:-1])
# gen inference: x cond=x, cond=(x+patch(+*learnt?)), if entropy/bos, pop patch and regen

# # fsq: dont realize tokens in layer
# layer:
# posemb
# x less quants patch more quants
# quant after linout
# # quant before linout


d_model=16
model = BLT(d_model, num_layers=3).to(device)
x = torch.rand(2,13,d_model, device=device)
out = model(x)
# out = model.generate(x)
print(out.shape)
# print(out)
 # [b,t,d], [b,len_cond,d], [b,len_patch,d], [b,t], [b,t,len_patch]

torch.Size([2, 13, 16])


##ghj

In [10]:
# @title HierarchialGPT

class HierarchialGPT(nn.Module):
    def __init__(self, in_dim, d_model, out_dim=None, num_layers=1):
        super().__init__()
        # self.tok_emb = nn.Embedding(in_dim, d_model)
        self.tok_emb = TTEmbedding([29, 1733], [64,1], rank=min(d_model,256))
        # self.pos_enc = RoPE(d_model, base=10000)
        self.blt = BLT(d_model, num_layers=num_layers)
        # self.out = nn.Linear(d_model, out_dim or in_dim)
        # self.out = lambda x: x @ self.tok_emb.weight.T  # weight tying
        self.out = lambda x: x @ self.tok_emb.weight().T  # weight tying

    #     self.apply(self.init_weights)
    # import math
    # def init_weights(self, m): # https://openreview.net/pdf?id=lkRjnNW0gb
    #     if isinstance(m, nn.Linear):
    #         # W ~ N(0, ( 1/ (sqrt(n_in) + sqrt(n_out)) )^2 )
    #         # want std = 1/ (sqrt(n_in) + sqrt(n_out))
    #         # n_in, n_out = module.weight.shape[0], module.weight.shape[1]
    #         n_in, n_out = m.weight.shape
    #         torch.nn.init.normal_(m.weight, std=1/(math.sqrt(n_in)+math.sqrt(n_out)))
    #         if m.bias is not None:
    #             nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.tok_emb(x)
        # x = self.pos_enc(x)
        x = self.blt(x)
        x = self.out(x)
        return x

vocab_size=50
vocab_size=train_loader.dataset.vocab_size#50
d_model, num_layers = 64, 3
model = HierarchialGPT(vocab_size, d_model, num_layers=num_layers).to(device)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683
optim = torch.optim.AdamW(model.parameters(), 1e-3)
x = torch.randint(0,vocab_size, (64,128), device=device)

# x, y = x[:,:-1], x[:,1:]
# x, y = x.to(device), y.to(device)

out = model(x)
print(out.shape)
# # loss = F.cross_entropy(out[:,:-1].flatten(0,-2), x[:,1:].flatten())
# loss = F.cross_entropy(out.flatten(0,1), y.flatten())
# print(loss)
# optim.zero_grad()
# loss.backward()
# optim.step()

# test 5.684650421142578
# Test Loss: 0.0
# this is what cealieqplal to ce ctmedee ss idedo ss tou
#  sst N tno-ulosetor t
# 18 time: 43.02241659164429


404800
torch.Size([64, 128, 50257])


In [20]:
# @title GPT
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class GPT(nn.Module):
    def __init__(self, in_dim, d_model=64, out_dim=None, n_heads=8, n_layers=1, ff_dim=256, dropout=0):
        super().__init__()
        self.d_model = d_model
        # self.pos_enc = PositionalEncoder(d_model, dropout=dropout)
        # self.pos_enc = LearntPosEnc(d_model, dropout=dropout)
        self.pos_enc = RoPE(d_model, base=10000)
        # self.tok_emb = nn.Embedding(in_dim, d_model)
        self.tok_emb = TTEmbedding([29, 1733], [64,1], rank=min(d_model,256))
        # self.encoder = Seq(*[EncoderLayer(d_model, n_heads, ff_dim, dropout) for _ in range(n_layers)])
        self.encoder = Seq(*[AttentionBlock(d_model, n_heads=4, cond_dim=d_model) for _ in range(n_layers)])
        # self.out = nn.Linear(d_model, out_dim)
        self.out = lambda x: x @ self.tok_emb.weight().T  # weight tying

    def forward(self, x, mask=None): # [b,t], [b,t,t]
        x = self.pos_enc(self.tok_emb(x))
        # print('gpt fwd', x.shape)
        # x = self.encoder(x, mask=mask)
        x = self.encoder(x, x, mask=mask) # [b,t,d], [nlyr,b,d]
        x = self.out(x)
        return x

input_size = num_classes = 50
# input_size = num_classes = train_data.vocab_size#50
# print(train_data.vocab_size)
# model = GPT(input_size, d_model=512, out_dim=num_classes, n_layers=6).to(device)
model = GPT(input_size, d_model=64, out_dim=num_classes, n_layers=3).to(device)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683
optim = torch.optim.AdamW(model.parameters(), 1e-3)

# batch 64, seqlen 100
# dim 64, nlyrs 1: 5.0gb
# 512, 1, 6.3, 95sec
# dim 512, 6 lyrs, 8.7 gb, 125 sec, 59393105 params
# dim 768, 12 lyrs, 12.4 gb, 257 sec, 110318161 params


x = torch.randint(0, input_size, (2, 5), device=device)
out = model(x)
print(out.shape)
# print(out)

# strain 2.1297216415405273
# Test Loss: 5.68384530884879
# this is what gl han ast atot t mad indswonshol e nlo refes the the laley emas
# 0 time: 10.823431253433228

# strain 1.3502650260925293
# Test Loss: 8.358739376068115
# this is what the signed and commerly the meeticly base finance forformance

# 29 time: 10.74680209159851

# no norm
# strain 2.1280951499938965
# Test Loss: 6.583536461421422
# this is what the coure <unk> cloh adintt myen llig hroute wanseg hipes sustin
# 0 time: 10.436080694198608

# strain 1.3436470031738281
# Test Loss: 10.137133121490479
# this is what americans incentic company 's <unk>
#  bear drop in this each hol
# 29 time: 11.038460969924927


# F.scaled_dot_product_attention
# strain 2.116980791091919
# Test Loss: 6.470040246418544
# this is what inds fougsterined a qouep omal <unk> e lon waby ine <unk> beack>
# 0 time: 9.980751752853394
# strain 1.373486042022705
# Test Loss: 8.955048438480922
# this is what legaision can said realizive univerence antinge chairman <unk> t
# 29 time: 10.000610589981079 9.92842394510905


379648
torch.Size([2, 5, 50257])


In [ ]:
# https://github.com/xbeat/Machine-Learning/blob/main/Python%20KV%20Caching%20Efficient%20Data%20Storage%20and%20Retrieval.md

In [ ]:
# bottom lyrs higher dim
# bottom emb mlp satisfiability top layers

# current status, job, personality, points of view
# dreams, aspirations
# expectations, family

# scicentre, jap garden
# aquarium beach catcafe dinner equestrian  museum/movie nature picnic/pool shopping zoo

In [ ]:
# optim.param_groups[0]['lr'] = 1e-4#

In [11]:
# @title wandb
!pip install -q wandb
import wandb # https://docs.wandb.ai/quickstart
wandb.login(key='487a2109e55dce4e13fc70681781de9f50f27be7')
try: run.finish()
except NameError: pass
run = wandb.init(project="gpt", config={"model": "res18",}) # violet

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bobdole to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# @title perplexity
import torch
import torch.nn as nn
import torch.nn.functional as F
# https://www.comet.com/site/blog/perplexity-for-llm-evaluation/

def Perplexity(logits, target): # [b,t,vocab_size], [b,t]
    log_probs = F.log_softmax(logits, dim=-1)
    nll = -log_probs.gather(dim=-1, index=target.unsqueeze(-1)).squeeze(-1) # [b,t]
    perplexity = nll.mean().exp()
    return perplexity

# logits = torch.randn(2, 4, 10)
# target = torch.tensor([[1, 2, 3, 4], [4, 3, 2, 1]])

# perplexity = Perplexity(logits, target)
# # perplexity = Perplexity(logits[:,:-1], y[:,1:])
# print(f'Perplexity: {perplexity}')


# @title train test generate
import torch
from torch.nn import functional as F
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
scaler = torch.GradScaler()

def strain(model, dataloader, optimizer, scheduler=None): # train function with automatic mixed precision
    model.train()
    for i, x in enumerate(dataloader):
        x, y = x[:,:-1], x[:,1:]
        x, y = x.to(device), y.to(device)

        with torch.autocast(device_type=device, dtype=torch.bfloat16):
            # causal_mask = torch.ones(x.size(1), x.size(1), dtype=bool, device=device).tril(diagonal=0).repeat(x.shape[0],1,1) # for F.scaled_dot_product_attention
            # # causal_mask = ~torch.ones(x.size(1), x.size(1), dtype=bool, device=device).tril(diagonal=0).repeat(x.shape[0],1,1)
            # logits = model(x, mask=causal_mask) #output = [batch size, trg len - 1, output dim]
            logits = model(x) #output = [batch size, trg len - 1, output dim]
            # logits, _ = model(x) # rnn
            loss = F.cross_entropy(logits.flatten(0,1), y.flatten()) # [b*t,d], [b*t]

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        # scaler.unscale_(optim)
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        scaler.step(optimizer)
        scaler.update()
        # if scheduler is not None: scheduler.step()
        if i % 100 == 0: print("strain",loss.item())
            # perplexity = Perplexity(logits.detach(), y).item()
        try: wandb.log({"train loss": loss.item()/len(y)})
        except NameError: pass

# def test(loader, model):
#     model.eval()
#     total_loss = 0
#     for i, (x, y) in enumerate(loader):
#         x, y = x.to(device), y.to(device)
#         with torch.no_grad():
#             logits = model(x)
#             # logits, _ = model(x) # rnn
#             perplexity = Perplexity(logits[:,:-1], y[:,1:])
#         loss = F.cross_entropy(logits.flatten(0,-2), y.flatten()) # [batch*seq_len, vocab_size], [batch*seq_len]
#         total_loss+=loss.item()
#         # if i % 100 == 0: print("test",loss.item())
#         if i % 100 == 0: print("test",loss.item(),"ppty",perplexity.item())
#         try: wandb.log({"test loss": loss.item()/len(y)})
#         except NameError: pass
#     return total_loss / len(loader)

def generate(model, context, max_steps=64, temperature=1):
    x = encode(context)#.to(device)
    model.eval()
    for n in range(max_steps):
        with torch.no_grad():
            output = model(x)
            # output, hidden = model(x, hidden) # rnn
        # print('generate', output.shape, hidden.shape)
        # hidden = hidden[:, -1, :].unsqueeze(1) # RNN/GRU
        output = output[:, -1] # get logit for last character
        output = output/temperature
        output = F.softmax(output, dim=-1) # vocab_size to char
        ix = torch.multinomial(output, num_samples=1) # rand sample by output distribution
        x = torch.cat((x, ix), dim=1)
    completion = decode(x.squeeze(0))
    return completion

# import time
# start = begin = time.time()
for i in range(1):
# for i in range(30):
    # train_loss = strain(model, train_loader, optim, scheduler=None)
    strain(model, train_loader, optim, scheduler=None)
    # test_loss = test(test_loader, model)
    # print('Test Loss:', test_loss)
    # print(generate(model, "this is what"))
    # print(i, 'time:',time.time() - start, (time.time()-begin)/(i+1))
    # start = time.time()




strain 13.153278350830078
strain 8.625616073608398
strain 7.532129287719727
strain 7.635157585144043
strain 7.318344593048096
strain 7.13112211227417
strain 7.953535556793213
strain 7.314005374908447
strain 7.358240127563477
strain 7.116394519805908
strain 7.228402137756348
strain 7.046687602996826
strain 6.697092533111572
strain 7.242772102355957
strain 7.065131187438965
strain 7.033816337585449
strain 7.072269916534424
strain 6.835287570953369
strain 6.56411075592041
strain 6.651105880737305
strain 6.974645614624023
strain 6.841508388519287
strain 7.360500812530518
strain 8.260704040527344
strain 7.255819797515869
strain 7.79181432723999
strain 6.3285603523254395
strain 6.8741278648376465
strain 6.494027614593506
strain 7.289155960083008
strain 6.912679672241211
strain 7.111609935760498
strain 6.812265396118164
strain 6.852604389190674
strain 6.8135504722595215
strain 6.717615604400635
strain 6.746161937713623
strain 6.72939920425415
strain 7.120828628540039
strain 6.625032901763916


In [ ]:
print(generate(model, "this is what"))

this is what rapidly on the crime article and CLIImage: Photograph.

[governmental as well as well-AlnotWsinkser fell through the DA namely Smith else, of the lowpower drawn from alarm. In the 52]
Still, Carops.
Recability's infrastructure.


9][111:


In [ ]:
# @title save/load
from google.colab import drive
drive.mount('/content/drive')
folder='/content/drive/MyDrive/jepa/'

# # modelsd, optimsd = torch.load(folder+'gpt.pkl', map_location=device).values()
# modelsd, optimsd = torch.load('gpt.pkl', map_location=device).values()
# model.load_state_dict(modelsd, strict=False)
# optim.load_state_dict(optimsd)

# device = "cuda" if torch.cuda.is_available() else "cpu"
# modelsd = torch.load(folder+'gpt.pkl', map_location=device)['model']#.values()
# # print(modelsd)
# model.load_state_dict(modelsd, strict=False)


Mounted at /content/drive


In [ ]:
checkpoint = {'model': model.state_dict(), 'optimizer': optim.state_dict()}
# checkpoint = {'model': model.state_dict()}
torch.save(checkpoint, folder+'blt.pkl')
# # torch.save(checkpoint, 'gpt.pkl')

## drawer

In [ ]:
# @title data
import requests
# url="https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt" # https://www.tensorflow.org/text/tutorials/text_generation
url="https://raw.githubusercontent.com/wojzaremba/lstm/master/data/ptb.train.txt" # train test valid # https://pytorch.org/text/stable/datasets.html#penntreebank
out=requests.get(url)
with open("data.txt", "wb") as f:
    f.write(out.content)
text = open("data.txt", 'rb').read().decode(encoding='utf-8')

url="https://raw.githubusercontent.com/wojzaremba/lstm/master/data/ptb.test.txt" # train test valid # https://pytorch.org/text/stable/datasets.html#penntreebank
out=requests.get(url)
with open("data_.txt", "wb") as f: f.write(out.content)
test_text = open("data_.txt", 'rb').read().decode(encoding='utf-8')

# print(len(text))
# print(text[000:1000])
# data = ''.join(text)
# chars = sorted(list(set(data)))
# print(chars)


In [ ]:
# @title char dataloader
# https://github.com/Sam-Armstrong/tinyGPT/blob/main/Training.py
# https://colab.research.google.com/github/karpathy/minGPT/blob/master/play_char.ipynb
# https://github.com/karpathy/nanoGPT
import torch
import torch.nn as nn
from torch.utils.data import Dataset

class CharDataset(Dataset): # https://github.com/karpathy/minGPT
    def __init__(self, raw_data, seq_len):
        data = ''.join(raw_data)
        chars = sorted(list(set(data)))
        self.vocab_size = len(chars) # 283
        self.stoi = {ch:i for i,ch in enumerate(chars)}
        self.itos = {i:ch for i,ch in enumerate(chars)}
        self.data = self.data_process(data) # list of int
        self.seq_len = seq_len

    def data_process(self, data): # str 10780437
        return torch.tensor([self.stoi.get(c) for c in data]) # list of int 4570571 # stoi.get(c,UNK_IDX)

    def __len__(self):
        # return len(self.data) - self.seq_len
        return len(self.data)//(self.seq_len+1)

    def __getitem__(self, idx):
        # dix = self.data[idx:idx + self.seq_len + 1]
        dix = self.data[idx*(self.seq_len+1) : (idx+1)*(self.seq_len+1)]
        x, y = dix[:-1], dix[1:]
        return x, y


seq_len = 100 # 128
train_data = CharDataset(text, seq_len) # one line of poem is roughly 50 characters
test_data = CharDataset(test_text, seq_len) # one line of poem is roughly 50 characters
from torch.utils.data.dataloader import DataLoader
batch_size = 64 #512
train_loader = DataLoader(train_data, shuffle = True, pin_memory = True, batch_size = batch_size, num_workers = 2) # num_workers = 4
test_loader = DataLoader(test_data, shuffle = True, pin_memory = True, batch_size = batch_size, num_workers = 0)


def encode(context): return torch.tensor([train_data.stoi.get(c) for c in context], device=device).unsqueeze(0)
def decode(x): return ''.join([train_data.itos[int(i)] for i in x])
# for x,y in train_loader:
#     break


In [ ]:
# @title RNN pytorch
import torch
from torch import nn
import torch.nn.functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"

class RNN(nn.Module):
    def __init__(self, in_dim, d_model, out_dim=None, num_layers=1):
        super().__init__()
        if out_dim is None: out_dim = in_dim
        self.num_layers = num_layers
        self.d_model = d_model
        self.rnn = nn.GRU(d_model, d_model, num_layers, batch_first=True)
        self.fc = nn.Linear(d_model, out_dim)
        self.emb = nn.Embedding(in_dim, d_model)

    def forward(self, x, h=None):
        x = self.emb(x)
        if h is None: h0 = torch.zeros((self.num_layers, x.size(0), self.d_model), device=device)
        else: h0 = h
        # print(x.shape, h0.shape)
        out, h = self.rnn(x, h0)
        # out = out[:, -1, :] # out: (n, 128)
        out = self.fc(out) # out: (n, 10)
        return out, h # [b,t,out_dim], [num_layers,b,d_model]


hidden_size = 64
num_layers = 3
input_size = num_classes = 50#train_dataset.vocab_size#65

model = RNN(input_size, hidden_size, num_classes, num_layers).to(device)
# print(model)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683
optim = torch.optim.AdamW(model.parameters(), lr=1e-3)

x = torch.randint(0, 10, (32,25), device=device) # [b,t]
out, h = model(x)
print(out.shape, h.shape)

# this is what said compaticated 's steel a debt rate up taps imposed house in
# 27 time: 9.265877962112427 9.091531344822474
# Test Loss: 8.320224022519762
# this is what indeed sancal quite pralic <unk> over third market that of he an
# 28 time: 8.423768997192383 9.068516492843628
# Test Loss: 8.308711514956709
# this is what banking the mr. $ N ta curres problems
#  delose tax come a judge

# this is what cents a share
#  bargain when a lot out the company 's times <unk
# 27 time: 12.4160475730896 12.368791265147072
# Test Loss: 8.183734503345214
# this is what holders unstock in tokyo-propernatives have sales
#  ast worked s
# 28 time: 12.395569086074829 12.36972609881697
# Test Loss: 8.16899547715118
# this is what purchamed of job
#  the wrong begy consent of despite of the sept
# 29 time: 12.502739667892456 12.374171813329061


81330
torch.Size([32, 25, 50]) torch.Size([3, 32, 64])


In [ ]:
# @title FSQ me
import torch
import torch.nn as nn
import torch.nn.functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"

def ste_round(x): return x.round().detach() + x - x.detach()

class FSQ(nn.Module):
    def __init__(self, levels):
        super().__init__()
        self.levels = torch.tensor(levels, device=device)
        self.basis = torch.cumprod(torch.tensor([*levels[1:], 1], device=device).flip(-1), dim=0).flip(-1)
        self.half_width = (self.levels-1)/2
        self.codebook_size = torch.prod(self.levels).item()
        # self.codebook = self.indexes_to_codes(torch.arange(self.codebook_size, device=device))
    @property
    def codebook(self): return self.indexes_to_codes(torch.arange(self.codebook_size, device=device))

    def forward(self, z):
        offset = (self.levels+1) % 2 /2 # .5 if even, 0 if odd
        # bound = (F.sigmoid(2*z)-1/2) * (self.levels-beta) + offset
        bound = (F.tanh(z)/2) * (self.levels-1) + offset
        quantized = ste_round(bound)
        return (quantized-offset) / self.half_width # split [-1,1]

    def codes_to_indexes(self, zhat):
        zhat = (zhat + 1) * self.half_width
        return (zhat * self.basis).sum(axis=-1).round().int()

    def indexes_to_codes(self, indices):
        indices = indices.unsqueeze(-1)
        codes = torch.remainder(indices//self.basis, self.levels)
        return codes / self.half_width - 1

    def codes_to_threshold(self, zhat):
        offset = (self.levels+1) % 2 /2 # .5 if even, 0 if odd
        # quantized = zhat * self.half_width + offset
        bound = zhat * self.half_width# + offset
        # bound = ste_round(quantized)
        # print(bound)
        # z = ((bound-offset) / (self.levels-0) * 2).atanh()
        z = (bound / (self.levels-0) * 2).atanh()

    def threshold(self, z):
        offset = (self.levels+1) % 2 /2 # .5 if even, 0 if odd
        bound = (F.tanh(z)/2) * (self.levels-1) + offset
        quantized = ste_round(bound)
        return ((quantized-offset) / (self.levels-0) * 2).atanh()

def soft_clamp_relu(x, lower=None, upper=None):
    if lower != None: x = lower+F.relu(x-lower)
    if upper != None: x = upper-F.relu(upper-x)
    return x

    def quant_code(self, z): # round code to nearest code
        offset = (self.levels+1) % 2 /2 # .5 if even, 0 if odd
        bound = zhat * self.half_width + offset
        quantized = ste_round(bound)
        zhat = (quantized-offset) / self.half_width # split [-1,1]
        return soft_clamp_relu(zhat, -1, 1)


fsq = FSQ(levels = [128,4,3,2])
# fsq = FSQ(levels = 4*[6])
# fsq = FSQ(levels = 16*[6])
# print(fsq.codebook)
batch_size, seq_len = 2, 4
# x = torch.linspace(-5,5,17).repeat(4,1).T # sig need larger variance to reach +-1
x = torch.linspace(-3,3,23).repeat(4,1).T
# x = torch.linspace(-3,3,17).repeat(16,1).T
# x=la
# la = fsq(x)
# print(la)
# lact = fsq.codes_to_indexes(la)
# print(lact)
# z = fsq.threshold(x)
# print(z)
# la = fsq.indexes_to_codes(lact)
# print(la)
# # print(fsq.codebook)
# # print(fsq.codebook())


In [ ]:
# @title test codequant
# zhat = torch.linspace(-3,3,23).repeat(4,1).T
# zhat = torch.linspace(-1,1,970)#.repeat(4,1).T
zhat = torch.linspace(-1.2,1.2,970)#.repeat(4,1).T

# print((zhat - zhat.detach())==0)
# print(soft_clamp_relu(zhat,-1,1)==1)

# def soft_clamp_relu(x, lower=None, upper=None):
#     if lower != None: x = lower+F.relu(x-lower)
#     if upper != None: x = upper-F.relu(upper-x)
#     return x

# offset = (fsq.levels+1) % 2 /2 # .5 if even, 0 if odd
# bound = zhat * fsq.half_width + offset
# quantized = ste_round(bound)
# zhat = (quantized-offset) / fsq.half_width # split [-1,1]
# zhat = soft_clamp_relu(zhat, -1, 1)
# print(zhat)

# zhat = (zhat *torch.pi/2).cos()
# zhat = zhat *(zhat *torch.pi/2).sin().abs()
# print(zhat)

# # y=zhat[:,0]
# # y=zhat
# # y=zhat.acos()/torch.pi*2-1
# y=zhat.asin()/torch.pi*2
# # print(y)
# # y = ((y+1)*torch.pi/2).cos()
# y = (y*-torch.pi/2).sin()
# # print(y)
# x=torch.ones_like(y)
# import numpy as np
# import matplotlib.pyplot as plt
# plt.hist(y, bins=50, alpha=.5, label='context mask')
# # plt.scatter(y.numpy(), x.numpy())
# # plt.plot(y)
# # plt.plot(new_x, resized_data, label='linear')
# plt.legend(loc='best')
# plt.show()


In [ ]:
# @title left_pad_tensor
import torch

# def right_to_left_pad(x, mask):
#     b,t,d = x.shape
#     # Get lengths of valid tokens
#     lengths = mask.sum(dim=1)  # (B,)

#     # Build index map (B, S)
#     idx = torch.arange(t, device=x.device).unsqueeze(0).repeat(b, 1)  # (B, S)
#     shift = (t - lengths).unsqueeze(1)                                # (B, 1)

#     # Masked index positions: left-shift valid tokens to the right
#     new_idx = torch.full_like(idx, t - 1)  # Fill with final index (for padding)
#     valid_counts = mask.cumsum(dim=1) - 1  # Position within valid region
#     new_pos = shift + valid_counts
#     new_idx[mask] = new_pos[mask]          # Only fill valid positions

#     # Create scattered result
#     out = torch.zeros_like(x)
#     out.scatter_(dim=1, index=new_idx.unsqueeze(-1).repeat(1,1,d), src=x)
#     return out

# def combine_left_padded(x1, mask1, x2, mask2):
#     # Convert both to right-padded
#     # x1_r = left_to_right_pad(x1, mask1)
#     # x2_r = left_to_right_pad(x2, mask2)
#     # Combine
#     # x_cat = torch.cat([x1_r, x2_r], dim=1)
#     x_cat = torch.cat([x1, x2], dim=1)
#     mask_cat = torch.cat([mask1.sum(1).unsqueeze(1).repeat(1, x1.size(1)),
#                           mask2.sum(1).unsqueeze(1).repeat(1, x2.size(1))], dim=1) > 0
#     # Count new lengths
#     new_lens = mask_cat.sum(1)
#     # Convert back to left-padded
#     x_left = right_to_left_pad(x_cat, mask_cat)
#     # Build new mask
#     B, S_out, D = x_left.shape
#     idxs = torch.arange(S_out, device=x_left.device).unsqueeze(0)
#     new_mask = idxs >= (S_out - new_lens).unsqueeze(1)
#     return x_left, new_mask

import torch
def left_pad_tensor(x, mask):
    sorted_mask, sorted_indices = mask.float().sort(dim=1, descending=True)  # (B, S)
    sorted_indices = sorted_indices.unsqueeze(-1).repeat(1,1,x.shape[-1])
    crop = sorted_mask.any(0).sum()
    x_padded = torch.gather(x, dim=1, index=sorted_indices)[:,:crop]
    sorted_mask = sorted_mask.bool()[:,:crop]
    return x_padded, sorted_mask


# b,t,d = 2,3,4
# x=torch.randn(b,t,d)
# y=torch.randn(b,t,d)
# mask=torch.tensor([[1,1,0],[1,0,0]]).bool()
# print(x)
# print(y)
# # b,t,_ = x.shape
# # lengths = mask.sum(dim=1) # (B,)
# # print(lengths)
# # x_=torch.zeros_like(x)
# # x_[torch.arange(b).unsqueeze(-1), :lengths] = x[torch.arange(b).unsqueeze(-1), t-lengths:]
# # print(x_)
# # out = right_to_left_pad(x, mask)
# # out = right_to_left_pad(torch.cat([x,y],dim=1), torch.cat([mask, mask],dim=1))
# out, mask_ = left_pad_tensor(torch.cat([x,y],dim=1), torch.cat([mask, mask],dim=1))
# print(out)
# print(mask_)

# x_,mask_ = combine_left_padded(x, mask, y, mask)
# print(x_, mask_)


In [ ]:
# @title Attention with kvcache
class Attention(nn.Module):
    # def __init__(self, d_model, cond_dim=None, n_heads=None, d_head=8, dropout=0.): # .1
    def __init__(self, query_dim, cond_dim=None, n_heads=8, d_head=64, drop=0):
        super().__init__()
        d_model = d_head * n_heads
        self.d_head, self.n_heads = d_head, n_heads
        # self.n_heads = d_model // d_head
        # self.d_head = d_model // n_heads
        self.cond_dim = cond_dim
        self.q = nn.Linear(query_dim, d_model, bias=False)
        self.kv = nn.Linear(cond_dim or query_dim, 2*d_model, bias=False)
        # self.lin = nn.Linear(d_model, d_model)
        self.lin = zero_module(nn.Linear(d_model, d_model))
        self.drop = nn.Dropout(drop) # indp before q,k,v; after linout
        self.scale = self.d_head**-.5
        self.cache = {'x':None, 'xq':None, 'xk':None, 'xv':None}
        # self.cache = {'x':torch.empty(0), 'xq':None, 'xk':None, 'xv':None}
# gen: xqkv, patch: xkv ~pq, gen: xqkv pvv
# so save xqkv

    def forward(self, x, cond=None, mask=None): # [batch, T, d_model]=[batch, h*w, c], [batch, num_tok, cond_dim], [batch,T]
        if self.cond_dim==None: cond=x # is self attn
        # m = torch.rand_like(x)<.00001
        # if self.cache['x']!=None and x.shape==self.cache['x'].shape and (x[m]==self.cache['x'][m]).all():
        if self.cache['x']!=None and x.shape==self.cache['x'].shape and (x[0,0,0]==self.cache['x'][0,0,0]).all():
            # print('cache')
            q = self.cache['xq']
            if cond==None: k, v = self.cache['xk'], self.cache['xv']
            else: k, v = self.kv(cond).unflatten(-1, (self.n_heads, 2*self.d_head)).transpose(1, 2).chunk(2, dim=-1) # [batch, n_heads, T/num_tok, d_head]
        else:
            print('no')
            q = self.q(x).unflatten(-1, (self.n_heads, self.d_head)).transpose(1, 2) # [batch, T, d_model] -> [batch, n_heads, T, d_head]
            k, v = self.kv(cond).unflatten(-1, (self.n_heads, 2*self.d_head)).transpose(1, 2).chunk(2, dim=-1) # [batch, n_heads, T/num_tok, d_head]
            self.cache = {'x':x, 'xq':q, 'xk':k, 'xv':v}
        # q @ k.T # [qlen,d]@[d,klen]=[qlen,klen]
        # q-> num_latent # k->seq_len
        if mask is not None: mask = mask.unsqueeze(1)
        # print('attn fwd',q.shape, k.shape, v.shape)
        out = F.scaled_dot_product_attention(q, k, v, attn_mask=mask, dropout_p=0) # F->mask # [batch, len_q, len_v]
        out = out.transpose(1, 2).flatten(2)
        return self.drop(self.lin(out)) # [batch, T, d_model]

dim = 512
model = Attention(dim) # 257 ms
x = torch.rand(64,100,dim)
out = model(x)
print(out.shape)


In [ ]:
# @title sliding window mask me idx kv
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

b,h,t,d = 5,4,10,16
# b,h,t,d = 16,8,1000,8
# # b,h,t,d = 1,1,10,4
# w=3
b,h,t,d = 16,8,4096,8
w=64

q = torch.rand(b,h,t,d, device=device)
kv = torch.rand(b,h,t,2*d, device=device)

# # midx = torch.arange(w).repeat(t,1) + torch.arange(1-w, t-w+1).unsqueeze(-1)
midx = torch.arange(w, dtype=torch.int32).repeat(t,1) + torch.arange(1-w, t-w+1, dtype=torch.int32).unsqueeze(-1)

# mask = torch.tril(torch.ones((t,t), dtype=bool, device=device)) * ~torch.tril(torch.ones((t,t), dtype=bool, device=device), diagonal=-w)

import time
start = time.time()

kv = kv[torch.arange(b)[:,None,None,None], torch.arange(h)[None,:,None,None], midx[None,None,...]]
mk, mv = kv.chunk(2, dim=-1) # [b,h,t,w,d]

# mkv = mask[None,None,...,None]*kv.unsqueeze(2) # [1,1,t,(t),1] @ [b,h,1,t,d] [b,h,t,(t),d]
# mkv = torch.cat([torch.zeros((b,h,w-1,2*d), device=device), mkv.flatten(2,3), torch.zeros((b,h,t-w+1,2*d), device=device)], dim=2) # [b,h,(t*(t+1)),d]
# mk, mv = mkv.unflatten(2,(t,t+1))[:,:,:,:w].chunk(2, dim=-1) # [b,h,t,w,d]
print(time.time()-start)


attn = q.unsqueeze(3) @ mk.transpose(-2,-1) # [b,h,t,1,d] @ [b,h,t,d,w] = [b,h,t,1,w]
attention = torch.softmax(attn, dim=-1) # [b,h,t,1,w]
out = (attention @ mv).squeeze(-2) # [b,h,t,1,w]@[b,h,t,w,d]=[b,h,t,1,d]
print(out.shape)




0.5837950706481934
torch.Size([16, 8, 4096, 8])


In [ ]:
# https://facebookresearch.github.io/xformers/components/ops.html
xformers.ops.memory_efficient_attention(query: Tensor, key: Tensor, value: Tensor, attn_bias: Optional[Union[Tensor, AttentionBias]] = None, p: float = 0.0, scale: Optional[float] = None, *, op: Optional[Tuple[Optional[Type[AttentionFwOpBase]], Optional[Type[AttentionBwOpBase]]]] = None, output_dtype: Optional[dtype] = None) → Tensor
# https://github.com/lucidrains/memory-efficient-attention-pytorch/blob/main/memory_efficient_attention_pytorch/memory_efficient_attention.py
SELF-ATTENTION DOES NOT NEED O(n^2) MEMORY oct 2022
https://arxiv.org/pdf/2112.05682


In [ ]:
# @title xformers.ops.memory_efficient_attention
!pip install -q xformers # 4min
import torch
# from xformers.ops import memory_efficient_attention, AttentionOpDispatch, attention_ops
from xformers.ops import memory_efficient_attention, attention_ops
device = "cuda" if torch.cuda.is_available() else "cpu"

b=5 # 256
d=16 # 1024
h=4
w=128 # 512
t=256 # 7000

q = torch.randn(b,t,d, device=device, requires_grad=True)
k = torch.randn(b,t,d, device=device, requires_grad=True)
v = torch.randn(b,t,d, device=device, requires_grad=True)

attn_bias = attention_ops.SlidingWindowBias(w)
out = memory_efficient_attention(q, k, v, attn_bias=attn_bias)
print(out.shape)  # (B, L, D)


In [ ]:
# @title gemini xformers.ops.memory_efficient_attention
import torch
import xformers.ops as xops
from xformers.ops.fmha.attn_bias import LocalAttentionFromBottomRightMask, LowerTriangularFromBottomRightLocalAttentionMask
device = "cuda" if torch.cuda.is_available() else "cpu"

batch_size = 2
seq_len = 1024
num_heads = 8
head_dim = 64

# Example Q, K, V tensors
query = torch.randn(batch_size, seq_len, num_heads, head_dim, device=device, dtype=torch.float16)
key = torch.randn(batch_size, seq_len, num_heads, head_dim, device=device, dtype=torch.float16)
value = torch.randn(batch_size, seq_len, num_heads, head_dim, device=device, dtype=torch.float16)



import torch
import xformers.ops as xops
from xformers.ops.fmha.attn_bias import AttentionBias

class MyCustomAttentionBias(AttentionBias):
    def __init__(self, some_custom_param):
        super().__init__()
        self.some_custom_param = some_custom_param

    def _materialize(self, shape: tuple[int, int, int, int], dtype: torch.dtype):
        batch_size, num_heads, query_seq_len, key_seq_len = shape

        # Create your custom mask logic here.
        # This mask will be ADDED to the QK^T scores before softmax.
        # -inf to block attention, 0 for no change.
        mask = torch.zeros(query_seq_len, key_seq_len, device="cuda", dtype=dtype)

        # Example: A simple causal mask (lower triangular)
        # For a truly memory-efficient custom mask, you'd want to avoid
        # directly creating a large mask if possible, or ensure it's
        # efficiently handled by xformers' kernels.
        # For demonstration, let's create a causal mask for a small example.
        for i in range(query_seq_len):
            mask[i, i + 1:] = float('-inf')

        # If you have specific non-standard patterns, implement them here.
        # For instance, a very specific block-sparse pattern, or
        # attention based on some external graph connectivity.

        # Example: Block attention for every 3rd token (conceptual, just for demo)
        # for i in range(query_seq_len):
        #     for j in range(key_seq_len):
        #         if (i % 3 == 0) or (j % 3 == 0):
        #             mask[i, j] = float('-inf') # Block if row or col index is a multiple of 3

        # You might also want to expand it to (batch_size, num_heads, Q_seq, K_seq)
        # if your mask is batch or head dependent.
        # For a general sequence-level mask, unsqueeze to add batch and head dims
        return mask.unsqueeze(0).unsqueeze(0) # unsqueeze for batch_size and num_heads if not batch/head dependent

# More complex example: Attention only within specific 'segments'
class SegmentedAttentionBias(AttentionBias):
    def __init__(self, segment_ids_q, segment_ids_k):
        super().__init__()
        # segment_ids_q: (batch_size, query_seq_len) tensor indicating segment for each query token
        # segment_ids_k: (batch_size, key_seq_len) tensor indicating segment for each key token
        self.segment_ids_q = segment_ids_q
        self.segment_ids_k = segment_ids_k

    def _materialize(self, shape: tuple[int, int, int, int], dtype: torch.dtype):
        batch_size, num_heads, query_seq_len, key_seq_len = shape

        # Ensure segment IDs match the sequence lengths
        assert self.segment_ids_q.shape[1] == query_seq_len
        assert self.segment_ids_k.shape[1] == key_seq_len
        assert self.segment_ids_q.shape[0] == batch_size
        assert self.segment_ids_k.shape[0] == batch_size

        # Create the base mask
        mask = torch.empty(batch_size, query_seq_len, key_seq_len, device=self.segment_ids_q.device, dtype=dtype)

        # Iterate over batches to apply segment logic
        # For true memory efficiency, this would ideally be done with
        # clever tensor ops rather than a Python loop if B is large.
        for b in range(batch_size):
            # Expand segment_ids for broadcasting
            seg_q_expanded = self.segment_ids_q[b].unsqueeze(1) # (Q_seq, 1)
            seg_k_expanded = self.segment_ids_k[b].unsqueeze(0) # (1, K_seq)

            # Attention only allowed if segment IDs match
            # This creates a boolean mask (True where segments match)
            segment_match = (seg_q_expanded == seg_k_expanded)

            # Convert boolean mask to attention bias (0 or -inf)
            mask[b] = torch.where(segment_match, 0.0, float('-inf'))

        # Expand for num_heads if the mask is independent of heads
        return mask.unsqueeze(1) # (B, 1, Q_seq, K_seq) -> will broadcast to all heads









In [ ]:
# @title argm

def soft_clamp_relu(x, lower=None, upper=None):
    if lower != None: x = lower+F.relu(x-lower)
    if upper != None: x = upper-F.relu(upper-x)
    return x

def argm_s(sx, x, h0, zz=None): # [1, d_model], [batch_, T, dim_a], [num_layers, 1, d_model], zz[batch_,T, dim_z] # batch argm z for search
    """batch optimise worst case z to increase cost"""
    batch_, T, _ = x.shape
    batch = 16 # 16
    # z = nn.Parameter(torch.zeros((batch*batch_, T, self.dim_z),device=device))
    z = nn.Parameter(torch.randn((batch*batch_, T, self.dim_z),device=device)*.02)
    # torch.nn.init.normal_(z, mean=0., std=1./z.shape[-1]**0.5) # 1.:norm~1 ; 1.
    optim_z = torch.optim.SGD([z], lr=1e3, momentum=0.999, maximize=True) # 3e3
    # optim_z = torch.optim.AdamW([z], 1e-1, (0.1, 0.5), maximize=True) # 1e-1, (0.1, 0.5)
    with torch.no_grad():
        z[:,:self.lz.shape[0]] = self.lz[:T].unsqueeze(0).repeat(batch*batch_,1,1) # [batch*batch_, seq_len, dim_z]
        if zz != None: z[:batch_]=zz #z[::batch]
    sx, h0 = sx.detach(), h0.detach() # [1, d_model], [num_layers, 1, d_model]
    x = x.detach().repeat(batch,1,1) # [batch, T, dim_a]
    # print("argm_s", z[0][0].squeeze())
    for i in range(5): # 2 5
        loss, lh0 = self.rnn_pred(sx, x, z, h0) # snap x to act emb
        loss.sum().backward() # [batch, T]
        optim_z.step()
        optim_z.zero_grad()
        z /= soft_clamp_relu(torch.norm(z, dim=-1).unsqueeze(-1), lower=1)
        # print(i, "argm_s z", z[0][0].squeeze().data)
        print(i, "argm_s loss", loss.sum(-1)[0].data)
        # # print(torch.norm(z, dim=-1)) # all 1
    idx = torch.argmax(loss.sum(-1).unflatten(0, (batch,batch_)), dim=0) # loss [batch*batch_, T] -> [batch_]
    # print("argm_s loss[idx]", loss[idx].sum().item())
    return z.unflatten(0, (batch,batch_))[idx, torch.arange(batch_)]



In [ ]:
# @title gpt train test generate
import torch
from torch.nn import functional as F
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
scaler = torch.GradScaler()

def strain(model, dataloader, optimizer, scheduler=None): # train function with automatic mixed precision
    model.train()
    total_loss = 0
    for i, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        with torch.autocast(device_type=device, dtype=torch.bfloat16):
            logits = model(x) # [b,t,d]
            loss = F.cross_entropy(logits.reshape(-1, logits.size(-1)), y.flatten()) # [512, 128, 283], [512, 128]
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        # scaler.unscale_(optim)
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        scaler.step(optimizer)
        scaler.update()
        # if scheduler is not None: scheduler.step()
        if i % 100 == 0: print("strain",loss.item())
        total_loss += loss.item()
        try: wandb.log({"train loss": loss.item()/len(y)})
        except NameError: pass
    return total_loss / len(dataloader)

def test(loader, model):
    model.eval()
    total_loss = 0
    for i, (x, y) in enumerate(loader):
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
        loss = F.cross_entropy(logits.flatten(end_dim=-2), y.flatten()) # [batch*seq_len, vocab_size], [batch*seq_len]
        if i % 100 == 0: print("test",loss.item())
        try: wandb.log({"test loss": loss.item()/len(y)})
        except NameError: pass
    return total_loss / len(loader)

def generate(model, context, max_steps=64, temperature=1): # [b,t]
    training = model.training
    model.eval()
    x = encode(context)
    for n in range(max_steps):
        with torch.no_grad():
            out = model(x)
        out = out[:,-1] # get logit for last character
        out = out/temperature
        out = F.softmax(out, dim=-1) # vocab_size to char
        ix = torch.multinomial(out, num_samples=1) # rand sample by output distribution
        x = torch.cat([x, ix],1)
    completion = decode(x.squeeze(0))
    if training: model.train()
    return completion

import time
start = begin = time.time()
for i in range(1):
# for i in range(30):
    train_loss = strain(model, train_loader, optim, scheduler=None)
    test_loss = test(test_loader, model)
    print('Test Loss:', test_loss)
    print(generate(model, "this is what "))
    # print(model.generate('this is wh'))
    print(i, 'time:',time.time() - start, (time.time()-begin)/(i+1))
    start = time.time()


In [ ]:
# https://github.com/Dao-AILab/flash-attention
!pip install -q flash-attn --no-build-isolation

In [ ]:
# @title Dao-AILab/flash-attention flash_attn_with_kvcache
def flash_attn_with_kvcache(
    q,
    k_cache,
    v_cache,
    k=None,
    v=None,
    rotary_cos=None,
    rotary_sin=None,
    cache_seqlens: Optional[Union[(int, torch.Tensor)]] = None,
    cache_batch_idx: Optional[torch.Tensor] = None,
    block_table: Optional[torch.Tensor] = None,
    softmax_scale=None,
    causal=False,
    window_size=(-1, -1),  # -1 means infinite context window
    rotary_interleaved=True,
    alibi_slopes=None,
):
    """
    If k and v are not None, k_cache and v_cache will be updated *inplace* with the new values from
    k and v. This is useful for incremental decoding: you can pass in the cached keys/values from
    the previous step, and update them with the new keys/values from the current step, and do
    attention with the updated cache, all in 1 kernel.

    If you pass in k / v, you must make sure that the cache is large enough to hold the new values.
    For example, the KV cache could be pre-allocated with the max sequence length, and you can use
    cache_seqlens to keep track of the current sequence lengths of each sequence in the batch.

    Also apply rotary embedding if rotary_cos and rotary_sin are passed in. The key @k will be
    rotated by rotary_cos and rotary_sin at indices cache_seqlens, cache_seqlens + 1, etc.
    If causal or local (i.e., window_size != (-1, -1)), the query @q will be rotated by rotary_cos
    and rotary_sin at indices cache_seqlens, cache_seqlens + 1, etc.
    If not causal and not local, the query @q will be rotated by rotary_cos and rotary_sin at
    indices cache_seqlens only (i.e. we consider all tokens in @q to be at position cache_seqlens).

    See tests/test_flash_attn.py::test_flash_attn_kvcache for examples of how to use this function.

    Supports multi-query and grouped-query attention (MQA/GQA) by passing in KV with fewer heads
    than Q. Note that the number of heads in Q must be divisible by the number of heads in KV.
    For example, if Q has 6 heads and K, V have 2 heads, head 0, 1, 2 of Q will attention to head
    0 of K, V, and head 3, 4, 5 of Q will attention to head 1 of K, V.

    If causal=True, the causal mask is aligned to the bottom right corner of the attention matrix.
    For example, if seqlen_q = 2 and seqlen_k = 5, the causal mask (1 = keep, 0 = masked out) is:
        1 1 1 1 0
        1 1 1 1 1
    If seqlen_q = 5 and seqlen_k = 2, the causal mask is:
        0 0
        0 0
        0 0
        1 0
        1 1
    If the row of the mask is all zero, the output will be zero.

    If window_size != (-1, -1), implements sliding window local attention. Query at position i
    will only attend to keys between
    [i + seqlen_k - seqlen_q - window_size[0], i + seqlen_k - seqlen_q + window_size[1]] inclusive.

    Note: Does not support backward pass.

    Arguments:
        q: (batch_size, seqlen, nheads, headdim)
        k_cache: (batch_size_cache, seqlen_cache, nheads_k, headdim) if there's no block_table,
            or (num_blocks, page_block_size, nheads_k, headdim) if there's a block_table (i.e. paged KV cache)
            page_block_size must be a multiple of 256.
        v_cache: (batch_size_cache, seqlen_cache, nheads_k, headdim) if there's no block_table,
            or (num_blocks, page_block_size, nheads_k, headdim) if there's a block_table (i.e. paged KV cache)
        k [optional]: (batch_size, seqlen_new, nheads_k, headdim). If not None, we concatenate
            k with k_cache, starting at the indices specified by cache_seqlens.
        v [optional]: (batch_size, seqlen_new, nheads_k, headdim). Similar to k.
        rotary_cos [optional]: (seqlen_ro, rotary_dim / 2). If not None, we apply rotary embedding
            to k and q. Only applicable if k and v are passed in. rotary_dim must be divisible by 16.
        rotary_sin [optional]: (seqlen_ro, rotary_dim / 2). Similar to rotary_cos.
        cache_seqlens: int, or (batch_size,), dtype torch.int32. The sequence lengths of the
            KV cache.
        block_table [optional]: (batch_size, max_num_blocks_per_seq), dtype torch.int32.
        cache_batch_idx: (batch_size,), dtype torch.int32. The indices used to index into the KV cache.
            If None, we assume that the batch indices are [0, 1, 2, ..., batch_size - 1].
            If the indices are not distinct, and k and v are provided, the values updated in the cache
                 might come from any of the duplicate indices.
        softmax_scale: float. The scaling of QK^T before applying softmax.
            Default to 1 / sqrt(headdim).
        causal: bool. Whether to apply causal attention mask (e.g., for auto-regressive modeling).
        window_size: (left, right). If not (-1, -1), implements sliding window local attention.
        rotary_interleaved: bool. Only applicable if rotary_cos and rotary_sin are passed in.
            If True, rotary embedding will combine dimensions 0 & 1, 2 & 3, etc. If False,
            rotary embedding will combine dimensions 0 & rotary_dim / 2, 1 & rotary_dim / 2 + 1
            (i.e. GPT-NeoX style).
        alibi_slopes: (nheads,) or (batch_size, nheads), fp32. A bias of
            (-alibi_slope * |i + seqlen_k - seqlen_q - j|)
            is added to the attention score of query i and key j.

    Return:
        out: (batch_size, seqlen, nheads, headdim).
    """


In [ ]:
# @title flash_attn_with_kvcache no custom mask
import torch
from flash_attn import flash_attn_with_kvcache

b,t,nheads,dhead = 2,5,3,4
# q: (batch_size, seqlen, nheads, headdim)
# k_cache: (batch_size_cache, seqlen_cache, nheads_k, headdim) if there's no block_table,
#     or (num_blocks, page_block_size, nheads_k, headdim) if there's a block_table (i.e. paged KV cache)
#     page_block_size must be a multiple of 256.
# v_cache: (batch_size_cache, seqlen_cache, nheads_k, headdim) if there's no block_table,
#     or (num_blocks, page_block_size, nheads_k, headdim) if there's a block_table (i.e. paged KV cache)
# k [optional]: (batch_size, seqlen_new, nheads_k, headdim). If not None, we concatenate
#     k with k_cache, starting at the indices specified by cache_seqlens.
# v [optional]: (batch_size, seqlen_new, nheads_k, headdim). Similar to k.

cache_len = 7
q = torch.rand(b,t,nheads,dhead)
k = torch.rand(b,t,nheads,dhead)
v = torch.rand(b,t,nheads,dhead)
k_cache = torch.rand(b,cache_len,nheads,dhead)
v_cache = torch.rand(b,cache_len,nheads,dhead)


out = flash_attn_with_kvcache(q, k_cache, v_cache, k=None, v=None)
out = flash_attn_with_kvcache(q, k_cache, v_cache, k, v)
print(out)

out = F.scaled_dot_product_attention(q, k, v, attn_mask=mask.unsqueeze(1) if mask != None else None, dropout_p=0) # mask: [batch,len_q, len_v]
# out = F.scaled_dot_product_attention(q, k, v, is_causal=True, dropout_p=0) # mask: [batch,len_q, len_v]
# attn = q @ k.transpose(-2,-1) * self.scale # [batch, n_heads, T] # [batch, n_heads, T, T/num_tok]
# # if mask != None: attn = attn.masked_fill(mask[:, None, :, None], -torch.finfo(attn.dtype).max) # [batch,T]->[batch,1,T,1]
# if mask != None: attn = attn.masked_fill(mask.unsqueeze(1), -torch.finfo(attn.dtype).max) # [b,t,t]->[b,1,t,t]
# attention = torch.softmax(attn, dim=-1)
# out = self.drop(attention) @ v # [batch, n_heads, T, d_head]





## old blt

In [ ]:
# @title BLTlayer transformer fsq
import torch
import torch.nn as nn
from torch.nn import functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"

class BLTlayer(nn.Module):
    def __init__(self, in_dim, d_model=None, quant=6, num_layers=1):
        super().__init__()
        d_model = d_model or in_dim
        self.pos_enc = RoPE(d_model, base=100) # 10000
        # self.latent = nn.Parameter(torch.randn(1,1,d_model)*.02) # init
        # self.perceiver = AttentionBlock(d_model, n_heads=4, cond_dim=d_model)
        self.perceiver = Seq(*[AttentionBlock(d_model, n_heads=4, cond_dim=d_model) for _ in range(num_layers)])
        self.lin = nn.Linear(d_model, d_model)
        self.patch_enc = nn.Parameter(torch.randn(1,1,d_model)*.02)
        self.conv_emb = nn.Conv1d(d_model, d_model, 1)
        self.vq = codequant(levels = d_model*[quant])

    def forward(self, x, cond=None, mask=None): # [b,t,d], [b,t,t]
        x = self.pos_enc(x)
        b,t,d = x.shape
        # if mask==None: mask = torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(b,1,1) # F->mask # for F.scaled_dot_product_attention
        # print('BLTlayer',x.shape, cond.shape if cond!=None else None, mask.shape if mask!=None else None)
        x = self.perceiver(x, cond=cond if cond!=None else x, mask=mask) # [b,t,d], [nlyr,b,d]
        out = self.lin(x)
        return out # [b,t,d]

    def gen_loss(self, y): # [b,t,d]
        b,t,d = y.shape
        mask = torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(b,1,1)
        y_ = self.forward(y, mask=mask) # [b,t,d], [b,t,t]
        print('gen_loss', y_.shape, y.shape)
        loss = F.mse_loss(y_[:,:-1], y[:,1:])
        return loss, y_

    def to_patch(self, x, out=None, entropies=None): # [b,t,d]
        if entropies==None: entropies = (out-self.vq(out)).mean(-1) # [b,t] float
        mask = patchify(entropies, 1.) # [b,t] # T at start of each patch, F otherwise
        patch_mask = padded_block_mask(mask) # [batch ,num_patch, seq_len]
        # print('lyr to_patch', x.shape, out.shape, patch_mask.shape)

        latent = (x.unsqueeze(1) *patch_mask.unsqueeze(-1)).mean(2) # [b ,num_patch, t, d] -> [b ,num_patch, d]
        # latent = self.latent.repeat(1,mask.shape[1],1) # [b ,num_patch, d]
        latent = latent + self.patch_enc
        out = self.conv_emb(x.transpose(-2,-1)).transpose(-2,-1) # instead of hashing
        patches = self.forward(latent, cond=out, mask=patch_mask) # [batch, num_patch, latent_dim/d_model]
        # patches = self.vqp(patches)
        return patches, mask, patch_mask # [b, num_patch, out_dim], [b,t], [b ,num_patch, t]
# AE: mask->pos_patch?/conv1d? + mask+ latent(mean+*leant) -> patch; pos_patch?+ mask.T + patch(+*learnt?)-> og

    def ae_loss(self, y, patches, mask, patch_mask): # [b,t,d], [b, n_patch, d], [b,t], [b, n_patch, t]
        b,t,d = y.shape
        # cond = patch_mask.transpose(-2,-1).to(patches.dtype) @ patches # [b, t, out_dim]
        patch_mask = patch_mask.transpose(-2,-1) # [b, t, n_patch]
        causal_mask = block_diag(mask) & torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(b,1,1) # [b,t,t] # only attend within the patch & is causal
        # print('ae_loss', y.shape, patches.shape, patch_mask.shape) # [2, 7, 16], [2, 3, 16], [2, 7, 3]
        y_ = self.forward(y, cond=torch.cat([y, patches], dim=1), mask=torch.cat([causal_mask, patch_mask], dim=-1)) # cond: [b, t+n_patch, d]; mask: [batch, len_q, len_v] [b, t, t+n_patch]
        print('ae_loss', y_.shape, y.shape)
        # loss = F.mse_loss(y[mask], y_[mask])
        loss = F.mse_loss(y_[:,:-1], y[:,1:])
        return loss

# generate one tok unless uncertain ? then patch up?
# gen one only bec blt need to tok
# or blt feed only one tok? nope, should parallelise
# feed all toks bec topatch need out
    def generate_one(self, x=None, cond=None, patch=None): # [1,t,d], [1,len_cond,d], [1,1,d]
        # print('lyr gen1 x cond patch', x.shape, cond.shape if cond!=None else None, patch.shape if patch!=None else None)
        _,t,d = x.shape
        if cond==None:
            cond = x
            mask = torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(1,1,1)
        else:
            mask = torch.ones((1,t,cond.shape[1]), dtype=bool, device=device) # [1,t,n_cond+n_patch]
        if patch!=None:
            cond = torch.cat([cond, patch], dim=1)
            pmask = torch.zeros((1,t,patch.shape[1]), dtype=bool, device=device)
            pmask[:,-1] = True # patch is only for last (aka new) tok
            mask = torch.cat([mask, pmask], dim=-1) # [1,t,t+1]
            # mask = torch.ones((1,t,cond.shape[1]), dtype=bool, device=device) # [1,t,n_cond+n_patch]
        # print('lyr gen1', x.shape, cond.shape, mask.shape)
        out = self.forward(x, cond=cond, mask=mask)
        qout = self.vq(out)
        etp = (out-qout).mean(-1) # [b,1] float
        # x = torch.cat([x, qout], dim=1)
        # return x, etp
        return qout, etp

# # generate till uncertain
#     def generate_all(self, x=None, cond=None, patch=None): # [1,t,d], [1,1,d]/[1,t,d]?
#         out, entropies = model.generate_one(x, patch=patch)
#         x = torch.cat([x, out[:,-1:]], dim=1)
#         mask = patchify(entropies, 1.)
#         while not mask[:,-1]:
#             out, etp = model.generate_one(x[:,-1:], cond=x[:,:-1], patch=patch)
#             x = torch.cat([x, out], dim=1)
#             entropies = torch.cat([entropies, etp], dim=1) # [b,t]
#             # if i==0: continue # want gen at least one
#             mask = patchify(entropies, 1.)
#         return x[:,:-1], entropies

# if no patch, generate till uncertain
# if got patch list, iter through
    def generate_all(self, x=None, pch_lst=[None]): # [1,t,d], [1,1,d]/[1,t,d]?
    # def generate_all(self, x=None, pch_lst=[]): # [1,t,d], [1,1,d]/[1,t,d]?
        # if pch_lst[0]==None:
        # out, entropies = self.generate_one(x, patch=pch_lst.pop())
        out, entropies = self.generate_one(x, patch=pch_lst[0])
        # x = torch.cat([x, out[:,-1:]], dim=1)
        mask = patchify(entropies, 1.)
        for patch in pch_lst:
            while not mask[:,-1]:
                out, etp = self.generate_one(x[:,-1:], cond=x[:,:-1], patch=patch)
                x = torch.cat([x, out], dim=1)
                entropies = torch.cat([entropies, etp], dim=1) # [b,t]
                # if i==0: continue # want gen at least one
                mask = patchify(entropies, 1.)
            # x, mask, entropies = x[:,:-1], mask[:,:-1], entropies[:,:-1]
        # return x[:,:-1], entropies
        # print('lyr gen_all', x.shape, entropies.shape)
        return x, entropies # [1,t,d], [1,t]

# trans: btd->btd, train: shift1
# bltlyr: btd->btd+bpd , gentrain: shift1, artrain:btd+bpd->btd

# f_ j ; vs f_ t_
# full text

# is gpt + aetrain?

# in: qunatseq
# fwd: qunatx -> y_
# to_patch: x, y_ -> patches, mask
# gen: x, (patch) ->


batch=25
d_model = 16
in_dim=128

model = BLTlayer(in_dim, d_model).to(device)
# model = BLTlayer(d_model).to(device)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683
optim = torch.optim.AdamW(model.parameters(), 1e-3, (0.9, 0.95)) # 1e-4 #3e-4


x = torch.rand((batch, 7, d_model), device=device)
out = model(x)
print('out', out.shape)

patches, mask, patch_mask = model.to_patch(x, out)
print('patches, mask',patches.shape, mask.shape)
gloss = model.gen_loss(out)
aloss = model.ae_loss(x, patches, mask, patch_mask)
# loss = gloss + aloss

# patch = torch.rand(batch,1,d_model)
# patch = torch.rand(1,1,d_model)
# pch_lst = [patch]
pch_lst = list(torch.rand(1,3,d_model).split(1, dim=1))
# out, mask = model.generate(x)
# out, mask = model.generate(x, patch)
x = torch.rand((1,7, d_model), device=device)
out, _ = model.generate_all(x)
# out, _ = model.generate_all(x, patch=patch)
# out, _ = model.generate_all(x[:,-1:], cond=x[:,:-1], patch=patch)
out, _ = model.generate_all(x, pch_lst=pch_lst)
# out, _ = model.generate_all(x[:,-1:], cond=x[:,:-1])

# out, _ = model.generate_one(x) # gen all till uncertain
# out, _ = model.generate_one(x, patch=patch) # patch cond from last tok onwards
# out, _ = model.generate_one(x[:,-1:], cond=x[:,:-1], patch=patch) # got cond no need to cond on all
# out, _ = model.generate_one(x[:,-1:], cond=x[:,:-1])
print(out.shape)
# out, mask_ = left_pad_tensor(torch.cat([x,y],dim=1), torch.cat([mask, mask],dim=1))

# print(patches.shape)
# print(out)


3840
out torch.Size([25, 7, 16])
patches, mask torch.Size([25, 6, 16]) torch.Size([25, 7])
gen_loss torch.Size([25, 7, 16]) torch.Size([25, 7, 16])
ae_loss torch.Size([25, 7, 16]) torch.Size([25, 7, 16])
torch.Size([1, 9, 16])


In [ ]:
# @title BLT me fsq
import torch
import torch.nn as nn
from torch.nn import functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"

class BLT(nn.Module):
    # def __init__(self, in_dim, d_model, out_dim=None, num_layers=1):
    def __init__(self, d_model, num_layers=1):
        super().__init__()
        self.layer = nn.ModuleList([BLTlayer(d_model, d_model, quant=6*2**i) for i in range(num_layers)])

    # def forward(self, x): # [b,t]
    #     x = self.tok_emb(x)
    #     for level in self.layer:
    #         # out, x = level(x) # [b,t,d]
    #         out = level(x) # [b,t,d]
    #         patches, mask, patch_mask = level.to_patch(x, out)
    #         # outs.append(out)
    #         x = level.to_patch(out)
    #     x = self.lin(x)
    #     return x # [batch, num_patch, out_dim]

    def loss(self, x): # [b,t,d]
        tt_loss = 0
        for i, level in enumerate(self.layer):
            b,t,d = x.shape
            # mask = torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(b,1,1) # F->mask # for F.scaled_dot_product_attention
            gloss, out = level.gen_loss(x)
            tt_loss += gloss
            if i != len(self.layer)-1: # last layer no patching
                patches, mask, patch_mask = level.to_patch(x, out)
                tt_loss += level.ae_loss(x, patches, mask, patch_mask)
                x = patches
        return tt_loss



#     def generate(self, x):
#         patches = []
#         for j, patch in enumerate(patches):
#             if len(patch)>0:
#                 break
#         # patch in patches
#         for i in reversed(range(j)):
#             patch = patches[i].pop(0)
#             layer = self.layers[i]
#             # = self.perceiver(x, cond=patch)
#             out = layer.generate(x if i==0 else None, patch)
#             if i!=0: patches[i-1].append(out[:,-1])
# # out, mask_ = left_pad_tensor(torch.cat([x,y],dim=1), torch.cat([mask, mask],dim=1))

#         # entropies = entropy(out) # [b,t] float
#         # mask = patchify(entropies, 1.) # [b,t,t] # T at start of each patch, F otherwise
#         return out[:,-1]

    def generate_all(self, x, src_lst=[], pch_lst=[]): # [1,t,d]
    # def generate_all(self, x, src_lst=[]): # [1,t,d]pch_lst=[]
        for i, level in enumerate(self.layer):
            # out, _ = level.generate_all(x)
            out, entropies = level.generate_all(x)
            src_lst.append(out)
            # pch_lst.append(out[:,x.shape[1]:])
            # print('ifyvh blt genall',out[:,x.shape[1]:].shape,list(out[:,x.shape[1]:].split(1, dim=1)))
            pch_lst.append(list(out[:,x.shape[1]:].split(1, dim=1)))

            if i == len(self.layer)-1: break # top layer dont need patch
            # if i != len(self.layer)-1:
            # patches, mask, patch_mask = level.to_patch(x, out, entropies=entropies)
            patches, mask, _ = level.to_patch(out, entropies=entropies)
            # new_pch = mask[:,x.shape[1]:].sum(-1)
            # pch_lst.append([patches[:,-new_pch:]])
            # pch_lst.append(patches[:,-new_pch:])
            x = patches
            # src_lst.append(patches)
            # src_lst[i+1] = patches

            # x = level.to_patch(out)
            # src_lst.append(x)
        # if self.lin: x = self.lin(x)
        # return x # [batch, num_patch, out_dim]
        patch = None
        # print('5555555555555', pch_lst)
        # print('5555555555555', len(pch_lst))
        # for i in reversed(range(len(self.layers))):
        for i, level in enumerate(reversed(self.layer)):
            # layer = self.layers[i]
            # x = src_lst.pop()
            # x = src_lst[i-1]
            if i == len(self.layer)-1:
                # out, _ = level.generate_one(x, patch=patch)
                out, _ = level.generate_one(x[:,-1:], cond=x[:,:-1])
            else:

                # out, _ = level.generate_all(x[:,-1:], cond=x[:,:-1], pch_lst=pch_lst)
                # out, _ = level.generate_all(x, pch_lst=pch_lst[-i])
                # print('blt genall', i, src_lst[-i].shape, [p.shape for p in pch_lst])
                # print('blt genall', i, src_lst[-i].shape, pch_lst[-i].shape)
                # print('blt genall', i, src_lst[-i].shape, len(pch_lst[-i]))
                # print(pch_lst[-i])
                # print('blt genall', i, src_lst[-i], [p for p in pch_lst])
                # torch.cat([pch_lst[-1-i]])
                # out[:,src_lst[-2-i]:]
                out, _ = level.generate_all(src_lst[-2-i], pch_lst=pch_lst[-1-i])
            # out, mask_ = level.generate(x, patch)
            # out = level.generate_all(x, patch=patch)
            # out, mask_ = left_pad_tensor(torch.cat([x,y],dim=1), torch.cat([mask, mask_],dim=1))
            # = self.perceiver(x, cond=patch)
            # patch = x
            # torch.cat([pch_lst[-i]], dim=)
            # out = layer.generate(x if i==0 else None, patch)
            # if i!=0: patches[i-1].append(out[:,-1])

        # entropies = entropy(out) # [b,t] float
        # mask = patchify(entropies, 1.) # [b,t,t] # T at start of each patch, F otherwise

        return out#[:,-1]



# if not confident, get patch from higher level
    def generate_one(self, x, src_lst=None, etp_lst=None): # [b,t]
        # x = self.tok_emb(x) # [b,t,d]
        b,t,d = x.shape
        src_lst = src_lst or len(self.layer)*[None]
        etp_lst = etp_lst or len(self.layer)*[torch.empty((b,0), dtype=torch.float, device=device)]

        # for i, level in enumerate(self.layer):
        for i, (level, entropies) in enumerate(zip(self.layer, etp_lst)):
            out, entropies = level.generate_one(x)
            # out, qout = level.generate_one(x, cond=src_lst[i])
            # print('gen one11', x.shape, out.shape)

            mask = patchify(entropies, 1.) # [1,t] # T at start of each patch, F otherwise

            # entropies = torch.cat([entropies, etp], dim=1) # [b,t]
            # etp_lst[i] = torch.cat([etp_lst[i], etp], dim=1) # [b,t]
            # mask = patchify(etp_lst[i], 1.) # [b,t,t] # T at start of each patch, F otherwise
            # print('gen one', mask.shape)
            # print('gen one', x.shape, out.shape)
            if not mask[:,-1]: break # newly generated tok is confident

            if i==len(self.layer)-1: break # last layer no patching
            src_lst.append(x)
            patches, _, _ = level.to_patch(x, entropies=entropies)
            x = patches

        # if i==0: return x, src_lst, etp_lst # 1st lvl is confident [1, t+1, d]
        patch = None
        # for i in reversed(range(len(self.layers))):
        for i, v in enumerate(reversed(src_lst)):
            # patch, etp = self.layer[len(src_lst)-i].generate_one(v, patch=patch)
            patch, etp = self.layer[len(src_lst)-i].generate_one(v[:,:-1], cond=v[:,-1:], patch=patch)
            # patch, etp = self.layer[len(src_lst)-i].generate_one(x, patch)

        # patch = self.out(patch)
        return patch, src_lst, etp_lst
# fwd, quant, entropy topatch; save qout, entropy
# query new tok, append entropy



# pos_all, entropy mask;
# AE: mask->pos_patch?/conv1d? + mask+ latent(mean+*leant) -> patch; pos_patch?+ mask.T + patch(+*learnt?)-> og
# gen train: mse(x[1:],out[:-1])
# gen inference: x cond=x, cond=(x+patch(+*learnt?)), if entropy/bos, pop patch and regen

# # fsq: dont realize tokens in layer
# layer:
# posemb
# x less quants patch more quants
# quant after linout
# # quant before linout

# blt:
# tokemb
# linout to tok


# gen:
# gen lower only, until uncertain, then patch, gen top then patch cond bottom
# gen 1 tok. if uncertain, to_patch then top gen

d_model=16
model = BLT(d_model, num_layers=3)
x = torch.rand(5,30,d_model)
# out = model(x)
out = model.loss(x)
# print(out)
# out = model.generate(x)
# out, src_lst, etp_lst = model.generate_one(x)

x = torch.rand(1,13,d_model)
# pch_lst = list(torch.rand(1,3,d_model).split(1, dim=1))
out = model.generate_all(x)
# out = model.generate_all(x, pch_lst=pch_lst)
print(out.shape)
# print(out)


torch.Size([1, 1, 16])


In [ ]:
# @title BLTlayer transformer fsq
import torch
import torch.nn as nn
from torch.nn import functional as F

# transformer
class BLTlayer(nn.Module):
    # def __init__(self, in_dim, d_model=None, out_dim=None, num_layers=1):
    def __init__(self, in_dim, d_model=None, quantxp=(6,12), num_layers=1):
        super().__init__()
        d_model = d_model or in_dim
        self.pos_enc = RoPE(d_model, base=100) # 10000
        # self.latent = nn.Parameter(torch.randn(1,1,d_model)*.02) # init
        # self.perceiver = AttentionBlock(d_model, n_heads=4, cond_dim=d_model)
        self.perceiver = Seq(*[AttentionBlock(d_model, n_heads=4, cond_dim=d_model) for _ in range(num_layers)])
        self.lin = nn.Linear(d_model, d_model)
        # self.out = nn.Linear(d_model, out_dim)
        self.patch_enc = nn.Parameter(torch.randn(1,1,d_model)*.02)
        self.conv_emb = nn.Conv1d(d_model, d_model, 1)
        self.vqx = codequant(levels = d_model*[quantxp[0]])
        self.vqp = codequant(levels = d_model*[quantxp[1]])
        # self.out = nn.Linear(d_model, in_dim)

    def forward(self, x, cond=None, mask=None): # [b,t,d], [b,t,t]
        x = self.pos_enc(x)
        b,t,d = x.shape
        # if mask==None: mask = torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(b,1,1) # F->mask # for F.scaled_dot_product_attention
        # if type(mask)!=torch.tensor: mask = torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(b,1,1) # F->mask # for F.scaled_dot_product_attention
        # if type(mask)==str: mask = torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(b,1,1) # F->mask # for F.scaled_dot_product_attention
        # print('BLTlayer',x.shape, cond.shape if cond!=None else None, mask.shape if mask!=None else None)
        x = self.perceiver(x, cond=cond if cond!=None else x, mask=mask) # [b,t,d], [nlyr,b,d]
        out = self.lin(x)
        return out # [b,t,d_model]

    def gen_loss(self, src): # [b,t,d]
        x, y = src[:,:-1], src[:,1:]
        b,t,d = x.shape
        mask = torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(b,1,1)
        y_ = self.forward(x, mask=mask) # [b,t,d], [b,t,t]
        # print('gen_loss', out.shape, y.shape)
        loss = F.mse_loss(y_, y)
        # loss = F.cross_entropy(out.flatten(0,1), y.flatten()) # [b*t,d], [b*t]
        return loss


    def to_patch(self, x, out): # [b,t,d]
        entropies = (out-self.vqx(out)).mean(-1) # [b,t] float
        mask = patchify(entropies, 1.) # [b,t,t] # T at start of each patch, F otherwise
        patch_mask = padded_block_mask(mask) # [batch ,num_patch, seq_len]
        # print('to_patch', x.shape, out.shape, patch_mask.shape)

        latent = (x.unsqueeze(1) *patch_mask.unsqueeze(-1)).mean(2) # [b ,num_patch, t, d] -> [b ,num_patch, d]
        # latent = self.latent.repeat(1,mask.shape[1],1) # [b ,num_patch, d]
        latent = latent + self.patch_enc
        out = self.conv_emb(x.transpose(-2,-1)).transpose(-2,-1)
        patches = self.forward(latent, cond=out, mask=patch_mask) # [batch, num_patch, latent_dim/d_model]
        # print('BLTlayer',out.shape)
        patches = self.vqp(patches)
        return patches, mask, patch_mask # [b, num_patch, out_dim], [b ,num_patch, t]

    def ae_loss(self, y, patches, mask, patch_mask): # [b,t,d], [b, n_patch, d], [b,t], [b, n_patch, t]
        # print('ae_loss', x.shape, self.tok_emb(x).shape) # [2, 7, 16], [2, 5, 128], [2, 7, 5]
        x = self.pos_enc(y)
        b,t,d = x.shape
        # cond = patch_mask.transpose(-2,-1).to(patches.dtype) @ patches # [b, t, out_dim]
        patch_mask = patch_mask.transpose(-2,-1) # [b, t, n_patch]
        causal_mask = block_diag(mask) & torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(b,1,1) # [b,t,t] # only attend within the patch & is causal
        # print('ae_loss', x.shape, patches.shape, patch_mask.shape) # [2, 7, 16], [2, 3, 16], [2, 7, 3]
        y_ = self.forward(x, cond=torch.cat([x, patches], dim=1), mask=torch.cat([causal_mask, patch_mask], dim=-1)) # cond: [b, t+n_patch, d]; mask: [batch, len_q, len_v] [b, t, t+n_patch]
        loss = F.mse_loss(y[mask], y_[mask])
        return loss

# AE: mask->pos_patch?/conv1d? + mask+ latent(mean+*leant) -> patch; pos_patch?+ mask.T + patch(+*learnt?)-> og

# generate till uncertain
    def generate_all(self, x=None, patch=None): # [b,t,d], [b,t,d]
    # def generate(self, x=None, patch=None, mask=None, patch_mask=None): # [b,t], [b,t,d], [b,t], [b,n_patch,t]
        b,t,d = x.shape
        entropies = torch.empty((b,0), device=device)
        # while True:
        for i in range(50):
            if patch!=None: cond = torch.cat([x[:,:-1], patch],1)
            else: cond = x[:,:-1]
            out = self.forward(x[:,-1:], cond=cond)
            qout = self.vqx(out)
            entropy = (out-qout).mean(-1) # [b,1] float
            x = torch.cat([x, qout], dim=1)
            entropies = torch.cat([entropies, entropy], dim=1) # [b,t]
            # if i==0: continue # want gen at least one
            mask = patchify(entropies, 1.)
            if (mask.sum(1)>=2).all(): break

        # print('BLTlayer gen',mask.shape)
        mask = mask.cumsum(dim=1)>=2 # [b,t]
        out = x[:,t:]*mask.unsqueeze(-1)
        # print('BLTlayer gen',x.shape, out.shape, mask.shape)

        # return x[1:-1]
        # return torch.cat((x, out),1), mask
        return torch.cat((x[:,:t], out),1), torch.cat([torch.ones(b,t, dtype=bool, device=device), mask], dim=1)


# generate till uncertain
    def generate_all(self, x=None, cond=None, patch=None): # [1,t,d], [1,1,d]/[1,t,d]?
        # _,t,d = x.shape
        # mask=None
        # if cond==None:
        #     cond = x
        #     mask = torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(1,1,1)
        # if patch!=None:
        #     cond = torch.cat([cond, patch], dim=1)
        #     pmask = torch.zeros((1,t,patch.shape[1]), dtype=bool, device=device)
        #     pmask[:,-1] = True # patch is only for last tok
        #     mask = torch.cat([mask, pmask], dim=-1) # [1,t,t+1]
        #     # mask = torch.ones((1,t,cond.shape[1]), dtype=bool, device=device) # [1,t,n_cond+n_patch] # patch

        # # print('lyr genall', x.shape, cond.shape, mask.shape)
        # out = self.forward(x, cond=cond, mask=mask)
        # qout = self.vqx(out)
        # entropies = (out-qout).mean(-1)

        out, entropies = model.generate_one(x, patch=patch)
        x = torch.cat([x, out[:,-1:]], dim=1)


        mask = patchify(entropies, 1.)
        while not mask[:,-1]:
        # for i in range(50):
            # if patch!=None: cond = torch.cat([x[:,:-1], patch],1)
            # else: cond = x[:,:-1]
            # out = self.forward(x[:,-1:], cond=cond)
            # qout = self.vqx(out)
            # etp = (out-qout).mean(-1) # [b,1] float

            out, etp = model.generate_one(x[:,-1:], cond=x[:,:-1], patch=patch)


            x = torch.cat([x, out], dim=1)
            entropies = torch.cat([entropies, etp], dim=1) # [b,t]
            # if i==0: continue # want gen at least one
            mask = patchify(entropies, 1.)
        return x[:,:-1], entropies



    def generate_one(self, x=None, patch=None): # [1,t,d], [1,1,d]
    # def generate(self, x=None, patch=None, mask=None, patch_mask=None): # [b,t], [b,t,d], [b,t], [b,n_patch,t]
        # t,d = x.shape
        _,t,d = x.shape
        # if patch!=None: cond = torch.cat([x[:,:-1], patch],1)
        # else: cond = x[:,:-1]
        # out = self.forward(x[:,-1:], cond=cond)

        mask = torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(1,1,1)
        if patch!=None:
            cond = torch.cat([x, patch],1)
            mask = torch.cat([mask, torch.ones((1,t,1), dtype=bool, device=device)], dim=-1)
        else: cond = x
        # print('lyr gen1', x.shape, cond.shape, mask.shape)
        out = self.forward(x, cond=cond, mask=mask)
        # print('lyr gen1', x.shape, out.shape)

        qout = self.vqx(out)
        etp = (out-qout).mean(-1) # [b,1] float
        # x = torch.cat([x, qout], dim=1)
        # x = torch.cat([x, qout[:,:-1]], dim=1)
        # return x, etp
        return qout, etp
# generate one tok unless uncertain ? then patch up?
# gen one only bec blt need to tok



# trans: btd->btd, train: shift1
# bltlyr: btd->btd+bpd , gentrain: shift1, artrain:btd+bpd->btd

# f_ j ; vs f_ t_
# full te
# full text

# is gpt + aetrain?



batch=25
d_model = 16
in_dim=128
# x = torch.randint(0, in_dim, (2, 7), device=device)
x = torch.rand((batch, 7, d_model), device=device)

model = BLTlayer(in_dim, d_model).to(device)
# model = BLTlayer(d_model).to(device)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683
optim = torch.optim.AdamW(model.parameters(), 1e-3, (0.9, 0.95)) # 1e-4 #3e-4

# in: qunatseq
# fwd: qunatx -> y_
# to_patch: x, y_ -> patches, mask
# gen: x, (patch) ->


# out, patches = model(x)
out = model(x)
print('out', out.shape)
# gloss = model.gen_loss(out)

patches, mask, patch_mask = model.to_patch(x, out)
print('patches, mask',patches.shape, mask.shape)
gloss = model.gen_loss(x)
aloss = model.ae_loss(x, patches, mask, patch_mask)
# loss = gloss + aloss

# patch = torch.rand(batch,1,d_model)
patch = torch.rand(1,1,d_model)
# out, mask = model.generate(x)
# out, mask = model.generate(x, patch)
x = torch.rand((1,7, d_model), device=device)
out, mask = model.generate_one(x, patch)
print(out.shape)
# out, mask_ = left_pad_tensor(torch.cat([x,y],dim=1), torch.cat([mask, mask],dim=1))

# print(patches.shape)
# print(out)


In [ ]:
# @title BLT me fsq
import torch
import torch.nn as nn
from torch.nn import functional as F

class BLT(nn.Module):
    def __init__(self, in_dim, d_model, out_dim=None, num_layers=1):
        super().__init__()
        self.pos_enc = RoPE(d_model, base=10000)
        self.tok_emb = nn.Embedding(in_dim, d_model)
        # self.layer = nn.ModuleList([BLTlayer(d_model, out_dim) for _ in range(num_layers)])
        self.layer = nn.ModuleList([BLTlayer(d_model, d_model, quantxp=(6*2**i,12*2**i)) for i in range(num_layers)])
        # self.layer = nn.ModuleList([BLTgpt(d_model, d_model, quantxp=(6,12))])
        # self.layer.append(BLTlayer(d_model, d_model, quantxp=(6*2**i,12*2**i)) for i in range(num_layers-1))
        self.out = nn.Linear(d_model, in_dim)

    # def forward(self, x): # [b,t]
    #     x = self.tok_emb(x)
    #     for level in self.layer:
    #         # out, x = level(x) # [b,t,d]
    #         out = level(x) # [b,t,d]
    #         patches, mask, patch_mask = level.to_patch(x, out)
    #         # outs.append(out)
    #         x = level.to_patch(out)
    #     x = self.lin(x)
    #     return x # [batch, num_patch, out_dim]

    def loss(self, src): # [b,t]
        x = self.tok_emb(src) # [b,t,d]
        tt_loss = 0
        for i, level in enumerate(self.layer):
            # out = level(x, mask=True) # [b,t,d]
            b,t = x.shape[:2]
            mask = torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(b,1,1) # F->mask # for F.scaled_dot_product_attention
            # out = level.perceiver(x, cond=x, mask='causal') # [b,t,d], [nlyr,b,d]
            out = level.perceiver(x, cond=x, mask=mask) # [b,t,d], [nlyr,b,d]
            out = level.lin(out)
            # tt_loss += level.gen_loss(x)
            if i==0: tt_loss += F.cross_entropy(self.out(out[:,:-1]).flatten(0,1), src[:,1:].flatten()) # [b*t,d], [b*t]
            else: tt_loss += F.mse_loss(out[:,:-1], x[:,1:])

            if i != len(self.layer)-1: # last layer no patching
                patches, mask, patch_mask = level.to_patch(x, out)
                tt_loss += level.ae_loss(x, patches, mask, patch_mask)
                x = patches
        return tt_loss



#     def generate(self, x):
#         patches = []
#         for j, patch in enumerate(patches):
#             if len(patch)>0:
#                 break
#         # patch in patches
#         for i in reversed(range(j)):
#             patch = patches[i].pop(0)
#             layer = self.layers[i]
#             # = self.perceiver(x, cond=patch)
#             out = layer.generate(x if i==0 else None, patch)
#             if i!=0: patches[i-1].append(out[:,-1])
# # out, mask_ = left_pad_tensor(torch.cat([x,y],dim=1), torch.cat([mask, mask],dim=1))

#         # entropies = entropy(out) # [b,t] float
#         # mask = patchify(entropies, 1.) # [b,t,t] # T at start of each patch, F otherwise
#         return out[:,-1]

    def generate_all(self, x): # [b,t]
        x = self.tok_emb(x) # [b,t,d]
        # src_lst = [x]
        # src_lst = [len(self.layer)*[]]
        # src_lst = [len(self.layer)*None]
        src_lst = len(self.layer)*[None]
        for i, level in enumerate(self.layer):
            # out, x = level(x) # [b,t,d]
            # out = level(x) # [b,t,d]

            out, _ = level.generate(x)
            # src_lst[i].append(out)
            src_lst[i] = out

            # outs.append(out)
            if i != len(self.layer)-1:
                patches, mask, patch_mask = level.to_patch(x, out)
                # src_lst[i+1].append(patches)
                src_lst[i+1] = patches

                # x = level.to_patch(out)
                # src_lst.append(x)
        # if self.lin: x = self.lin(x)
        # return x # [batch, num_patch, out_dim]
        patch = None
        # for i in reversed(range(len(self.layers))):
        for i, level in enumerate(reversed(self.layer)):
            # layer = self.layers[i]
            # x = src_lst.pop()
            x = src_lst[i-1]
            out, mask_ = level.generate(x, patch)
            # out, mask_ = left_pad_tensor(torch.cat([x,y],dim=1), torch.cat([mask, mask_],dim=1))
            # = self.perceiver(x, cond=patch)
            patch = x
            # out = layer.generate(x if i==0 else None, patch)
            # if i!=0: patches[i-1].append(out[:,-1])

        # entropies = entropy(out) # [b,t] float
        # mask = patchify(entropies, 1.) # [b,t,t] # T at start of each patch, F otherwise


        output = output[:, -1] # get logit for last character
        output = output/temperature
        output = F.softmax(output, dim=-1) # vocab_size to char
        ix = torch.multinomial(output, num_samples=1) # rand sample by output distribution
        x = torch.cat((x, ix),1)
        completion = ''.join([train_data.itos[int(i)] for i in x.flatten()])


        return out[:,-1]




    def generate_one(self, x, src_lst=None, etp_lst=None): # [b,t]
        x = self.tok_emb(x) # [b,t,d]
        b,t,d = x.shape
        src_lst = src_lst or [] #len(self.layer)*[None]
        etp_lst = etp_lst or [torch.empty((b,0), dtype=torch.float, device=device)]*len(self.layer)

        # for i, level in enumerate(self.layer):
        for i, (level, entropies) in enumerate(zip(self.layer, etp_lst)):
            out, entropies = level.generate_one(x)
            # print('gen one11', x.shape, out.shape)

            # mask = torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(b,1,1)
            # out = level(x, mask=mask)
            # entropies = (out-level.vqx(out)).mean(-1) # [b,t] float
            mask = patchify(entropies, 1.) # [1,t] # T at start of each patch, F otherwise

            # entropies = torch.cat([entropies, etp], dim=1) # [b,t]
            # etp_lst[i] = torch.cat([etp_lst[i], etp], dim=1) # [b,t]
            # mask = patchify(etp_lst[i], 1.) # [b,t,t] # T at start of each patch, F otherwise
            # print('gen one', mask.shape)
            # if mask[:,-1]: # newly generated tok is uncertain
            # print('gen one', x.shape, out.shape)
            patches, _, _ = level.to_patch(x, out)
            x = patches
            # else: break
            src_lst.append(out)

        if i==0: return x, src_lst, etp_lst # 1st lvl is confident [1, t+1, d]
        patch = None
        # for i in reversed(range(len(self.layers))):
        for i, v in enumerate(reversed(src_lst)):
            patch, etp = self.layer[len(src_lst)-i].generate_one(v, patch)
            # patch, etp = self.layer[len(src_lst)-i].generate_one(x, patch)

        patch = self.out(patch)
        return patch, src_lst, etp_lst


    def generate_all(self, context, max_steps=64, temperature=1): # [b,t]
        training = self.training
        self.eval()
        x = torch.tensor([train_data.stoi.get(c) for c in context], device=device).unsqueeze(0)
        for n in range(max_steps):
            with torch.no_grad():
                out, _, _ = self.generate_one(x)
            out = out[:,-1] # get logit for last character
            out = out/temperature
            out = F.softmax(out, dim=-1) # vocab_size to char
            ix = torch.multinomial(out, num_samples=1) # rand sample by output distribution
            print(x.shape, ix.shape)
            x = torch.cat((x, ix.unsqueeze(1)),1)
        completion = ''.join([train_data.itos[int(i)] for i in x.flatten()])
        if training: self.train()
        return completion


# pos_all, entropy mask;
# AE: mask->pos_patch?/conv1d? + mask+ latent(mean+*leant) -> patch; pos_patch?+ mask.T + patch(+*learnt?)-> og
# gen train: mse(x[1:],out[:-1])
# gen inference: x cond=x, cond=(x+patch(+*learnt?)), if entropy/bos, pop patch and regen

# # fsq: dont realize tokens in layer
# layer:
# posemb
# x less quants patch more quants
# quant after linout
# # quant before linout

# blt:
# tokemb
# linout to tok


# gen:
# gen lower only, until uncertain, then patch, gen top then patch cond bottom
# gen 1 tok. if uncertain, to_patch then top gen

vocab_size=50
in_dim, d_model, out_dim, num_layers = 4,16,vocab_size,1
# x = torch.rand(2,5,in_dim)
x = torch.rand(1,5,d_model)
model = BLT(vocab_size, d_model, out_dim, num_layers)
# model = BLTlayer(d_model, d_model)
# out = model(x)
# out = model.loss(x)
# print(out)
# out = model.generate(x)
x = torch.randint(0,vocab_size, (2,500))
out, src_lst, etp_lst = model.generate_one(x)
# out = model.generate_all(x)
print(out.shape)
# print(out)


torch.Size([2, 260, 16])


In [ ]:
# @title BLTlayer transformer gpt
import torch
import torch.nn as nn
from torch.nn import functional as F

# transformer
class BLTgpt(BLTlayer):
    # def __init__(self, in_dim, d_model=None, out_dim=None, num_layers=1):
    def __init__(self, in_dim, d_model=None, quantxp=(6,12), num_layers=1):
        super().__init__()
        d_model = d_model or in_dim
        self.tok_emb = nn.Embedding(in_dim, d_model)
        self.pos_enc = RoPE(d_model, base=100) # 10000
        # self.perceiver = Seq(*[AttentionBlock(d_model, n_heads=4, cond_dim=d_model) for _ in range(num_layers)])
        # self.lin = nn.Linear(d_model, d_model)
        self.out = nn.Linear(d_model, in_dim)

    # def forward(self, x, cond=None, mask=None): # [b,t,d], [b,t,t]
    #     x = self.pos_enc(x)
    #     b,t,d = x.shape
    #     # if mask==None: mask = torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(b,1,1) # F->mask # for F.scaled_dot_product_attention
    #     # print('BLTlayer',x.shape, causal_mask.shape)
    #     x = self.perceiver(x, cond=cond if cond!=None else x, mask=mask) # [b,t,d], [nlyr,b,d]
    #     out = self.lin(x)
    #     return out # [b,t,d_model]

    def gen_loss(self, src): # [b,t]
        x, y = src[:,:-1], src[:,1:]
        # x = self.pos_enc(x)
        x = self.pos_enc(self.tok_emb(x))
        b,t,d = x.shape
        mask = torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(b,1,1)
        # print('gpt gen_loss', out.shape, y.shape)
        x = self.perceiver(x, cond=x, mask=mask) # [b,t,d], [nlyr,b,d]
        out = self.out(x)
        loss = F.cross_entropy(out.flatten(0,1), y.flatten()) # [b*t,d], [b*t]
        return loss



    # def to_patch(self, x, out): # [b,t,d]
    #     entropies = (x-self.vqx(x)).mean(-1) # [b,t] float
    #     mask = patchify(entropies, 1.) # [b,t,t] # T at start of each patch, F otherwise
    #     patch_mask = padded_block_mask(mask) # [batch ,num_patch, seq_len]
    #     # print('to_patch', x.shape,mask.shape)

    #     latent = (x.unsqueeze(1) *patch_mask.unsqueeze(-1)).mean(2) # [b ,num_patch, t, d] -> [b ,num_patch, d]
    #     # latent = self.latent.repeat(1,mask.shape[1],1) # [b ,num_patch, d]
    #     latent = latent + self.patch_enc
    #     out = self.conv_emb(x.transpose(-2,-1)).transpose(-2,-1)
    #     patches = self.forward(latent, cond=out, mask=patch_mask) # [batch, num_patch, latent_dim/d_model]
    #     # print('BLTlayer',out.shape)
    #     patches = self.vqp(patches)
    #     return patches, mask, patch_mask # [b, num_patch, out_dim], [b ,num_patch, t]

    # def ae_loss(self, y, patches, mask, patch_mask): # [b,t,d], [b, n_patch, d], [b,t], [b, n_patch, t]
    #     # print('ae_loss', x.shape, self.tok_emb(x).shape) # [2, 7, 16], [2, 5, 128], [2, 7, 5]
    #     x = self.pos_enc(y)
    #     b,t,d = x.shape
    #     # cond = patch_mask.transpose(-2,-1).to(patches.dtype) @ patches # [b, t, out_dim]
    #     patch_mask = patch_mask.transpose(-2,-1) # [b, t, n_patch]
    #     causal_mask = block_diag(mask) & torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(b,1,1) # [b,t,t] # only attend within the patch & is causal
    #     # print('ae_loss', x.shape, patches.shape, patch_mask.shape) # [2, 7, 16], [2, 3, 16], [2, 7, 3]
    #     y_ = self.forward(x, cond=torch.cat([x, patches], dim=1), mask=torch.cat([causal_mask, patch_mask], dim=-1)) # cond: [b, t+n_patch, d]; mask: [batch, len_q, len_v] [b, t, t+n_patch]
    #     loss = F.mse_loss(y[mask], y_[mask])
    #     return loss

# AE: mask->pos_patch?/conv1d? + mask+ latent(mean+*leant) -> patch; pos_patch?+ mask.T + patch(+*learnt?)-> og

    def generate(self, x=None, patch=None): # [b,t,d], [b,t,d]
    # def generate(self, x=None, patch=None, mask=None, patch_mask=None): # [b,t], [b,t,d], [b,t], [b,n_patch,t]
        b,t,d = x.shape
        # entropies = torch.empty((b,0), device=device)
        # while True:
        for i in range(50):
            if patch!=None: cond = torch.cat([x[:,:-1], patch],1)
            else: cond = x[:,:-1]
            x = self.pos_enc(self.tok_emb(x))
            out = self.perceiver(x[:,-1:], cond=cond)
            out = self.out(out)

            # entropy = (out-qout).mean(-1) # [b,1] float
            entropi = entropy(out) # [b,1] float

            temperature = 1.
            out = out/temperature
            out = F.softmax(out, dim=-1) # vocab_size to char
            ix = torch.multinomial(out, num_samples=1) # rand sample by output distribution

            x = torch.cat([x, ix], dim=1)
            entropies = torch.cat([entropies, entropi], dim=1) # [b,t]
            # if i==0: continue # want gen at least one
            mask = patchify(entropies, 1.)
            if (mask.sum(1)>=2).all(): break

        # print('BLTlayer gen',mask.shape)
        mask = mask.cumsum(dim=1)>=2 # [b,t]
        out = x[:,t:]*mask.unsqueeze(-1)
        print('BLTlayer gen',x.shape, out.shape, mask.shape)

        # return x[1:-1]
        # return torch.cat((x, out),1), mask
        return torch.cat((x[:,:t], out),1), torch.cat([torch.ones(b,t, dtype=bool, device=device), mask], dim=1)








batch=25
d_model = 16
in_dim=128
x = torch.randint(0, in_dim, (2, 7), device=device)
# x = torch.rand((batch, 7, d_model), device=device)

model = BLTlayer(in_dim, d_model).to(device)
# model = BLTlayer(d_model).to(device)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683
optim = torch.optim.AdamW(model.parameters(), 1e-3, (0.9, 0.95)) # 1e-4 #3e-4

# in: qunatseq
# fwd: qunatx -> y_
# to_patch: x, y_ -> patches, mask
# gen: x, (patch) ->


# BLTgpt

# out, patches = model(x)
out = model(x)
print('out', out.shape)
# gloss = model.gen_loss(out)

patches, mask, patch_mask = model.to_patch(x, out)
print('patches, mask',patches.shape, mask.shape)
gloss = model.gen_loss(x)
aloss = model.ae_loss(x, patches, mask, patch_mask)
# loss = gloss + aloss

patch = torch.rand(batch,1,d_model)
# out, mask = model.generate(x)
out, mask = model.generate(x, patch)
print(out.shape)
# out, mask_ = left_pad_tensor(torch.cat([x,y],dim=1), torch.cat([mask, mask],dim=1))

# print(patches.shape)
# print(out)


In [ ]:
# @title BLTlayer transformer tok
import torch
import torch.nn as nn
from torch.nn import functional as F

# transformer
class BLTlayer(nn.Module):
    def __init__(self, in_dim, d_model=None, out_dim=None, num_layers=1):
        super().__init__()
        d_model = d_model or in_dim
        self.tok_emb = nn.Embedding(in_dim, d_model)
        self.pos_enc = RoPE(d_model, base=100) # 10000
        # self.latent = nn.Parameter(torch.randn(1,1,d_model)*.02) # init
        # self.perceiver = AttentionBlock(d_model, n_heads=4, cond_dim=d_model)
        self.perceiver = Seq(*[AttentionBlock(d_model, n_heads=4, cond_dim=d_model) for _ in range(num_layers)])
        self.lin = nn.Linear(d_model, in_dim)
        # self.out = nn.Linear(d_model, out_dim)
        self.patch_enc = nn.Parameter(torch.randn(1,1,d_model)*.02)
        self.conv_emb = nn.Conv1d(d_model, d_model, 1)

    # def forward(self, x, mask=None): # [b,t], [b,t,t]
    def forward(self, x, cond=None, mask=None): # [b,t], [b,t,t]
        x = self.pos_enc(self.tok_emb(x))
        b,t,d = x.shape
        # if mask==None:
        #     mask = torch.ones((t,t), dtype=bool, device=device).tril().repeat(b,1,1) # for F.scaled_dot_product_attention
        #     # mask = torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(b,1,1) # F->mask
        # print('BLTlayer',x.shape, causal_mask.shape)
        x = self.perceiver(x, cond=cond or x, mask=mask) # [b,t,d], [nlyr,b,d]
        out = self.lin(x)
        return out # [b,t,d_model]

    def gen_loss(self, src): # [b,t,d]
        x, y = src[:,:-1], src[:,1:]
        b,t = x.shape
        mask = torch.tril(torch.ones((t,t), device=device)).bool().unsqueeze(0).repeat(b,1,1)
        out = self.forward(x, mask=mask) # [b,t,d], [nlyr,b,d]
        # print('gen_loss', out.shape, y.shape)
        loss = F.cross_entropy(out.flatten(0,1), y.flatten()) # [b*t,d], [b*t]
        return loss

    def to_patch(self, x, out): # [b,t]
        entropies = entropy(out) # [b,t] float
        mask = patchify(entropies, 1.) # [b,t,t] # T at start of each patch, F otherwise
        patch_mask = padded_block_mask(mask) # [batch ,num_patch, seq_len]
        # print('to_patch', x.shape,mask.shape)
        x = self.tok_emb(x)

        latent = (x.unsqueeze(1) *patch_mask.unsqueeze(-1)).mean(2) # [b ,num_patch, t, d] -> [b ,num_patch, d]
        # latent = self.latent.repeat(1,mask.shape[1],1) # [b ,num_patch, d]
        latent = latent + self.patch_enc
        out = self.conv_emb(x.transpose(-2,-1)).transpose(-2,-1)
        patches = self.perceiver(latent, cond=out, mask=patch_mask) # [batch, num_patch, latent_dim/d_model]
        # patches = self.lin(patches)
        # patches = self.out(patches)
        # patches = self.forward(latent, cond=out, mask=mask) # [batch, num_patch, latent_dim/d_model]
        # print('BLTlayer',out.shape)
        # if self.lin: patches = self.lin(patches)
        return patches, mask, patch_mask # [b, num_patch, out_dim], [b ,num_patch, t]

    def ae_loss(self, y, patches, mask, patch_mask): # [b,t,d], [b, n_patch, d], [b, n_patch, t]
        # print('ae_loss', x.shape, self.tok_emb(x).shape) # [2, 7, 16], [2, 5, 128], [2, 7, 5]
        x = self.pos_enc(self.tok_emb(y))
        b,t,d = x.shape
        # cond = patch_mask.transpose(-2,-1).to(patches.dtype) @ patches # [b, t, out_dim]
        patch_mask = patch_mask.transpose(-2,-1) # [b, t, n_patch]

        causal_mask = block_diag(mask) & torch.tril(torch.ones((t,t), device=device)).bool().unsqueeze(0).repeat(b,1,1) # [b,t,t]

        # x_ = self.perceiver(x, cond=cond, mask=mask)
        # print('ae_loss', x.shape, patches.shape, patch_mask.shape) # [2, 7, 16], [2, 3, 16], [2, 7, 3]
        out = self.perceiver(x, cond=torch.cat([x, patches], dim=1), mask=torch.cat([causal_mask, patch_mask], dim=-1)) # cond: [b, t+n_patch, d]; mask: [batch, len_q, len_v] [b, t, t+n_patch]
        out = self.lin(out)
        # x_ = self.out(x_)
        # x_ = self.forward(x, cond=cond, mask=mask)
        # loss = F.mse_loss(x, x_)
        print('ae_loss', x.shape, out.shape, mask.shape)
        # loss = F.mse_loss(y[mask], x_[mask])
        loss = F.cross_entropy(out.flatten(0,1), y.flatten()) # [b*t,d], [b*t]
        return out

    # def loss(self, x, out): # [b,t,d]
    #     out = model(x)
    #     gloss = model.gen_loss(out)
    #     patches, mask = model.to_patch(x, out)
    #     aloss = model.ae_loss(x, patches, mask)
    #     # loss = gloss + aloss

# pos_all, entropy mask;
# AE: mask->pos_patch?/conv1d? + mask+ latent(mean+*leant) -> patch; pos_patch?+ mask.T + patch(+*learnt?)-> og
# gen train: mse(x[1:],out[:-1])
# gen inference: x cond=x, cond=(x+patch(+*learnt?)), if entropy/bos, pop patch and regen


    def generate(self, x=None, patch=None): # [b,t], [b,t,d]
    def generate(self, x=None, patch=None, mask=None, patch_mask=None): # [b,t], [b,t,d], [b,t], [b,n_patch,t]
        # if patch==None: patch=x
        x = self.pos_enc(self.tok_emb(x))
        b,t,d = x.shape
        out = torch.empty((b,0,d), device=device)
        while True:
        # for i in range(5):
            print('generate x',x.shape, x[:,-1:].shape)
            # src = self.pos_enc(torch.cat((x, out),1))
            if patch!=None: cond = torch.cat((x[:,:-1], patch),1)
            else: cond = x[:,:-1]

            # # output = self.perceiver(src, cond=patch)
            # # output = self.perceiver(src, cond=torch.cat((src, patch),1))
            # output = self.perceiver(src, cond=torch.cat((src[:,-1:], patch),1))
            # output = self.forward(x[:,-1:], cond=torch.cat((x, patch),1))
            output = self.forward(x[:,-1:], cond=cond)
            # # print('generate x',x.shape)
            # # print('generate output',output)

            # out = torch.cat([out, output[:,-1:]], dim=1)
            out = torch.cat([out, output[:,:0]], dim=1)
            # out = torch.cat([out, output], dim=1)
            entropies = entropy(out) # [b,t] float

            # print('entropies',entropies)
            mask = patchify(entropies, 1.)
            # print('generate',out.shape, mask.shape)
            # print(mask)
            # print((mask.sum(1)>=2))
            if (mask.sum(1)>=2).all():
            # if (mask.sum(1)>=2 or (EOS_IDX == mask).any(1)).all():
                break

            temperature=1.
            # output = output[:, -1] # get logit for last character
            output = output/temperature
            output = F.softmax(output, dim=-1) # vocab_size to char
            ix = torch.multinomial(output, num_samples=1) # rand sample by output distribution
            x = torch.cat((x, ix),1)


        # print('BLTlayer gen',mask.shape)
        print(mask)
        # mask = padded_block_mask(mask)[:,1:]
        mask = mask.cumsum(dim=1)>=2 # [b,t]
        print(mask)
        out.unsqueeze(1) *mask.unsqueeze(-1)

        # return x[1:-1]
        # return torch.cat((x, out),1), mask
        return torch.cat((x, out),1), mask



# tok in/out
# typical causal generation
# stop gen when trans entroy high
# typical autoregressive

# ae train




# def generate(model, context, max_steps=64, temperature=1):
#     x = torch.tensor([train_data.stoi.get(c) for c in context], device=device).unsqueeze(0)
#     model.eval()
#     for n in range(max_steps):
#         with torch.no_grad():
#             output = model(x)
#         # print('generate', output.shape, hidden.shape)
#         output = output[:, -1] # get logit for last character
#         output = output/temperature
#         output = F.softmax(output, dim=-1) # vocab_size to char
#         ix = torch.multinomial(output, num_samples=1) # rand sample by output distribution
#         x = torch.cat((x, ix),1)
#     completion = ''.join([train_data.itos[int(i)] for i in x.flatten()])
#     return completion

d_model = 16
in_dim=128
x = torch.randint(0, in_dim, (2, 7), device=device)

model = BLTlayer(in_dim, d_model).to(device)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683
optim = torch.optim.AdamW(model.parameters(), 1e-3, (0.9, 0.95)) # 1e-4 #3e-4

# # out, patches = model(x)
# out = model(x)
# print('out', out.shape)
# # gloss = model.gen_loss(out)
# gloss = model.gen_loss(x)

# # patch = torch.rand(2,1,d_model)
# patches, mask, patch_mask = model.to_patch(x, out)
# print('patches, mask',patches.shape, mask.shape)
# aloss = model.ae_loss(x, patches, mask, patch_mask)
# # loss = gloss + aloss

out, mask = model.generate(x)
# out, mask = model.generate(x, patch)
# print(out.shape)
# out, mask_ = left_pad_tensor(torch.cat([x,y],dim=1), torch.cat([mask, mask],dim=1))

# print(patches.shape)
# print(out)


In [ ]:
# @title BLT me tok
import torch
import torch.nn as nn
from torch.nn import functional as F

class BLT(nn.Module):
    def __init__(self, in_dim, d_model, out_dim=None, num_layers=1):
        super().__init__()
        # self.layer = nn.ModuleList([BLTlayer(d_model, out_dim) for _ in range(num_layers)])
        self.layer = nn.ModuleList([BLTlayer(d_model, d_model) for _ in range(num_layers)])
        # self.lin = nn.Linear(d_model, out_dim) if out_dim and out_dim != d_model else None

    def forward(self, x): # [b,t]
        for level in self.layer:
            # out, x = level(x) # [b,t,d]
            out = level(x) # [b,t,d]
            # outs.append(out)
            x = level.to_patch(out)
        # if self.lin: x = self.lin(x)
        return x # [batch, num_patch, out_dim]


#     def generate(self, x):
#         patches = []
#         for j, patch in enumerate(patches):
#             if len(patch)>0:
#                 break
#         # patch in patches
#         for i in reversed(range(j)):
#             patch = patches[i].pop(0)
#             layer = self.layers[i]
#             # = self.perceiver(x, cond=patch)
#             out = layer.generate(x if i==0 else None, patch)
#             if i!=0: patches[i-1].append(out[:,-1])
# # out, mask_ = left_pad_tensor(torch.cat([x,y],dim=1), torch.cat([mask, mask],dim=1))

#         # entropies = entropy(out) # [b,t] float
#         # mask = patchify(entropies, 1.) # [b,t,t] # T at start of each patch, F otherwise
#         return out[:,-1]

    def generate(self, x):
        # batch, seq_len = x.shape
        # x = self.pos_enc(self.tok_emb(x)) # [batch_size, seq_len, d_model]
        src_lst = [x]
        msk_lst = []
        for level in self.layer:
            # out, x = level(x) # [b,t,d]
            out = level(x) # [b,t,d]
            # outs.append(out)
            if i != len(self.levels)-1
                x, mask = model.to_patch(x, out)
                # x = level.to_patch(out)
                src_lst.append(x)
        # if self.lin: x = self.lin(x)
        # return x # [batch, num_patch, out_dim]
        patch = None
        for i in reversed(range(len(self.layers))):
            layer = self.layers[i]
            x = src_lst.pop()
            mask = msk_lst.pop()
            out, mask_ = model.generate(x, patch)
            out, mask_ = left_pad_tensor(torch.cat([x,y],dim=1), torch.cat([mask, mask_],dim=1))
            # = self.perceiver(x, cond=patch)
            patch = x
            # out = layer.generate(x if i==0 else None, patch)
            if i!=0: patches[i-1].append(out[:,-1])

        # entropies = entropy(out) # [b,t] float
        # mask = patchify(entropies, 1.) # [b,t,t] # T at start of each patch, F otherwise
        return out[:,-1]

# out, patches = model(x)
out = model(x)
print('out', out.shape)
patches, mask, patch_mask = model.to_patch(x, out)
print('patches, mask',patches.shape, mask.shape)
gloss = model.gen_loss(x)
aloss = model.ae_loss(x, patches, mask, patch_mask)
# loss = gloss + aloss


    def loss(self, x):
        # batch, seq_len = x.shape
        x = self.tok_emb(x) #[batch_size, seq_len, d_model]
        tt_loss = 0
        for level in self.layer:
            # out, x = level(x) # [b,t,d]
            # out = level(x) # [b,t,d]
            out = level(x) # [b,t,d]
            # outs.append(out)
            # loss = F.mse_loss(x[:,1:], out[:,:-1])
            # tt_loss += loss
            # x, mask = level.to_patch(out)


            tt_loss += model.gen_loss(out)
            if i != len(self.levels)-1
                patches, mask = model.to_patch(x, out)
                tt_loss += model.ae_loss(x, patches, mask)

            # # output = self.perceiver(src, cond=torch.cat((src, patch),1))
            # output = self.perceiver(pos(torch.cumsum()), cond=patch, mask=mask)

        # if self.lin: x = self.lin(x)
        # return x # [batch, num_patch, out_dim]
        return tt_loss#, out


# pos_all, entropy mask;
# AE: mask->pos_patch?/conv1d? + mask+ latent(mean+*leant) -> patch; pos_patch?+ mask.T + patch(+*learnt?)-> og
# gen train: mse(x[1:],out[:-1])
# gen inference: x cond=x, cond=(x+patch(+*learnt?)), if entropy/bos, pop patch and regen


vocab_size=50
in_dim, d_model, out_dim, num_layers = 4,16,vocab_size,1
# x = torch.rand(2,5,in_dim)
x = torch.randint(0,vocab_size, (2,500))
model = BLT(d_model, out_dim, num_layers)
# model = BLTlayer(d_model, d_model)
# out = model(x)
# out = model.loss(x)
out = model.generate(x)
# print(out.shape)
print(out)


In [ ]:
# @title BLTlayer transformer old
import torch
import torch.nn as nn
from torch.nn import functional as F


# transformer
class BLTlayer(nn.Module):
    # def __init__(self, in_dim, d_model=None, out_dim=None, num_layers=1):
    def __init__(self, d_model, out_dim=None, num_layers=1):
        super().__init__()
        # d_model = d_model or in_dim
        self.pos_enc = RoPE(d_model, base=100) # 10000
        # self.latent = nn.Parameter(torch.randn(1,1,d_model)*.02)
        self.perceiver = AttentionBlock(d_model, n_heads=4, cond_dim=d_model)
        self.lin = nn.Linear(d_model, out_dim) if out_dim and out_dim != d_model else None
        self.bos = nn.Parameter(torch.randn(1,1,d_model)*.02)
        self.eos = nn.Parameter(torch.randn(1,1,d_model)*.02)
        self.patch_enc = nn.Parameter(torch.randn(1,1,d_model)*.02)
        self.conv_emb = nn.Conv1d(d_model, d_model, 1)

    # def forward(self, x): # [b,t,d]
    def forward(self, x, mask=None): # [b,t,d]
        x = self.pos_enc(x)

        b,t,d = x.shape
        mask = torch.ones((t,t), dtype=bool, device=device).tril().repeat(b,1,1) # for F.scaled_dot_product_attention
        mask = torch.tril(torch.ones((t,t), dtype=bool, device=device)).repeat(b,1,1) # F->mask
        # with torch.no_grad():
        # print('BLTlayer',x.shape, causal_mask.shape)
        out = self.perceiver(x, cond=x, mask=mask) # [b,t,d], [nlyr,b,d]

        return out # [b,t,d_model]

    def gen_loss(self, src): # [b,t,d]
        x, y = src[:,:-1], src[:,1:]
        b,t,d = x.shape
        mask = torch.tril(torch.ones((t,t), device=device)).bool().unsqueeze(0).repeat(b,1,1)
        out = self.perceiver(x, cond=x, mask=mask) # [b,t,d], [nlyr,b,d]
        loss = F.mse_loss(out, y)
        return loss

    def to_patch(self, x, out): # [b,t,d]
        entropies = entropy(out) # [b,t] float
        # entropies = out @ self.eos.transpose(-2,-1)
        mask = patchify(entropies, 1.) # [b,t,t] # T at start of each patch, F otherwise
        mask = padded_block_mask(mask) # [batch ,num_patch, seq_len]
        # latent = (out.unsqueeze(1) *mask.unsqueeze(-1)).mean(2) # [b ,num_patch, t, d] -> [b ,num_patch, d]
        latent = (x.unsqueeze(1) *mask.unsqueeze(-1)).mean(2) # [b ,num_patch, t, d] -> [b ,num_patch, d]
        latent = latent + self.patch_enc
        # latent = self.latent.repeat(1,mask.shape[1],1) # [b ,num_patch, d]
        out = self.conv_emb(x.transpose(-2,-1)).transpose(-2,-1)
        patches = self.perceiver(latent, cond=out, mask=mask) # [batch, num_patch, latent_dim/d_model]
        # print('BLTlayer',out.shape)

        if self.lin: patches = self.lin(patches)
        return patches, mask # [batch, num_patch, out_dim], [batch ,num_patch, seq_len]

# pos_all, entropy mask;
# AE: mask->pos_patch?/conv1d? + mask+ latent(mean+*leant) -> patch; pos_patch?+ mask.T + patch(+*learnt?)-> og
# gen train: mse(x[1:],out[:-1])
# gen inference: x cond=x, cond=(x+patch(+*learnt?)), if entropy/bos, pop patch and regen

    def ae_loss(self, x, patches, mask): # [b,t,d], [b, n_patch, latent_dim/d_model], [b, n_patch, t]
        b,t,d = x.shape
        cond = mask.transpose(-2,-1).to(patches.dtype) @ patches # [b, t, latent_dim/d_model]
        mask = torch.tril(torch.ones((t,t), device=device)).bool().unsqueeze(0).repeat(b,1,1)
        x_ = self.perceiver(x, cond=cond, mask=mask)
        loss = F.mse_loss(x, x_)
        return out

    def generate(self, x=None, patch=None):
        # x = x or self.bos.repeat(patch.shape[0],1,1)
        if patch==None: patch=x
        # x=patch=self.bos
        b,t,d = x.shape
        out = torch.empty((b,0,d), device=device)
        while True:
        # for i in range(5):
            # print('generate x',x.shape)
            src = self.pos_enc(torch.cat((x, out),1))

            # output = self.perceiver(src, cond=patch)
            # output = self.perceiver(src, cond=torch.cat((src, patch),1))
            output = self.perceiver(src, cond=torch.cat((src[:,-1:], patch),1))
            # print('generate x',x.shape)
            # print('generate output',output)

            # out = torch.cat([out, output[:,-1:]], dim=1)
            out = torch.cat([out, output[:,:0]], dim=1)

            # entropies = entropy(out) # [b,t] float
            entropies = (out @ self.eos.transpose(-2,-1)).squeeze(-1).cumsum(1) # [b,t]
            # print('entropies',entropies)
            mask = patchify(entropies, 1.)
            # print('generate',out.shape, mask.shape)
            # print(mask)
            # print((mask.sum(1)>=2))
            if (mask.sum(1)>=2).all():
            # if (mask.sum(1)>=2 or (EOS_IDX == mask).any(1)).all():
                break
        # print('BLTlayer gen',mask.shape)
        print(mask)
        # mask = padded_block_mask(mask)[:,1:]
        mask = mask.cumsum(dim=1)>=2 # [b,t]
        print(mask)
        out.unsqueeze(1) *mask.unsqueeze(-1)

        # return x[1:-1]
        return torch.cat((x, out),1), mask
# BOS_IDX, EOS_IDX


# def generate(model, context, max_steps=64, temperature=1):
#     x = torch.tensor([train_data.stoi.get(c) for c in context], device=device).unsqueeze(0)
#     model.eval()
#     for n in range(max_steps):
#         with torch.no_grad():
#             output = model(x)
#         # print('generate', output.shape, hidden.shape)
#         output = output[:, -1] # get logit for last character
#         output = output/temperature
#         output = F.softmax(output, dim=-1) # vocab_size to char
#         ix = torch.multinomial(output, num_samples=1) # rand sample by output distribution
#         x = torch.cat((x, ix),1)
#     completion = ''.join([train_data.itos[int(i)] for i in x.flatten()])
#     return completion

d_model = 4
x = torch.rand(2,3,d_model)
patch = torch.rand(2,1,d_model)
model = BLTlayer(d_model, d_model).to(device)
# print(model)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683
optim = torch.optim.AdamW(model.parameters(), 1e-3, (0.9, 0.95)) # 1e-4 #3e-4

# out, patches = model(x)
out = model(x)
gloss = model.gen_loss(out)
patches, mask = model.to_patch(x, out)
aloss = model.ae_loss(x, patches, mask)
# loss = gloss + aloss

# out = model.loss(x)
# out, mask = model.generate(x)
# out, mask = model.generate(x, patch)
# print(out.shape)
# out, mask_ = left_pad_tensor(torch.cat([x,y],dim=1), torch.cat([mask, mask],dim=1))

# print(patches.shape)
# print(out)


276


In [ ]:
# @title BLT me
import torch
import torch.nn as nn
from torch.nn import functional as F

class BLT(nn.Module):
    # def __init__(self, in_dim, d_model, out_dim, num_layers):
    def __init__(self, d_model, out_dim=None, num_layers=1):
        super().__init__()
        self.tok_emb = nn.Embedding(50, d_model)
        # self.pos_enc = RoPE(d_model, base=10000)
        # self.layer = nn.ModuleList([BLTlayer(d_model, out_dim) for _ in range(num_layers)])
        self.layer = nn.ModuleList([BLTlayer(d_model, d_model) for _ in range(num_layers)])
        # self.lin = nn.Linear(d_model, out_dim) if out_dim and out_dim != d_model else None

    def forward(self, x):
        # batch, seq_len = x.shape
        # x = self.pos_enc(self.tok_emb(x)) # [batch_size, seq_len, d_model]
        for level in self.layer:
            # out, x = level(x) # [b,t,d]
            out = level(x) # [b,t,d]
            # outs.append(out)
            x = level.to_patch(out)
        # if self.lin: x = self.lin(x)
        return x # [batch, num_patch, out_dim]


#     def generate(self, x):
#         patches = []
#         for j, patch in enumerate(patches):
#             if len(patch)>0:
#                 break
#         # patch in patches
#         for i in reversed(range(j)):
#             patch = patches[i].pop(0)
#             layer = self.layers[i]
#             # = self.perceiver(x, cond=patch)
#             out = layer.generate(x if i==0 else None, patch)
#             if i!=0: patches[i-1].append(out[:,-1])
# # out, mask_ = left_pad_tensor(torch.cat([x,y],dim=1), torch.cat([mask, mask],dim=1))

#         # entropies = entropy(out) # [b,t] float
#         # mask = patchify(entropies, 1.) # [b,t,t] # T at start of each patch, F otherwise
#         return out[:,-1]

    def generate(self, x):
        # batch, seq_len = x.shape
        # x = self.pos_enc(self.tok_emb(x)) # [batch_size, seq_len, d_model]
        src_lst = [x]
        msk_lst = []
        for level in self.layer:
            # out, x = level(x) # [b,t,d]
            out = level(x) # [b,t,d]
            # outs.append(out)
            if i != len(self.levels)-1
                x, mask = model.to_patch(x, out)
                # x = level.to_patch(out)
                src_lst.append(x)
        # if self.lin: x = self.lin(x)
        # return x # [batch, num_patch, out_dim]
        patch = None
        for i in reversed(range(len(self.layers))):
            layer = self.layers[i]
            x = src_lst.pop()
            mask = msk_lst.pop()
            out, mask_ = model.generate(x, patch)
            out, mask_ = left_pad_tensor(torch.cat([x,y],dim=1), torch.cat([mask, mask_],dim=1))
            # = self.perceiver(x, cond=patch)
            patch = x
            # out = layer.generate(x if i==0 else None, patch)
            if i!=0: patches[i-1].append(out[:,-1])

        # entropies = entropy(out) # [b,t] float
        # mask = patchify(entropies, 1.) # [b,t,t] # T at start of each patch, F otherwise
        return out[:,-1]

# out, patches = model(x)
out = model(x)
print('out', out.shape)
patches, mask, patch_mask = model.to_patch(x, out)
print('patches, mask',patches.shape, mask.shape)
gloss = model.gen_loss(x)
aloss = model.ae_loss(x, patches, mask, patch_mask)
# loss = gloss + aloss

# out = model.loss(x)
# out, mask = model.generate(x)
# out, mask = model.generate(x, patch)
# out, mask_ = left_pad_tensor(torch.cat([x,y],dim=1), torch.cat([mask, mask],dim=1))


    def loss(self, x):
        # batch, seq_len = x.shape
        x = self.tok_emb(x) #[batch_size, seq_len, d_model]
        tt_loss = 0
        for level in self.layer:
            # out, x = level(x) # [b,t,d]
            # out = level(x) # [b,t,d]
            out = level(x) # [b,t,d]
            # outs.append(out)
            # loss = F.mse_loss(x[:,1:], out[:,:-1])
            # tt_loss += loss
            # x, mask = level.to_patch(out)


            tt_loss += model.gen_loss(out)
            if i != len(self.levels)-1
                patches, mask = model.to_patch(x, out)
                tt_loss += model.ae_loss(x, patches, mask)

            # # output = self.perceiver(src, cond=torch.cat((src, patch),1))
            # output = self.perceiver(pos(torch.cumsum()), cond=patch, mask=mask)

        # if self.lin: x = self.lin(x)
        # return x # [batch, num_patch, out_dim]
        return tt_loss#, out


# pos_all, entropy mask;
# AE: mask->pos_patch?/conv1d? + mask+ latent(mean+*leant) -> patch; pos_patch?+ mask.T + patch(+*learnt?)-> og
# gen train: mse(x[1:],out[:-1])
# gen inference: x cond=x, cond=(x+patch(+*learnt?)), if entropy/bos, pop patch and regen


vocab_size=50
in_dim, d_model, out_dim, num_layers = 4,16,vocab_size,1
# x = torch.rand(2,5,in_dim)
x = torch.randint(0,vocab_size, (2,500))
model = BLT(d_model, out_dim, num_layers)
# model = BLTlayer(d_model, d_model)
# out = model(x)
# out = model.loss(x)
out = model.generate(x)
# print(out.shape)
print(out)


In [ ]:
# @title BLTlayer
import torch
import torch.nn as nn
from torch.nn import functional as F

# first
class BLTlayer(nn.Module):
    # def __init__(self, in_dim, d_model=None, out_dim=None, num_layers=1):
    def __init__(self, d_model, out_dim=None, num_layers=1):
        super().__init__()
        # d_model = d_model or in_dim
        # self.rnn = nn.GRU(in_dim, d_model, num_layers, batch_first=True)
        self.rnn = nn.GRU(d_model, d_model, num_layers, batch_first=True)
        # self.latent = nn.Parameter(torch.randn(1,1,d_model)*.02)
        self.perceiver = AttentionBlock(d_model, n_heads=4, cond_dim=d_model)
        self.lin = nn.Linear(d_model, out_dim) if out_dim and out_dim != d_model else None

    def forward(self, x, h0=None): # [batch_size, seq_len, d_model]
    # def forward(self, x): # [batch_size, seq_len, d_model]
        out, h0 = self.rnn(x, h0) # [b,t,d], [nlyr,b,d]
        # out = self.perceiver(x) # [b,t,d], [nlyr,b,d]
        entropies = entropy(out) # [b,t] float
        mask = patchify(entropies, 1.) # [b,t,t] # T at start of each patch, F otherwise
        mask = padded_block_mask(mask) # [batch ,num_patch, seq_len]
        latent = (x.unsqueeze(1) *mask.unsqueeze(-1)).mean(2) # [b ,num_patch, t, d] -> [b ,num_patch, d]
        # latent = self.latent.repeat(1,mask.shape[1],1) # [b ,num_patch, d]
        out = self.perceiver(latent, cond=x, mask=mask) # [batch, num_patch, latent_dim/d_model]
        # print('BLTlayer',out.shape)
        if self.lin: out = self.lin(out)
        return out # [batch, num_patch, out_dim]


# rnn
class BLTlayer(nn.Module):
    # def __init__(self, in_dim, d_model=None, out_dim=None, num_layers=1):
    def __init__(self, d_model, out_dim=None, cond_dim=None, num_layers=1):
        super().__init__()
        # d_model = d_model or in_dim
        # self.rnn = nn.GRU(in_dim, d_model, num_layers, batch_first=True)
        self.rnn = nn.GRU(d_model, d_model, num_layers, batch_first=True)
        # # self.latent = nn.Parameter(torch.randn(1,1,d_model)*.02)
        self.lin = nn.Linear(d_model, out_dim) if out_dim and out_dim != d_model else None
        # self.cond_dim = d_model

    def forward(self, x, h0=None): # [batch_size, seq_len, d_model]
        # torch.cat([x, cond])
        out, h0 = self.rnn(x, h0) # [b,t,d], [nlyr,b,d]
        entropies = entropy(out) # [b,t] float
        mask = patchify(entropies, 1.) # [b,t,t] # T at start of each patch, F otherwise
        mask = padded_block_mask(mask) # [batch ,num_patch, seq_len]
        latent = (x.unsqueeze(1) *mask.unsqueeze(-1)).mean(2) # [b ,num_patch, t, d] -> [b ,num_patch, d]

        out, h0 = self.rnn(torch.cat([x, latent]), h0) # [b,t,d], [nlyr,b,d]

        # print('BLTlayer',out.shape)
        if self.lin: out = self.lin(out)
        return out # [batch, num_patch, out_dim]

# transformer
class BLTlayer(nn.Module):
    # def __init__(self, in_dim, d_model=None, out_dim=None, num_layers=1):
    def __init__(self, d_model, out_dim=None, num_layers=1):
        super().__init__()
        # d_model = d_model or in_dim
        # self.latent = nn.Parameter(torch.randn(1,1,d_model)*.02)
        self.perceiver = AttentionBlock(d_model, n_heads=4, cond_dim=d_model)
        self.lin = nn.Linear(d_model, out_dim) if out_dim and out_dim != d_model else None
        self.bos = nn.Parameter(torch.randn(1,1,d_model)*.02)

    def forward(self, x): # [batch_size, seq_len, d_model]
        b,t,d = x.shape
        causal_mask = torch.tril(torch.ones((t,t), device=device)).bool().unsqueeze(0).repeat(b,1,1)
        # with torch.no_grad():
        # print('BLTlayer',x.shape, causal_mask.shape)
        out = self.perceiver(x, cond=x, mask=causal_mask) # [b,t,d], [nlyr,b,d]
        entropies = entropy(out) # [b,t] float
        mask = patchify(entropies, 1.) # [b,t,t] # T at start of each patch, F otherwise
        mask = padded_block_mask(mask) # [batch ,num_patch, seq_len]
        latent = (x.unsqueeze(1) *mask.unsqueeze(-1)).mean(2) # [b ,num_patch, t, d] -> [b ,num_patch, d]
        # latent = self.latent.repeat(1,mask.shape[1],1) # [b ,num_patch, d]
        patches = self.perceiver(latent, cond=x, mask=mask) # [batch, num_patch, latent_dim/d_model]
        # print('BLTlayer',out.shape)
        if self.lin: patches = self.lin(patches)
        return out, patches # [batch, num_patch, out_dim]

    def generate(self, x, patch=None):
        x = self.bos
        if patch==None: patch=x
        while True:
            x = self.perceiver(x, cond=patch)
            entropies = entropy(out) # [b,t] float
            mask = patchify(entropies, 1.)
            if (mask.sum(1)>=2).all():
            # if (mask.sum(1)>=2 or (EOS_IDX == mask).any(1)).all():
                mask = padded_block_mask(mask)[:,1:]
                x.unsqueeze(1) *mask.unsqueeze(-1)
                return x[1:-1]
# BOS_IDX, EOS_IDX


In [ ]:
# @title BLT me
import torch
import torch.nn as nn
from torch.nn import functional as F

class BLT(nn.Module):
    def __init__(self, in_dim, d_model, out_dim, num_layers):
        super().__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.tok_emb = nn.Embedding(50, d_model)

        self.rnn = nn.GRU(d_model, d_model, num_layers, batch_first=True)
        # self.latent = nn.Parameter(torch.randn(1,1,d_model)*.02)
        self.perceiver = AttentionBlock(d_model, n_heads=4, cond_dim=d_model)
        self.fc = nn.Linear(d_model, out_dim)

    def forward(self, x, h0=None):
        # batch, seq_len = x.shape
        x = self.tok_emb(x) #[batch_size, seq_len, d_model]
        # if h0==None: h0 = torch.zeros(1, batch, self.d_model, device=device) # [nlyr,b,d]

        # for level in self.rnn:
        # out, h0 = level(x, h0)
        out, h0 = self.rnn(x, h0) # [b,t,d], [nlyr,b,d]
        entropies = entropy(out) # [b,t] float

        mask = patchify(entropies, 1.) # [b,t,t] # T at start of each patch, F otherwise
        mask = padded_block_mask(mask) # [batch ,num_patch, seq_len]
        latent = (x.unsqueeze(1) *mask.unsqueeze(-1)).mean(2) # [b ,num_patch, t, d] -> [b ,num_patch, d]
        # latent = self.latent.repeat(1,mask.shape[1],1) # [b ,num_patch, d]
        out = self.perceiver(latent, cond=x, mask=mask) # [batch, num_patch, latent_dim/d_model]
        print('out',out.shape)

        out = self.fc(out)
        return out # [batch, num_patch, out_dim]


vocab_size=50
in_dim, d_model, out_dim, num_layers = 4,16,vocab_size,1
# x = torch.rand(2,5,in_dim)
x = torch.randint(0,vocab_size, (2,5))
model = BLT(in_dim, d_model, out_dim, num_layers)
out = model(x)
print(out.shape)


out torch.Size([2, 3, 16])
torch.Size([2, 3, 50])


In [ ]:
# @title block_diag

# patch_start_ids = find_entropy_patch_start_ids(
#     scores,
#     self.patch_size,
#     include_next_token=include_next_token,
#     threshold=threshold if threshold is not None else self.threshold,
#     threshold_add=self.threshold_add,
#     monotonicity=self.monotonicity,
# )
# patch_lengths = patch_lengths_from_start_ids(
#     patch_start_ids, seq_len_next_tok
# )


def block_diag(mask): # [b,t]
    """Convert a 1D bool mask to a block diagonal matrix, where each `True` starts a new block.
    mask = [1,0,1,1] ->
    [[1,1,0,0],
     [1,1,0,0],
     [0,0,1,0],
     [0,0,0,1]]"""
    block_ids = mask.cumsum(1) # Assign a unique group index to each block e.g., [0, 0, 1, 2]
    row_ids = block_ids.unsqueeze(2)  # [b,t,1]
    col_ids = block_ids.unsqueeze(1)  # [b,1,t]
    out = row_ids == col_ids # Compare row and col IDs — same block → True
    return out # [b,t,t]



In [ ]:
# @title blockwise_mean
import torch

def blockwise_mean(x, mask): # [b,t]
    """
    Given a 1D tensor x and a boolean mask, compute block-wise means,
    where each True in mask starts a new block.

    Example:
    x = [1,2,3,4], mask = [T,F,T,T] → [ (1+2)/2, 3, 4 ]
    """
    batch = x.shape[0]
    # Assign a block ID to each element
    block_ids = mask.cumsum(1) - 1  # e.g., [0,0,1,2]

    # Get number of blocks
    # num_blocks = int(block_ids[-1]) + 1
    num_blocks = block_ids[:,-1] + 1
    print(block_ids)
    # Sum values and count per block
    # sums = torch.zeros(num_blocks, dtype=x.dtype, device=x.device).index_add(0, block_ids, x)
    # counts = torch.zeros(num_blocks, dtype=x.dtype, device=x.device).index_add(0, block_ids, torch.ones_like(x))
    sums = torch.zeros(batch, num_blocks, dtype=x.dtype, device=x.device).index_add(0, block_ids, x)
    counts = torch.zeros(num_blocks, dtype=x.dtype, device=x.device).index_add(0, block_ids, torch.ones_like(x))
    return sums / counts


import torch

def blockwise_mean(x: torch.Tensor, mask: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
    """
    x:     (B, N) - input values
    mask:  (B, N) - boolean mask (True starts new block)

    Returns:
    - mean_values:  (total_blocks,) — concatenated mean values across all blocks
    - block_lens:   (B,) — number of blocks per sequence (can be used to split result)
    """
    B, N = x.shape

    # Compute block IDs for each sequence
    block_ids = mask.cumsum(dim=1) - 1  # (B, N)
    block_lens = mask.sum(dim=1)        # (B,)
    total_blocks = block_lens.sum()     # scalar

    # Offset block IDs to make them unique across the batch
    block_offsets = torch.cumsum(torch.cat([torch.zeros(1, device=x.device, dtype=block_lens.dtype), block_lens[:-1]]), dim=0)
    block_ids += block_offsets.unsqueeze(1)  # shape: (B, N)

    # Flatten
    flat_ids = block_ids.reshape(-1)           # (B*N,)
    flat_x = x.reshape(-1)                     # (B*N,)
    ones = torch.ones_like(flat_x)

    # Compute sums and counts using index_add
    mean_sums = torch.zeros(total_blocks, device=x.device, dtype=x.dtype).index_add(0, flat_ids, flat_x)
    mean_counts = torch.zeros(total_blocks, device=x.device, dtype=x.dtype).index_add(0, flat_ids, ones)

    # Return mean and optional info to reshape if needed
    return mean_sums / mean_counts, block_lens


# x = torch.tensor([1.0, 2.0, 3.0, 4.0])
x = torch.rand(2,4)
print(x)
mask = torch.tensor([1, 0, 1, 1], dtype=torch.bool).repeat(2,1)
print(blockwise_mean(x, mask))  # Output: tensor([1.5, 3.0, 4.0])



tensor([[0.4062, 0.2538, 0.5809, 0.0500],
        [0.7076, 0.0494, 0.7538, 0.4290]])
(tensor([0.3300, 0.5809, 0.0500, 0.3785, 0.7538, 0.4290]), tensor([3, 3]))


In [ ]:
# @title facebookresearch bytelatent/model/blt.py
# https://github.com/facebookresearch/blt/blob/main/bytelatent/model/blt.py
from enum import Enum, auto
from typing import Any, Optional

import torch
from pydantic import ConfigDict, model_validator
from torch import nn
from torch.nn.attention.flex_attention import create_block_mask
from typing_extensions import Self

from bytelatent.base_transformer import (BaseTransformerArgs, InitStdFactor, SequenceModelWithOutput, TransformerBlock)
from bytelatent.data.patcher import Patcher, PatcherArgs
from bytelatent.model.latent_transformer import GlobalTransformer
from bytelatent.model.local_models import LocalDecoder, LocalEncoder, LocalModelArgs
from bytelatent.model.utils import downsample
from bytelatent.tokenizers.constants import BOE_ID, BOS_ID, EOS_ID, OFFSET, PAD_ID

def causal_mask(b, h, q_idx, kv_idx):
    return q_idx >= kv_idx

def setattrs(_self, **kwargs):
    for k, v in kwargs.items():
        setattr(_self, k, v)

def get_encoder_dim_token_emb(args):
    if args.dim_token is not None:
        dim_token_emb = args.dim_token
    elif args.use_local_encoder_transformer:
        dim_token_emb = args.dim_local_encoder
    else:
        dim_token_emb = args.dim_global // args.patch_size
    return dim_token_emb


def get_encoder_dim_patch_emb(args):
    dim_patch_emb = None
    if args.cross_attn_encoder:
        if args.cross_attn_init_by_pooling:
            dim_patch_emb = args.dim_local_encoder
        else:
            dim_patch_emb = args.dim_global
    return dim_patch_emb


def parse_ngram_to_size(ngram_to_size_str: str | None) -> dict[int, int]:
    if ngram_to_size_str is None:
        return None
    ngram_to_size = {}
    for entry in ngram_to_size_str.split(","):
        ngram, size = entry.split(":")
        ngram = int(ngram)
        size = int(size)
        ngram_to_size[ngram] = size
    return ngram_to_size


def fill_tokens(tokens, patch_size, fill_id):
    batch_size, seq_len = tokens.shape
    if seq_len % patch_size == 0:
        return tokens
    else:
        remaining = patch_size - seq_len % patch_size
        final_padding = tokens.new(batch_size, remaining).fill_(fill_id)
        return torch.cat((tokens, final_padding), dim=1)

def decoder_patch_ids_from_lengths(patch_lengths, nb_boe, seq_len):
    first_patch_length = patch_lengths[0, 0]
    assert torch.all(first_patch_length == patch_lengths[:, 0]), "first patch should always be the same size (1 for dynamic, patch_size for static)."
    assert (first_patch_length - nb_boe == 1), f"First patch (patch length: {first_patch_length}) should have one non-boe token (boe toks: {nb_boe})"
    # Remove first patch from patch_ids for local decoder inputs and shift the last patch.
    # decoder_patch_lengths = patch_lengths[:, 1:].clone()
    # decoder_patch_lengths = add_to_last_nonzero_patch(decoder_patch_lengths, 1)
    decoder_patch_lengths = patch_lengths[:, 1:]
    assert (decoder_patch_lengths.sum() + (nb_boe + 1) * patch_lengths.shape[0] == patch_lengths.sum()), f"{decoder_patch_lengths.sum() + (nb_boe + 1) * patch_lengths.shape[0]} != {patch_lengths.sum()}"
    assert torch.all(decoder_patch_lengths >= 0), f"{decoder_patch_lengths}"
    decoder_patch_ids = patch_ids_from_lengths(patch_lengths=decoder_patch_lengths, seq_len=seq_len)
    return decoder_patch_ids

def create_patch_mask_from_ids(patch_ids, num_patches, window=None, patches_as_queries=False):
    """
    Creates a tensor of shape [bs, seq_len, num_patches] where each element at position (i, j, k)
    is True if the patch id at position (i, j) is less than or equal to k.
    Args:
        patch_ids (torch.Tensor): Tensor of shape [bs, seq_len] containing patch ids.
        num_patches (int): Total number of patches.
        window (int): If not None, only considers patches within a window of size window.
        patches_as_queries (bool): If True, the patches are used as queries
    Returns:
        torch.Tensor: Tensor of shape [bs, q_len, kv_len] with the desired mask.
    """
    bs, seq_len = patch_ids.shape
    if not patches_as_queries:
        q_ids = patch_ids.unsqueeze(-1).expand(bs, seq_len, num_patches)
        kv_ids = (torch.arange(num_patches, device=patch_ids.device)[None,None,...].expand(bs, seq_len, num_patches))
    else:
        kv_ids = patch_ids.unsqueeze(1).expand(bs, num_patches, seq_len)
        q_ids = (torch.arange(num_patches, device=patch_ids.device)[None,...,None].expand(bs, num_patches, seq_len))
    if window is None:
        mask = q_ids == kv_ids
    else:
        mask = (kv_ids <= q_ids) & (q_ids < kv_ids + window)
    return mask


def cross_attn_mask(patch_ids, patch_lengths, N, patches_as_queries=False, cross_attn_k=1, window=None, block_mask=True):
    bs = patch_ids.shape[0]
    with torch.no_grad():
        # Create the patch mask
        cross_mask = create_patch_mask_from_ids(patch_ids, patch_lengths.shape[1], window=window, patches_as_queries=patches_as_queries).repeat_interleave(cross_attn_k, dim=1 if patches_as_queries else -1)
        q_len = patch_lengths.shape[1] * cross_attn_k if patches_as_queries else N
        kv_len = N if patches_as_queries else patch_lengths.shape[1] * cross_attn_k
        assert cross_mask.shape == (bs, q_len, kv_len), f"{cross_mask.shape} != {(bs, q_len, kv_len)}"
        if block_mask:
            def patch_mask(b, h, q_idx, kv_idx):
                return cross_mask[b, q_idx, kv_idx]
            block_mask = create_block_mask(patch_mask, B=bs, H=None, Q_LEN=q_len, KV_LEN=kv_len, _compile=True,)
            return block_mask
        else:
            return torch.where(cross_mask, torch.tensor(0.0), torch.tensor(float("-inf"))).unsqueeze(1)  # [bs, 1, q_len, kv_len]


def get_blt_input(tokens: torch.Tensor, enforce_patch_size_multiple: bool, nb_boe: torch.Tensor, patch_size: int, boe_id: int):
    """
        This function returns X_et, X_gt and X_dt, the encoder, global, and decoder
    tokens respectively.

    Consider the input and target sequences:
    X=[3,4,5,6,7,eos,bos,8,9,10,eos,bos,11,12,13]
    Y=[4,5,6,7,eos,bos,8,9,10,eos,bos,11,12,13,14]
    with patch_size=4

    Note 1: that there will be no special tokens introduced at the patch level.
    Note 2: X_e needs to be trimmed to be passed to Global

    Current without boe:
    X_et = [[boe,boe,boe,boe] [3,4,5,6],      [7,eos,bos,8],    [9,10,eos,bos] [11,12,13, pad]]
    X_g =  [[boe,boe,boe,boe] [3,4,5,6],      [7,eos,bos,8],    [9,10,eos,bos] [11,12,13, pad]] # remove last glob patch
    X_dt = [[3,4,5,6]         [7,eos,bos,8],  [9,10,eos,bos],   [11,12,13]]
    Y =    [[4,5,6,7]         [eos,bos,8,9],  [10,eos,bos,11],  [12,13,14]]

    Note 1: that there will be no special tokens introduced at the patch level.
    Note 2: X_e needs to be trimmed to be passed to Global
    """
    batch_size, seq_len = tokens.shape
    local_encoder_tokens = tokens
    local_decoder_tokens = tokens

    if nb_boe > 0:
        padded_patch = tokens.new(batch_size, nb_boe).fill_(boe_id)
        local_encoder_tokens = torch.cat((padded_patch, local_encoder_tokens), dim=1)
    # global_tokens = tokens.new(batch_size, ((seq_len-1) // patch_size)+1).fill_(boe_id)

    # create global tokens, contains boe tokens and eos
    # padded_local_encoder_tokens = fill_tokens(local_encoder_tokens, patch_size, boe_id)
    # patches = padded_local_encoder_tokens.view(batch_size, -1, patch_size)
    # global_tokens = (patches.eq(eos_id).any(dim=2).int() * eos_id)[:, 1:]
    # global_tokens += global_tokens.eq(0).int() * boe_id
    # TODO: fix this when we want to use block causal in the global.
    if enforce_patch_size_multiple and local_encoder_tokens.shape[-1] % patch_size != 0:
        local_encoder_tokens = fill_tokens(local_encoder_tokens, patch_size, boe_id)
    return local_encoder_tokens, None, local_decoder_tokens


def patch_ids_from_lengths(patch_lengths, seq_len):
    bs, num_patches = patch_lengths.shape
    # Create a tensor of cumulative sums of the patch lengths
    cum_d = torch.cat([torch.zeros(bs, 1, dtype=patch_lengths.dtype, device=patch_lengths.device), patch_lengths.cumsum(dim=-1)], dim=-1)
    patch_ids = (cum_d.unsqueeze(-1) <= torch.arange(seq_len, device=cum_d.device)).sum(dim=-2) - 1
    assert not (torch.max(patch_ids) > patch_lengths.shape[-1] or torch.min(patch_ids) < 0), f"{torch.max(patch_ids)} > {patch_lengths.shape[-1]} or {torch.min(patch_ids)} < 0"
    return patch_ids


class ByteLatentTransformer(nn.Module, SequenceModelWithOutput):
    """
    The ByteLatentTransformer (BLT) is a byte-level language model architecture that processes byte sequences
    by dynamically segmenting them into patches. It uses a combination of local encoders, global transformers,
    and local decoders to efficiently encode and decode byte sequences, leveraging patch-based processing for
    improved performance and inference efficiency.
    """
    def __init__(self, args: ByteLatentTransformerArgs):
        super().__init__()

        # General configuration
        self.weight_tying = args.weight_tying
        self.patch_size = args.patch_size
        self.patching_mode = args.patching_mode

        self.downsampling_by_pooling = args.downsampling_by_pooling
        self.patching_threshold = args.patching_threshold
        self.dim = args.dim
        self.init_base_std = args.init_base_std
        self.init_std_factor = InitStdFactor(args.init_std_factor)
        self.max_seqlen = args.max_seqlen

        # Cross attention configuration
        self.cross_attn_encoder = args.cross_attn_encoder
        self.cross_attn_decoder = args.cross_attn_decoder
        self.cross_attn_k = args.cross_attn_k
        self.cross_attn_window_encoder = args.cross_attn_window_encoder
        self.cross_attn_window_decoder = args.cross_attn_window_decoder
        self.cross_attn_use_flex_attention = args.cross_attn_use_flex_attention

        # Encoder hash configuration
        self.encoder_hash_byte_group_size = args.encoder_hash_byte_group_size
        self.encoder_hash_byte_group_vocab = args.encoder_hash_byte_group_vocab
        self.encoder_hash_byte_group_nb_functions = (
            args.encoder_hash_byte_group_nb_functions
        )

        # ByteLatent modules
        self.local_encoder = create_local_encoder(args)
        self.global_transformer = create_global_transformer(args)
        self.local_decoder = create_local_decoder(args)

        # Patcher module
        if args.patch_in_forward:
            self.patcher = Patcher()

    def get_output_seq_len(self):
        return self.max_seqlen

    def forward(self, tokens: torch.Tensor,
        patch_lengths: Optional[torch.Tensor] = None,
        ngram_ids: Optional[torch.Tensor] = None):
        # Ensure ngram_ids is either a tensor or None
        assert (isinstance(ngram_ids, torch.Tensor) or ngram_ids is None), f"ngram_ids must be a tensor or None, but was: {type(ngram_ids)}"

        bs, N = tokens.shape  # Batch size and sequence length

        # Get megabyte inputs
        nb_boe = int(0 if self.patching_mode != "" else self.patch_size - 1)
        local_encoder_tokens, _, local_decoder_tokens = get_blt_input(tokens=tokens, enforce_patch_size_multiple=False, nb_boe=nb_boe, patch_size=self.patch_size, boe_id=self.boe_id)

        # Patching
        if patch_lengths is None:
            assert (getattr(self, "patcher", None) is not None), "Patcher not defined and no patch_lengths passed."
            patch_lengths, tok_scores = self.patcher.patch(local_encoder_tokens, include_next_token=True, threshold=self.patcher.threshold)
        else:
            if nb_boe > 0:
                patch_lengths[:, 0] += nb_boe

        assert torch.min(patch_lengths) >= 0

        # Generate patch IDs from patch_lengths
        patch_ids = patch_ids_from_lengths(patch_lengths, local_encoder_tokens.shape[-1])
        assert torch.max(patch_ids) + 1 <= torch.max((patch_lengths != 0).sum(dim=-1)), f"{torch.max(patch_ids) + 1} > {torch.max((patch_lengths != 0).sum(dim=-1))}"

        cross_attn_mask_enc = None
        # Cross-attention encoder
        if self.cross_attn_encoder:
            cross_attn_mask_enc = cross_attn_mask(patch_ids, patch_lengths, N, patches_as_queries=True,
                cross_attn_k=self.cross_attn_k, window=self.cross_attn_window_encoder, block_mask=self.cross_attn_use_flex_attention)

        # Hashing and embedding
        local_encoder_embeds = compute_hash_embeddings(local_encoder_tokens=local_encoder_tokens, local_encoder=self.local_encoder, encoder_hash_tok_embedding=self.encoder_hash_tok_embedding,
            encoder_hash_byte_group_nb_functions=self.encoder_hash_byte_group_nb_functions, encoder_hash_byte_group_size=self.encoder_hash_byte_group_size, encoder_hash_byte_group_vocab=self.encoder_hash_byte_group_vocab)

        # N-gram table embeddings
        if self.encoder_ngram_embedding is not None:
            assert ngram_ids is not None, "ngram_ids must be provided"
            if local_encoder_embeds is None:
                local_encoder_embeds = self.local_encoder.tok_embeddings(local_encoder_tokens)
            assert len(ngram_ids) == len(self.encoder_ngram_embedding), f"ngram_ids.shape[0]={ngram_ids.shape[0]} versus len(encoder_ngram_embedding)={len(self.encoder_ngram_embedding)}, ngram_ids.shape={ngram_ids.shape}"
            for i in range(ngram_ids.shape[0]):
                ngram_embedding = self.encoder_ngram_embedding[i]
                ngram_embeds = ngram_embedding(ngram_ids[i])
                assert (local_encoder_embeds.shape == ngram_embeds.shape), f"Shape mismatch: {local_encoder_embeds.shape} vs {ngram_embeds.shape}, ngram_ids.shape={ngram_ids.shape}"
                local_encoder_embeds = local_encoder_embeds + ngram_embeds

        # Local encoder
        (h_encoder, h_cross), cache_encoder = self.local_encoder(tokens=local_encoder_tokens, embeds=local_encoder_embeds,
            patch_embeds=None, cross_mask=cross_attn_mask_enc, num_patches=patch_lengths.shape[1], patch_ids=patch_ids)

        # Downsampling
        if not self.cross_attn_encoder:
            assert (patch_ids.shape[1] == h_encoder.shape[1]), f"{patch_ids.shape[1]} != {h_encoder.shape[1]}"
            h = downsample(h_encoder, patch_lengths.shape[1], patch_lengths,
                patch_ids, downsampling_by_pooling=self.downsampling_by_pooling, patch_size=self.patch_size)
        else:
            # Reshape h_cross
            h = h_cross.view(bs, patch_lengths.shape[1], -1)

        # Global transformer
        global_tokens = tokens.new(h.shape[0], h.shape[1]).fill_(self.boe_id)
        rows, cols = torch.where(local_encoder_tokens == self.eos_id)
        eos_patch_ids = patch_ids[rows, cols]
        global_tokens[rows, eos_patch_ids] = self.eos_id

        h, _ = self.global_transformer(embeds=h, tokens=global_tokens)

        # Unpatching
        dec_embeds = h_encoder[:, nb_boe : nb_boe + N, :]

        # Generate decoder patch IDs
        decoder_patch_ids = decoder_patch_ids_from_lengths(patch_lengths, nb_boe, local_decoder_tokens.shape[-1])
        assert (torch.max(decoder_patch_ids) + 1 <= h.shape[1]), f"{torch.max(decoder_patch_ids) + 1} > {h.shape[1]}"
        assert (decoder_patch_ids.shape[1] == dec_embeds.shape[1]), f"{decoder_patch_ids.shape[1]} != {dec_embeds.shape[1]}"

        # Cross-attention decoder
        if not self.cross_attn_decoder:
            h = torch.gather(h, 1, decoder_patch_ids.unsqueeze(-1).expand(-1, -1, h.shape[-1]))
            cross_attn_mask_dec = None
            assert local_decoder_tokens.shape == h.shape[:-1]
        else:
            cross_attn_mask_dec = cross_attn_mask(decoder_patch_ids, patch_lengths, N, patches_as_queries=False, cross_attn_k=self.cross_attn_k, window=self.cross_attn_window_decoder, block_mask=self.cross_attn_use_flex_attention)

        # Local decoder
        output, _ = self.local_decoder(embeds=dec_embeds, patch_embeds=h, tokens=local_decoder_tokens, cross_mask=cross_attn_mask_dec)
        return output


local_encoder_tokens, _, local_decoder_tokens = get_blt_input(tokens=tokens, enforce_patch_size_multiple=False, nb_boe=nb_boe, patch_size=self.patch_size, boe_id=self.boe_id)
patch_lengths, tok_scores = self.patcher.patch(local_encoder_tokens, include_next_token=True, threshold=self.patcher.threshold)
patch_ids = patch_ids_from_lengths(patch_lengths, local_encoder_tokens.shape[-1])
cross_attn_mask_enc = cross_attn_mask(patch_ids, patch_lengths, N, patches_as_queries=True,
local_encoder_embeds = local_encoder_embeds + ngram_embeds
(h_encoder, h_cross), cache_encoder = self.local_encoder(tokens=local_encoder_tokens, embeds=local_encoder_embeds,



## other blt

In [ ]:
# @title anine09 ByteLatentTransformer.py
# https://github.com/anine09/EyesNet/blob/master/eyesnet/reranker/xrd_encoder/ByteLatentTransformer.py

import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class LayerNorm(nn.Module):
    """Custom Layer Normalization."""

    def __init__(self, ndim, bias=True):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, x):
        return F.layer_norm(x, self.weight.shape, self.weight, self.bias, 1e-5)


class Block(nn.Module):
    """Transformer Block with LayerNorm, Attention, and MLP."""

    def __init__(self, n_embd, n_head, dropout):
        super().__init__()
        self.ln1 = LayerNorm(n_embd)
        self.attn = nn.MultiheadAttention(
            embed_dim=n_embd, num_heads=n_head, dropout=dropout, batch_first=True
        )
        self.ln2 = LayerNorm(n_embd)
        self.mlp = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        attn_out, _ = self.attn(self.ln1(x), self.ln1(x), self.ln1(x))
        x = x + attn_out
        x = x + self.mlp(self.ln2(x))
        return x


class CrossAttentionLayer(nn.Module):
    """Cross Attention Layer for Encoder and Decoder."""

    def __init__(self, query_dim, key_dim, n_head, dropout):
        super().__init__()
        self.ln_q = LayerNorm(query_dim)
        self.ln_kv = LayerNorm(key_dim)
        self.attn = nn.MultiheadAttention(
            embed_dim=query_dim, num_heads=n_head, dropout=dropout, batch_first=True
        )
        self.proj = nn.Linear(query_dim, query_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value):
        query = self.ln_q(query)
        key = self.ln_kv(key)
        value = self.ln_kv(value)
        attn_out, _ = self.attn(query, key, value)
        attn_out = self.proj(attn_out)
        attn_out = self.dropout(attn_out)
        return query + attn_out


class HashNGramEmbedding(nn.Module):
    """Hash n-gram Embeddings."""
    def __init__(self, hash_sizes, hash_table_size, n_embd):
        super().__init__()
        self.hash_sizes = hash_sizes
        self.hash_table_size = hash_table_size
        self.n_embd = n_embd
        self.hash_embeddings = nn.ModuleDict({f"hash_{n}": nn.Embedding(hash_table_size, n_embd) for n in hash_sizes})

    def forward(self, x):
        B, T = x.shape
        embeddings = torch.zeros(B, T, self.n_embd, device=x.device)
        for n in self.hash_sizes:
            if T < n:
                continue
            # Extract n-grams
            ngrams = x.unfold(1, n, 1)  # [B, T - n +1, n]
            # Compute hash
            hashes = self.roll_poly_hash(ngrams)
            hashes = hashes % self.hash_table_size
            # Lookup embeddings
            hash_emb = self.hash_embeddings[f"hash_{n}"](
                hashes
            )  # [B, T - n +1, n_embd]
            # Scatter add
            embeddings[:, n - 1 : T, :] += hash_emb
        # Normalize
        embeddings = embeddings / len(self.hash_sizes)
        return embeddings  # [B, T, n_embd]

    def roll_poly_hash(self, ngrams):
        """Simple polynomial rolling hash."""
        base = 257
        hash_val = torch.zeros(
            ngrams.size(0), ngrams.size(1), device=ngrams.device, dtype=torch.long
        )
        for i in range(ngrams.size(2)):
            hash_val = (hash_val * base + ngrams[:, :, i].long()) % (2**32)
        return hash_val


class LocalEncoder(nn.Module):
    """Local Encoder that encodes input bytes into patch representations."""

    def __init__(
        self,
        vocab_size,
        n_embd,
        patch_size,
        hash_sizes,
        hash_table_size,
        n_head,
        dropout,
        lE,
    ):
        super().__init__()
        self.patch_size = patch_size
        self.n_embd = n_embd
        self.byte_embedding = nn.Embedding(vocab_size, n_embd)
        self.hash_ngram = HashNGramEmbedding(hash_sizes, hash_table_size, n_embd)
        self.transformer_blocks = nn.ModuleList(
            [Block(n_embd, n_head, dropout) for _ in range(lE)]
        )
        self.cross_attn = CrossAttentionLayer(n_embd, n_embd, n_head, dropout)
        self.ln = LayerNorm(n_embd)

    def forward(self, x):
        B, T = x.shape
        # Byte Embedding
        x_emb = self.byte_embedding(x)  # [B, T, C]
        # Hash n-gram Embedding
        hash_emb = self.hash_ngram(x)  # [B, T, C]
        x_emb = x_emb + hash_emb  # [B, T, C]
        # Transformer Layers
        for block in self.transformer_blocks:
            x_emb = block(x_emb)
        # Cross-Attention to form patches
        # Assume patches are non-overlapping
        # Pad if necessary
        if T % self.patch_size != 0:
            pad_len = self.patch_size - (T % self.patch_size)
            pad = torch.zeros((B, pad_len), dtype=x.dtype, device=x.device).long()
            pad_emb = self.byte_embedding(pad)  # [B, pad_len, C]
            pad_emb += self.hash_ngram(pad)  # Incorporate hash embeddings
            x_emb = torch.cat([x_emb, pad_emb], dim=1)  # [B, T + pad_len, C]
            T += pad_len
        # Reshape and pool to create patch representations
        patches = x_emb.view(
            B, T // self.patch_size, self.patch_size, self.n_embd
        ).mean(
            dim=2
        )  # [B, N_patches, C]
        patches = self.cross_attn(patches, x_emb, x_emb)  # [B, N_patches, C]
        patches = self.ln(patches)
        return patches  # [B, N_patches, C]


class LatentTransformer(nn.Module):
    """Latent Transformer over patch representations."""

    def __init__(self, n_embd, n_head, n_layers, dropout):
        super().__init__()
        self.blocks = nn.ModuleList(
            [Block(n_embd, n_head, dropout) for _ in range(n_layers)]
        )
        self.ln_f = LayerNorm(n_embd)

    def forward(self, x):
        for block in self.blocks:
            x = block(x)
        return self.ln_f(x)


class ByteLatentTransformer(nn.Module):
    """Byte Latent Transformer combining encoder, transformer, and decoder."""
    def __init__(
        self,
        vocab_size,
        n_embd,
        n_head,
        n_layers_encoder,
        n_layers_latent,
        dropout,
        patch_size,
        hash_sizes,
        hash_table_size,
        block_size,
    ):
        super().__init__()
        self.local_encoder = LocalEncoder(
            vocab_size,
            n_embd,
            patch_size,
            hash_sizes,
            hash_table_size,
            n_head,
            dropout,
            n_layers_encoder,
        )
        self.latent_transformer = LatentTransformer(
            n_embd, n_head, n_layers_latent, dropout
        )
        self.pos_embedding = nn.Parameter(
            torch.zeros(1, math.ceil(block_size / patch_size), n_embd)
        )
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, x):
        # Encode bytes to patches
        patches = self.local_encoder(x)  # [B, N_patches, C]
        # Add positional embeddings
        patches = (patches + self.pos_embedding[:, : patches.size(1), :]) # [B, N_patches, C]
        # Transform patches
        transformed_patches = self.latent_transformer(patches)  # [B, N_patches, C]
        return transformed_patches

ByteLatentTransformer(vocab_size,
        n_embd,
        n_head,
        n_layers_encoder,
        n_layers_latent,
        dropout,
        patch_size,
        hash_sizes,
        hash_table_size,
        block_size,
    )



In [ ]:
# @title Kitsunp/Prueba-de-modelo-de-ByteLatentTransformer blt_model.py
# https://github.com/Kitsunp/Prueba-de-modelo-de-ByteLatentTransformer/blob/main/blt_model.py

import torch
torch.backends.cuda.enable_flash_sdp(enabled=True)

import torch.nn as nn
import torch.nn.functional as F
import math
from dataclasses import dataclass
from typing import Optional, Dict, List


class EncoderLayer(nn.Module):
    """
    Capa de encoder que combina self-attention, cross-attention (opcional)
    y feed-forward.
    """
    def __init__(self, config):
        super().__init__()
        self.self_attn = MultiHeadAttention(config)
        self.cross_attn = CrossAttention(config)
        self.feed_forward = FeedForward(config)
        self.dropout = nn.Dropout(config.resid_dropout)

    def forward(self, x, cross_context=None, self_mask=None, cross_mask=None, positions=None):
        # print("\n[EncoderLayer] - Input shape:", x.shape)
        h = x + self.self_attn(x, mask=self_mask, positions=positions, is_causal=False)
        # print("[EncoderLayer] - After Self-Attn shape:", h.shape)
        h = self.dropout(h)

        if cross_context is not None:
            h = h + self.cross_attn(h, cross_context, cross_mask)
            # print("[EncoderLayer] - After Cross-Attn shape:", h.shape)
            h = self.dropout(h)

        out = h + self.feed_forward(h)
        # print("[EncoderLayer] - After FeedForward shape:", out.shape)
        out = self.dropout(out)
        return out

class DecoderLayer(nn.Module):
    """
    Capa de decoder con cross-attention, self-attention con enmascaramiento causal
    y feed-forward.
    """
    def __init__(self, config):
        super().__init__()
        self.cross_attn = CrossAttention(config)
        self.self_attn = MultiHeadAttention(config)
        self.feed_forward = FeedForward(config)
        self.dropout = nn.Dropout(config.resid_dropout)

    def forward(self, x, encoder_output, self_mask=None, cross_mask=None, positions=None):
        # print("\n[DecoderLayer] - Input shape:", x.shape)
        h = x + self.cross_attn(x, encoder_output, cross_mask)
        # print("[DecoderLayer] - After Cross-Attn shape:", h.shape)
        h = self.dropout(h)

        h = h + self.self_attn(h, self_mask, positions, is_causal=True)
        # print("[DecoderLayer] - After Self-Attn shape:", h.shape)
        h = self.dropout(h)

        out = h + self.feed_forward(h)
        # print("[DecoderLayer] - After FeedForward shape:", out.shape)
        out = self.dropout(out)
        return out


# =============================================================================
#                          EMBEDDINGS A NIVEL DE BYTE
# =============================================================================
import torch
import torch.nn as nn

class ByteEmbedding(nn.Module):
    def __init__(self, config):
        """
        Inicializa la capa ByteEmbedding.

        Args:
            config (Namespace): Objeto de configuración que debe contener:
                - hidden_size (int): Dimensión de los embeddings
                - ngram_vocab_size (int): Tamaño del vocabulario para cada n-grama
                - resid_dropout (float): Tasa de dropout para embeddings y gating
                - noise_std (float): Desviación estándar del ruido Gaussiano
        """
        super().__init__()

        # Embedding para bytes individuales (256 posibles valores)
        self.byte_embeddings = nn.Embedding(256, config.hidden_size)

        # Lista de embeddings para n-gramas de tamaño 3 a 8
        self.ngram_hash_embeddings = nn.ModuleList([nn.Embedding(config.ngram_vocab_size, config.hidden_size) for _ in range(6)])

        # Proyecciones lineales para después de la concatenación
        self.projections = nn.ModuleList([nn.Linear(config.hidden_size * 2, config.hidden_size) for _ in range(6)])

        # Normalización por capa aplicada al final
        self.layer_norm = nn.LayerNorm(config.hidden_size)

        self.noise_scale = nn.Parameter(torch.tensor(0.09))

        # Parámetros aprendibles de gating para cada tamaño de n-grama (3 a 8)
        self.ngram_gates = nn.Parameter(torch.ones(6))

        # Gates residuales aprendibles
        self.residual_gates = nn.ModuleList([
            nn.Sequential(
                nn.Linear(config.hidden_size * 2, config.hidden_size),
                nn.LayerNorm(config.hidden_size), nn.GELU(), nn.Linear(config.hidden_size, 1),
                nn.Sigmoid()
            ) for _ in range(6)])

        # Dropouts
        self.dropout = nn.Dropout(config.resid_dropout)  # Dropout en embeddings base
        self.gate_dropout = nn.Dropout(config.resid_dropout)  # Dropout en gates estocásticos
        self.residual_dropout = nn.Dropout(config.resid_dropout)  # Dropout en embeddings expandidos
        self.residual_gate_dropout = nn.Dropout(config.resid_dropout)  # Dropout en gates residuales
        self.projection_dropout = nn.Dropout(config.resid_dropout)  # Dropout post-proyección

    def compute_ngram_hash(self, bytes_sequence, n):
        """
        Calcula índices de hash para cada n-grama en la secuencia.

        Implementa un esquema de hashing que:
        1. Extrae n-gramas consecutivos
        2. Calcula hashes usando pesos exponenciales
        3. Aplica factores de escala adaptativos
        4. Utiliza offsets primos para mejor distribución

        Args:
            bytes_sequence (torch.Tensor): Tensor de bytes [batch_size, seq_length]
            n (int): Tamaño del n-grama (3-8)

        Returns:
            torch.Tensor: Índices de hash [batch_size, seq_length - n + 1]
        """
        device = bytes_sequence.device
        batch_size, seq_length = bytes_sequence.shape

        if seq_length < n:
            return torch.empty((batch_size, 0), dtype=torch.long, device=device)

        # Extraer n-gramas: [B, (seq_length - n + 1), n]
        ngrams = bytes_sequence.unfold(dimension=1, size=n, step=1)

        # Factores adaptativos basados en n
        scale_factor = 1.0 - (n - 3) * 0.1  # Decrece con n
        offset = ((n - 3) * 37) % 256  # Offset primo

        # Pesos exponenciales para el hash
        exponents = torch.arange(n, device=device).float()
        weights = (256 ** exponents).unsqueeze(0).unsqueeze(0)  # [1,1,n]

        # Cálculo del hash
        hash_values = (ngrams.float() * weights).sum(dim=-1)  # [B, seq_length-n+1]
        hash_values = hash_values * scale_factor + offset
        hash_values = hash_values.long()

        # Ajustar al tamaño del vocabulario
        vocab_size = self.ngram_hash_embeddings[n-3].num_embeddings
        return hash_values % vocab_size

    def forward(self, bytes_input):
        """
        Procesa la secuencia de bytes para generar embeddings enriquecidos.

        El proceso incluye:
        1. Generación de embeddings base
        2. Procesamiento de n-gramas
        3. Aplicación de gates estocásticos y residuales
        4. Concatenación y proyección de embeddings
        5. Normalización final

        Args:
            bytes_input (torch.Tensor): Tensor de bytes [batch_size, seq_length]

        Returns:
            torch.Tensor: Embeddings procesados [batch_size, seq_length, hidden_size]
        """
        device = bytes_input.device
        batch_size, seq_length = bytes_input.shape

        # Embeddings base
        embeds = self.byte_embeddings(bytes_input).float()
        embeds = self.dropout(embeds)

        # Escala de ruido adaptativa
        current_noise_scale = torch.sigmoid(self.noise_scale)

        # Procesamiento de n-gramas
        for i, n in enumerate(range(3, 9)):
            if seq_length < n:
                continue

            # Generar embeddings de n-gramas
            ngram_hashes = self.compute_ngram_hash(bytes_input, n)
            ngram_embeds = self.ngram_hash_embeddings[i](ngram_hashes)

            # Gate estocástico con ruido
            noise = torch.randn_like(self.ngram_gates[i]) * current_noise_scale
            alpha_n = self.ngram_gates[i] + noise
            alpha_n = self.gate_dropout(alpha_n)
            alpha_n = torch.relu(alpha_n)

            # Aplicar gate y normalización
            gated_embeds = (ngram_embeds / n) * alpha_n

            # Gate residual
            gate_input = torch.cat([embeds[:, :seq_length - n + 1], gated_embeds], dim=-1)
            gate_input = self.residual_gate_dropout(gate_input)
            residual_gate = self.residual_gates[i](gate_input)
            residual_gate = self.residual_gate_dropout(residual_gate)

            # Concatenar y proyectar
            concatenated = torch.cat([
                embeds[:, :seq_length - n + 1],
                gated_embeds * residual_gate
            ], dim=-1)

            projected = self.projections[i](concatenated)
            embeds[:, :seq_length - n + 1, :] = projected
        # Normalización final
        embeds = self.layer_norm(embeds)

        return embeds

# =============================================================================
#                        MODELOS DE ENCODER Y DECODER
# =============================================================================

class LocalEncoder(nn.Module):
    """
    Encoder local que procesa los bytes de forma detallada.
    """
    def __init__(self, config):
        super().__init__()
        self.byte_embeddings = ByteEmbedding(config)
        self.embedding_dropout = nn.Dropout(config.resid_dropout)
        self.layers = nn.ModuleList([EncoderLayer(config) for _ in range(config.encoder_layers)])
        self.dropout = nn.Dropout(config.resid_dropout)

    def forward(self, bytes_input, patch_boundaries=None):
        # print("\n[LocalEncoder] - Input shape:", bytes_input.shape)
        h = self.byte_embeddings(bytes_input)
        # print("[LocalEncoder] - After ByteEmbedding shape:", h.shape)
        h = self.embedding_dropout(h)

        positions = torch.arange(bytes_input.size(1), device=bytes_input.device)

        for idx, layer in enumerate(self.layers):
            # print(f"[LocalEncoder] - Passing through EncoderLayer {idx}")
            h = layer(h, positions=positions)
            # print(f"[LocalEncoder] - EncoderLayer {idx} output shape:", h.shape)
            h = self.dropout(h)
        # print("[LocalEncoder] - Final output shape:", h.shape)
        return h

class GlobalTransformer(nn.Module):
    """
    Procesa la información a nivel de parches con atención global.
    Optimizado internamente para reducir redundancias y uso de VRAM,
    SIN añadir nuevas caches ni eliminar la lógica principal.
    Conserva la misma interfaz y atributos para compatibilidad.
    """
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.expansion_rate = getattr(config, 'expansion_rate', 2)

        # ------------------------------------------------------------
        #  Submódulos principales: (se mantienen igual para compatibilidad)
        # ------------------------------------------------------------
        self.layers = nn.ModuleList([
            EncoderLayer(config) for _ in range(config.global_layers)
        ])

        # Sistemas de Dropout
        self.dropout = nn.Dropout(config.resid_dropout)
        self.adaptive_dropout = nn.Dropout(0.0855)
        self.gate_dropout = nn.Dropout(0.0855)
        self.mem_dropout = nn.Dropout(0.0855)
        self.skip_dropout = nn.Dropout(0.0855)

        # Normalizaciones
        self.layer_norms = nn.ModuleList([
            RMSNorm(config.hidden_size, eps=1e-6)
            for _ in range(config.global_layers)
        ])
        self.input_norm = RMSNorm(config.hidden_size, eps=1e-6)
        self.output_norm = RMSNorm(config.hidden_size, eps=1e-6)

        # Normalizaciones específicas
        self.pre_width_norm = RMSNorm(config.hidden_size, eps=1e-6)
        self.post_width_norm = (
            RMSNorm(config.n_states, eps=1e-6) if hasattr(config, 'n_states') else None
        )
        self.post_alpha_norm = (
            RMSNorm(config.n_states, eps=1e-6) if hasattr(config, 'n_states') else None
        )
        self.pre_gate_norm = RMSNorm(config.hidden_size, eps=1e-6)
        self.post_laurel_norm = RMSNorm(config.hidden_size, eps=1e-6)
        self.pre_memory_norm = RMSNorm(config.hidden_size, eps=1e-6)
        self.post_memory_norm = RMSNorm(config.hidden_size, eps=1e-6)
        self.post_combined_norm = RMSNorm(config.hidden_size, eps=1e-6)

        # Skip Gates
        self.skip_gates = nn.ModuleList([
            nn.Sequential(
                nn.Linear(config.hidden_size * 2, config.hidden_size),
                nn.Dropout(0.0855),
                nn.Sigmoid()
            ) for _ in range(config.global_layers)
        ])

        # LAUREL
        self.laurel_alphas = nn.Parameter(torch.ones(config.global_layers))
        self.laurel_g = nn.ModuleList([
            nn.Linear(config.hidden_size, config.hidden_size)
            for _ in range(config.global_layers)
        ])

        # Pesos Adaptativos
        self.adaptive_weights = nn.Parameter(torch.ones(config.global_layers))

        # Hyper-Connections
        self.hyper_static_beta = nn.Parameter(torch.ones(self.expansion_rate))

        init_alpha0 = torch.zeros((config.global_layers, self.expansion_rate, 1))
        for i in range(config.global_layers):
            init_alpha0[i, i % self.expansion_rate, 0] = 1.0

        self.hyper_static_alpha = nn.Parameter(
            torch.cat([
                init_alpha0,
                torch.eye(self.expansion_rate).unsqueeze(0).repeat(config.global_layers, 1, 1)
            ], dim=2)
        )

        hidden_size = config.hidden_size
        self.hyper_dynamic_alpha_fn = nn.Parameter(
            torch.zeros((config.global_layers, hidden_size, self.expansion_rate + 1))
        )
        self.hyper_dynamic_alpha_scale = nn.Parameter(
            torch.ones(config.global_layers) * 0.01
        )
        self.hyper_dynamic_beta_fn = nn.Parameter(
            torch.zeros((config.global_layers, hidden_size))
        )
        self.hyper_dynamic_beta_scale = nn.Parameter(
            torch.ones(config.global_layers) * 0.01
        )

        # Memoria Jerárquica
        self.hierarchical_mem = nn.Parameter(
            torch.zeros(config.global_layers, config.hidden_size) + 1e-6
        )
        self.mem_gate = nn.Sequential(
            nn.Linear(config.hidden_size * 2, config.hidden_size),
            nn.Dropout(0.085),
            nn.Sigmoid()
        )

    # ----------------------------------------------------------------
    # Métodos internos para skip+laurel, hyper-connections y memoria
    # Se reorganizan para optimizar la implementación, sin añadir caches.
    # ----------------------------------------------------------------

    def _apply_skip_and_laurel(self, x, residual, layer_output, layer_idx):
        """
        Mezcla la salida de la capa (layer_output) con la entrada (x, residual)
        mediante gates adaptativos (skip_gates) y el mecanismo LAUREL.
        """
        # Normalizaciones fusionadas
        norm_x = self.layer_norms[layer_idx](x) * 0.1
        norm_r = self.layer_norms[layer_idx](residual) * 0.1

        gx = self.pre_gate_norm(norm_x) * 0.1
        gr = self.pre_gate_norm(norm_r) * 0.1

        # Gate input
        gate_input = torch.cat([gx, gr], dim=-1)  # (B, S, 2D)
        gate_val = self.skip_gates[layer_idx](gate_input)  # Sigmoid
        gate_val = self.gate_dropout(gate_val)

        # Pesos adaptativos
        aw = torch.sigmoid(self.adaptive_weights[layer_idx]) * 0.1
        aw = aw.view(1, 1, 1)
        weighted_r = aw * gate_val * norm_r
        weighted_r = self.skip_dropout(weighted_r)

        # LAUREL
        alpha = torch.sigmoid(self.laurel_alphas[layer_idx]) * 0.1
        alpha = self.adaptive_dropout(alpha)

        g_x = self.laurel_g[layer_idx](x) * 0.1
        g_x = self.skip_dropout(g_x)

        laurel_out = layer_output * alpha + g_x
        laurel_out = self.post_laurel_norm(laurel_out) * 0.1

        combined_output = laurel_out + weighted_r
        return combined_output

    def _apply_hyper_connections(self, x, hyper_h, layer_idx):
        """
        Mezcla la señal x con el tensor hyper_h a través de matrices alpha y beta,
        manteniendo la lógica original y sin añadir cache externo.
        """
        # Normalizar hyper_h
        norm_h = self.layer_norms[layer_idx](hyper_h)
        B, S, N, D = norm_h.shape  # N = n_states (expansion_rate)

        # Flatten + pre_width_norm
        h_flat = norm_h.reshape(B * S * N, D)
        h_flat = self.pre_width_norm(h_flat)

        # dynamic alpha
        alpha_fn = self.hyper_dynamic_alpha_fn[layer_idx][:, :N]  # (D, N)
        wc_weight = torch.matmul(h_flat, alpha_fn)

        if self.post_width_norm is not None:
            wc_weight = self.post_width_norm(wc_weight.reshape(-1, N))
        wc_weight = wc_weight.reshape(B, S, N, N)
        wc_weight = torch.tanh(wc_weight)

        alpha_scale = self.hyper_dynamic_alpha_scale[layer_idx].view(1, 1, 1, 1)
        dynamic_alpha = wc_weight * alpha_scale

        static_alpha = self.hyper_static_alpha[layer_idx][:N, :N]
        static_alpha = static_alpha.view(1, 1, N, N).expand(B, S, -1, -1)
        alpha = dynamic_alpha + static_alpha

        if self.post_alpha_norm is not None:
            alpha_view = alpha.reshape(-1, N)
            alpha_view = self.post_alpha_norm(alpha_view)
            alpha = alpha_view.reshape(B, S, N, N)

        # dynamic beta
        beta_fn = self.hyper_dynamic_beta_fn[layer_idx]  # (D,)
        dc_weight = torch.matmul(norm_h, beta_fn.view(-1, 1)).squeeze(-1)
        dc_weight = torch.tanh(dc_weight)

        beta_scale = self.hyper_dynamic_beta_scale[layer_idx].view(1, 1, 1)
        dynamic_beta = dc_weight * beta_scale

        static_beta = self.hyper_static_beta[:N].view(1, 1, -1)
        beta = dynamic_beta + static_beta

        # Mezcla final
        # alpha: (B,S,N,N), hyper_h: (B,S,N,D)
        mix_h = torch.matmul(alpha, hyper_h)

        x_expanded = x.unsqueeze(2).expand(-1, -1, N, -1)
        depth_conn = x_expanded * beta.unsqueeze(-1)

        return mix_h + depth_conn

    def _apply_hierarchical_memory(self, x, layer_idx, batch_size):
        """
        Integra la memoria jerárquica de la capa layer_idx,
        manteniendo la lógica original sin añadir cachés.
        """
        # Sumar offset y reescalar
        x = (x + 1e-6) * 0.1
        x = self.pre_memory_norm(x) + 1e-6

        # Extraer y expandir la memoria para esta capa
        mem = self.hierarchical_mem[layer_idx:layer_idx+1] + 1e-6
        mem = mem.unsqueeze(0).expand(batch_size, x.size(1), -1)
        mem = self.mem_dropout(mem)

        # Calcular gate
        mem_input = torch.cat([x, mem], dim=-1)
        mem_gate_val = self.mem_gate(mem_input) * 0.1 + 1e-6

        memory_output = mem_gate_val * mem + 1e-6
        memory_output = self.post_memory_norm(memory_output) * 0.1 + 1e-6
        memory_output = self.mem_dropout(memory_output) * 0.1

        result = x + memory_output
        if torch.isnan(result).any():
            result = torch.nan_to_num(result, nan=1e-6)
        return result

    # ----------------------------------------------------------------
    #  Forward principal: igual firma y pasos, sin añadir caches extras
    # ----------------------------------------------------------------

    def forward(self,
                patch_embeddings: torch.Tensor,
                attention_mask: Optional[torch.Tensor] = None
                ):
        """
        Mismo forward y firma, sin cache adicional.
        """
        batch_size = patch_embeddings.size(0)

        # Normalización de entrada + dropout
        h = self.input_norm(patch_embeddings)
        h = self.dropout(h)

        # Expansión "virtual" para hyper-connections
        hyper_h = h.unsqueeze(2).expand(-1, -1, self.expansion_rate, -1)
        positions = torch.arange(patch_embeddings.size(1), device=patch_embeddings.device)

        for idx, layer in enumerate(self.layers):
            # Paso por la capa
            prev_h = self.dropout(h)
            layer_out = layer(h, self_mask=attention_mask, positions=positions)

            # Combinación Skip & Laurel
            combined_out = self._apply_skip_and_laurel(prev_h, prev_h, layer_out, idx)

            # Hyper-connections
            hyper_h = self._apply_hyper_connections(combined_out, hyper_h, idx)

            # Suma con la media de hyper_h
            combined_features = combined_out + hyper_h.mean(dim=2)
            combined_features = self.post_combined_norm(combined_features)

            # Memoria jerárquica
            h = self._apply_hierarchical_memory(combined_features, idx, batch_size)
            h = self.dropout(h)

        # Normalización de salida + dropout
        h = self.output_norm(h)
        h = self.dropout(h)
        return h
class LocalDecoder(nn.Module):
    """
    Decoder local que reconvierte las representaciones latentes en logits de bytes.
    """
    def __init__(self, config):
        super().__init__()
        self.layers = nn.ModuleList([DecoderLayer(config) for _ in range(config.decoder_layers)])
        self.byte_predictor = nn.Linear(config.hidden_size, 256)
        self.dropout = nn.Dropout(config.resid_dropout)

    def forward(self, encoded_bytes, global_output, byte_mask=None, cross_mask=None):
        # print("\n[LocalDecoder] - Input encoded_bytes shape:", encoded_bytes.shape)
        h = encoded_bytes
        positions = torch.arange(encoded_bytes.size(1), device=encoded_bytes.device)

        for idx, layer in enumerate(self.layers):
            # print(f"[LocalDecoder] - Passing through DecoderLayer {idx}")
            h = layer(h, global_output, self_mask=byte_mask, cross_mask=cross_mask, positions=positions)
            # print(f"[LocalDecoder] - DecoderLayer {idx} output shape:", h.shape)
            h = self.dropout(h)
        logits = self.byte_predictor(h)
        # print("[LocalDecoder] - Final logits shape:", logits.shape)
        return logits


# =============================================================================
#                    MODELO DE ENTROPÍA (SIN USO DE CACHÉ)
# =============================================================================
class EntropyLM(nn.Module):
    """
    Modelo de lenguaje basado en entropía dual con análisis local y global.

    Este modelo implementa un sistema de entropía dual que:
    1. Analiza patrones locales en ventanas pequeñas.
    2. Captura contexto global en una dimensión reducida.
    3. Combina ambas medidas de forma adaptativa mediante pesos aprendidos.

    El modelo utiliza skip connections y gates aprendibles para mejorar
    la eficiencia y el flujo de información, sin necesidad de cache adicional.
    """

    # ========================================================
    #                SUBMÓDULOS INTERNOS
    # ========================================================
    class AdaptiveWaveletLayer(nn.Module):
        """
        Capa optimizada de análisis global usando wavelets neuronales adaptativos.
        Implementa procesamiento por lotes vectorizado sin uso de caché.
        """
        def __init__(self, hidden_size, global_size, num_wavelets=8, dropout=0.1, chunk_size=1024):
            super().__init__()
            self.hidden_size = hidden_size
            self.global_size = global_size
            self.num_wavelets = num_wavelets
            self.chunk_size = chunk_size

            # Wavelets optimizados para procesamiento por lotes
            self.mother_wavelets = nn.Parameter(
                torch.randn(1, num_wavelets, hidden_size, 1) * 0.02
            )

            # Escalas con broadcasting optimizado
            self.scales = nn.Parameter(torch.ones(1, num_wavelets, 1, 1))

            # Mixer optimizado con menos parámetros y mejor regularización
            self.coeff_mixer = nn.Sequential(
                nn.Linear(num_wavelets, hidden_size // 2),
                nn.LayerNorm(hidden_size // 2), nn.GELU(), nn.Dropout(dropout), nn.Linear(hidden_size // 2, hidden_size)
            )

            # Proyección con skip connection residual
            self.output_proj = nn.Sequential(
                nn.LayerNorm(hidden_size), nn.Linear(hidden_size, global_size), nn.Dropout(dropout)
            )

            # MultiheadAttention optimizada
            self.num_heads = 4
            self.head_dim = hidden_size // self.num_heads
            self.scale = self.head_dim ** -0.5

            # Proyecciones para Q, K, V
            self.q_proj = nn.Linear(hidden_size, hidden_size, bias=False)
            self.k_proj = nn.Linear(hidden_size, hidden_size, bias=False)
            self.v_proj = nn.Linear(hidden_size, hidden_size, bias=False)
            self.out_proj = nn.Linear(hidden_size, hidden_size, bias=False)

            self.attention_dropout = nn.Dropout(dropout)

        def _scaled_dot_product_attention(self, q, k, v, mask=None):
            """Implementación optimizada de atención sin caché."""
            attn_weights = torch.matmul(q, k.transpose(-2, -1)) * self.scale

            if mask is not None:
                attn_weights = attn_weights.masked_fill(mask, float('-inf'))

            attn_weights = F.softmax(attn_weights, dim=-1)
            attn_weights = self.attention_dropout(attn_weights)

            return torch.matmul(attn_weights, v)

        def _attention_forward(self, x):
            """Forward pass de atención optimizado sin caché."""
            batch_size, seq_len, _ = x.shape

            # Proyectar Q, K, V y asegurar que sean contiguos
            q = self.q_proj(x).reshape(batch_size, seq_len, self.num_heads, self.head_dim).contiguous()
            k = self.k_proj(x).reshape(batch_size, seq_len, self.num_heads, self.head_dim).contiguous()
            v = self.v_proj(x).reshape(batch_size, seq_len, self.num_heads, self.head_dim).contiguous()

            # Transponer después de asegurar que son contiguos
            q = q.transpose(1, 2)
            k = k.transpose(1, 2)
            v = v.transpose(1, 2)

            # Generar máscara causal on-the-fly
            if self.training:
                mask = torch.triu(torch.ones(seq_len, seq_len, device=x.device, dtype=torch.bool), diagonal=1).unsqueeze(0).unsqueeze(0)
            else:
                mask = None

            # Calcular atención
            attn_output = self._scaled_dot_product_attention(q, k, v, mask)

            # Reorganizar y proyectar salida
            attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, -1)
            return self.out_proj(attn_output)

        def _compute_wavelet_coeffs_batched(self, x):
            """Calcula coeficientes wavelet de forma vectorizada para todo el batch."""
            batch_size, seq_len, channels = x.shape

            # Normalizar wavelets on-the-fly
            normalized_wavelets = F.normalize(self.mother_wavelets, dim=2)
            scaled_wavelets = normalized_wavelets * torch.sigmoid(self.scales)

            # Reorganizar para la convolución y asegurar que sea contiguo
            x_reshaped = x.reshape(batch_size * seq_len, 1, channels).contiguous()

            # Preparar wavelets para convolución
            wavelet_kernels = scaled_wavelets.squeeze(0).transpose(1, 2).contiguous()

            # Convolución vectorizada para todos los wavelets simultáneamente
            coeffs = F.conv1d(x_reshaped, wavelet_kernels, groups=1)

            return coeffs.reshape(batch_size, seq_len, self.num_wavelets)

        def _process_chunk(self, x_chunk):
            """Procesa un chunk de la secuencia."""
            # Calcular coeficientes wavelet
            coeffs = self._compute_wavelet_coeffs_batched(x_chunk)

            # Mezclar coeficientes
            mixed = self.coeff_mixer(coeffs)

            # Aplicar atención
            attn_output = self._attention_forward(mixed)

            return self.output_proj(attn_output + mixed)

        def forward(self, x):
            """Forward pass optimizado con procesamiento por chunks."""
            batch_size, seq_len, _ = x.shape

            # Para secuencias cortas, procesar directamente
            if seq_len <= self.chunk_size:
                return self._process_chunk(x)

            # Para secuencias largas, procesar por chunks
            outputs = []
            for start in range(0, seq_len, self.chunk_size):
                end = min(start + self.chunk_size, seq_len)
                chunk = x[:, start:end, :]
                chunk_output = self._process_chunk(chunk)
                outputs.append(chunk_output)

            # Concatenar resultados
            return torch.cat(outputs, dim=1)

    class CrossAttention(nn.Module):
        """
        Atención cruzada optimizada para interacción bidireccional entre características
        locales y globales. Mantiene la precisión en la detección de patrones mientras
        mejora la eficiencia computacional.
        """
        def __init__(self, hidden_size, global_size, num_heads, dropout):
            super().__init__()
            self.hidden_size = hidden_size
            self.global_size = global_size
            self.num_heads = num_heads
            self.dropout = dropout

            # Escalado para estabilidad numérica
            self.local_scale = (global_size // (num_heads // 2 if num_heads > 1 else 1)) ** -0.5
            self.global_scale = (hidden_size // num_heads) ** -0.5

            # Proyecciones optimizadas sin bias para reducir parámetros
            # pero manteniendo capacidad representativa
            self.local2global = nn.Linear(hidden_size, global_size, bias=True)
            self.global2local = nn.Linear(global_size, hidden_size, bias=True)

            # Proyecciones Q,K,V para atención local->global
            self.local_to_global_qkv = nn.ModuleDict({
                'q': nn.Linear(global_size, global_size, bias=True),
                'k': nn.Linear(global_size, global_size, bias=True),
                'v': nn.Linear(global_size, global_size, bias=True)
            })

            # Proyecciones Q,K,V para atención global->local
            self.global_to_local_qkv = nn.ModuleDict({
                'q': nn.Linear(hidden_size, hidden_size, bias=True),
                'k': nn.Linear(hidden_size, hidden_size, bias=True),
                'v': nn.Linear(hidden_size, hidden_size, bias=True)
            })

            # Proyecciones de salida
            self.local_out = nn.Linear(hidden_size, hidden_size, bias=False)
            self.global_out = nn.Linear(global_size, global_size, bias=False)

            # Dropouts estratégicos
            self.attn_dropout = nn.Dropout(dropout)
            self.proj_dropout = nn.Dropout(dropout)

        def _scaled_dot_product_attention(self, q, k, v, mask=None, scale=None):
            """
            Implementación optimizada de atención que usa flash attention cuando está disponible.
            """
            if hasattr(F, 'scaled_dot_product_attention'):
                # Usar Flash Attention si está disponible
                attn_output = F.scaled_dot_product_attention(
                    q, k, v,
                    attn_mask=mask,
                    dropout_p=self.dropout if self.training else 0.0,
                    is_causal=False
                )
            else:
                # Implementación estándar optimizada
                scores = torch.matmul(q, k.transpose(-2, -1)) * (scale or 1.0)

                if mask is not None:
                    scores = scores.masked_fill(mask, float('-inf'))

                attn_weights = F.softmax(scores, dim=-1)
                attn_weights = self.attn_dropout(attn_weights)
                attn_output = torch.matmul(attn_weights, v)

            return attn_output

        def _process_attention(self, q, k, v, num_heads, head_dim, mask=None, scale=None):
            """
            Procesa la atención manteniendo la precisión necesaria para detectar patrones.
            """
            batch_size, seq_len, _ = q.shape

            # Reshape preservando la información
            q = q.reshape(batch_size, seq_len, num_heads, head_dim).transpose(1, 2)
            k = k.reshape(batch_size, seq_len, num_heads, head_dim).transpose(1, 2)
            v = v.reshape(batch_size, seq_len, num_heads, head_dim).transpose(1, 2)

            # Calcular atención
            attn_output = self._scaled_dot_product_attention(q, k, v, mask, scale)

            # Reshape de vuelta
            return attn_output.transpose(1, 2).reshape(batch_size, seq_len, -1)

        def forward(self, local_feats, global_feats, local_mask=None, global_mask=None):
            """
            Forward pass optimizado que mantiene la precisión en la detección de patrones.

            Args:
                local_feats: [batch_size, seq_len, hidden_size]
                global_feats: [batch_size, seq_len, global_size]
                local_mask/global_mask: Máscaras opcionales para atención
            """
            batch_size, seq_len, _ = local_feats.shape

            # Proyección local->global
            local_as_global = self.local2global(local_feats)

            # Atención local->global
            q_global = self.local_to_global_qkv['q'](global_feats)
            k_global = self.local_to_global_qkv['k'](local_as_global)
            v_global = self.local_to_global_qkv['v'](local_as_global)

            global_head_dim = self.global_size // (self.num_heads // 2 if self.num_heads > 1 else 1)
            global_attn = self._process_attention(
                q_global, k_global, v_global,
                num_heads=self.num_heads // 2 if self.num_heads > 1 else 1,
                head_dim=global_head_dim,
                mask=local_mask,
                scale=self.local_scale
            )
            global_attn = self.global_out(global_attn)
            global_attn = self.proj_dropout(global_attn)

            # Proyección global->local
            global_as_local = self.global2local(global_feats)

            # Atención global->local
            q_local = self.global_to_local_qkv['q'](local_feats)
            k_local = self.global_to_local_qkv['k'](global_as_local)
            v_local = self.global_to_local_qkv['v'](global_as_local)

            local_head_dim = self.hidden_size // self.num_heads
            local_attn = self._process_attention(
                q_local, k_local, v_local,
                num_heads=self.num_heads,
                head_dim=local_head_dim,
                mask=global_mask,
                scale=self.global_scale
            )
            local_attn = self.local_out(local_attn)
            local_attn = self.proj_dropout(local_attn)

            # Actualización residual con dropout estratégico
            local_updated = local_feats + self.proj_dropout(local_attn)
            global_updated = global_feats + self.proj_dropout(global_attn)

            return local_updated, global_updated

    # ========================================================
    #                INICIALIZACIÓN DE LA CLASE
    # ========================================================

    def __init__(
        self,
        hidden_size: int = 512,
        global_size: int = 128,
        num_layers: int = 2,
        num_heads: int = 8,
        context_size: int = 512,
        dropout: float = 0.1,
        learnable_dropout: float = 0.12,
        max_seq_length: int = 2048,
        window_size: int = 128,
        vocab_size: int = 256  # Añadido para mayor generalidad
    ):
        super().__init__()

        self.context_size = context_size
        self.hidden_size = hidden_size
        self.global_size = global_size
        self.window_size = window_size
        self.vocab_size = vocab_size

        # ========== EMBEDDING ================
        self.byte_embedding = nn.Embedding(256, hidden_size)
        self.embedding_dropout = nn.Dropout(dropout)

        # ========== ANALISIS LOCAL =============
        local_encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_size,
            nhead=num_heads,
            dim_feedforward=hidden_size * 4,
            dropout=dropout,
            batch_first=True,
            norm_first=True
        )
        self.local_encoder = nn.TransformerEncoder(
            encoder_layer=local_encoder_layer,
            num_layers=num_layers,
            norm=nn.LayerNorm(hidden_size),
            enable_nested_tensor=True
        )

        # ========== REDUCCION A GLOBAL (MEJORADA) ============
        self.global_reduction = self.AdaptiveWaveletLayer(
            hidden_size=hidden_size,
            global_size=global_size,
            num_wavelets=8,  # Ajustable según necesidades
            dropout=dropout
        )
        self.global_reduction_dropout = nn.Dropout(dropout)

        # ========== ANALISIS GLOBAL =============
        global_encoder_layer = nn.TransformerEncoderLayer(
            d_model=global_size,
            nhead=num_heads // 2 if num_heads > 1 else 1,
            dim_feedforward=global_size * 4,
            dropout=dropout,
            batch_first=True,
            norm_first=True
        )
        self.global_encoder = nn.TransformerEncoder(
            encoder_layer=global_encoder_layer,
            num_layers=num_layers,
            norm=nn.LayerNorm(global_size),
            enable_nested_tensor=True
        )

        # ========== SKIP CONNECTIONS LOCALES =============
        self.local_skip_weights = nn.ParameterList([
            nn.Parameter(torch.ones(1, 1, hidden_size))
            for _ in range(num_layers)
        ])
        self.local_gates = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_size * 2, hidden_size), nn.Dropout(learnable_dropout), nn.Sigmoid()
            )
            for _ in range(num_layers)
        ])

        # ========== SKIP CONNECTIONS GLOBALES =============
        self.global_skip_weights = nn.ParameterList([
            nn.Parameter(torch.ones(1, 1, global_size))
            for _ in range(num_layers)
        ])
        self.global_gates = nn.ModuleList([
            nn.Sequential(
                nn.Linear(global_size * 2, global_size), nn.Dropout(learnable_dropout), nn.Sigmoid()
            )
            for _ in range(num_layers)
        ])

        # ========== DROPOUT PARA SKIP CONNECTIONS =============
        self.skip_dropout = nn.Dropout(learnable_dropout)

        # ========== ATENCION CRUZADA =============
        self.cross_attention = self.CrossAttention(
            hidden_size=hidden_size,
            global_size=global_size,
            num_heads=num_heads,
            dropout=dropout
        )

        # ========== PONDERACION ADAPTATIVA =============
        self.weight_network = nn.Sequential(
            nn.Linear(hidden_size + global_size, 1), nn.Dropout(dropout), nn.Sigmoid()
        )

        # ========== GATE FINAL Y SALIDA ============
        self.output_gate = nn.Sequential(
            nn.Linear((hidden_size + global_size) * 2, hidden_size + global_size), nn.Dropout(learnable_dropout), nn.Sigmoid()
        )
        self.output = nn.Linear(hidden_size + global_size, vocab_size)
        self.output_dropout = nn.Dropout(dropout)

        # ========== NORMALIZACIONES GENERALES =============
        self.dropout = nn.Dropout(dropout)
        self.learnable_dropout = nn.Dropout(learnable_dropout)
        self.layer_norm = nn.LayerNorm(hidden_size)

        # ========== MASCARA CAUSAL PRE-GENERADA =============
        self._initialize_mask_cache(max_seq_length)
        self.base_smoothing = 0.1
        self.smoothing_scale = nn.Parameter(torch.tensor(0.5))
    # ========================================================
    #              METODOS INTERNOS
    # ========================================================
    def _compute_adaptive_smoothing(self, logits: torch.Tensor) -> torch.Tensor:
        """
        Calcula un factor de suavizado adaptativo basado en la confianza del modelo
        y la temperatura dinámica del batch.

        Este método:
        1. Calcula un factor de suavizado base usando la probabilidad máxima
        2. Ajusta la temperatura según la entropía del batch
        3. Combina ambos efectos para suavizar adaptativamente las probabilidades

        Args:
            logits (torch.Tensor): Tensor de logits sin normalizar con shape [B, S, V]
                donde B es batch_size, S es sequence_length, y V es vocab_size.

        Returns:
            torch.Tensor: Factor de suavizado por posición con shape [B, S].
                        Los valores están entre [0, base_smoothing].
                        - Valores más altos indican más suavizado (alta confianza)
                        - Valores más bajos indican menos suavizado (baja confianza)

        Ejemplo:
            Para una secuencia donde el modelo está:
            - Muy seguro (p=0.9) → factor ≈ base_smoothing
            - Moderadamente seguro (p=0.6) → factor ≈ 0.5 * base_smoothing
            - Inseguro (p=0.3) → factor ≈ 0.1 * base_smoothing
        """
        # 1. Obtener distribución de probabilidad base
        probs = F.softmax(logits, dim=-1)

        # 2. Calcular máxima probabilidad por posición [B, S]
        max_probs = probs.max(dim=-1)[0]

        # 3. Calcular entropía por posición y promedio del batch
        entropy = -torch.sum(probs * torch.log(probs + 1e-10), dim=-1)
        mean_entropy = entropy.mean()

        # 4. Factor de temperatura basado en entropía
        temperature = torch.sigmoid(mean_entropy)
        temperature = temperature.clamp(0.1, 2.0)

        # 5. Calcular factor adaptativo basado en confianza
        confidence_factor = torch.sigmoid(self.smoothing_scale * (max_probs - 0.5))

        # 6. Combinar efectos
        # - Alta temperatura reduce el efecto del suavizado
        # - Baja temperatura mantiene el suavizado basado en confianza
        adaptive_factor = confidence_factor / temperature.unsqueeze(-1).expand_as(confidence_factor)

        # 7. Escalar al rango final y aplicar límites
        smoothing = (self.base_smoothing * adaptive_factor).clamp(0.0, self.base_smoothing)

        return smoothing

    def _initialize_mask_cache(self, max_seq_length: int):
        masks = torch.triu(torch.ones(max_seq_length, max_seq_length), diagonal=1).bool()
        self.register_buffer('cached_masks', masks, persistent=False)

    def get_causal_mask(self, seq_length: int) -> torch.Tensor:
        if seq_length <= self.cached_masks.size(0):
            return self.cached_masks[:seq_length, :seq_length]
        return torch.triu(torch.ones(seq_length, seq_length), diagonal=1).bool()

    def _apply_local_skip(self, x: torch.Tensor, skip: torch.Tensor, layer_idx: int) -> torch.Tensor:
        weighted_skip = self.skip_dropout(self.local_skip_weights[layer_idx]) * skip
        gate_input = torch.cat([x, weighted_skip], dim=-1)
        gate = self.local_gates[layer_idx](gate_input)
        out = x + self.learnable_dropout(gate * weighted_skip)
        return out

    def _apply_global_skip(self, x: torch.Tensor, skip: torch.Tensor, layer_idx: int) -> torch.Tensor:
        weighted_skip = self.skip_dropout(self.global_skip_weights[layer_idx]) * skip
        gate_input = torch.cat([x, weighted_skip], dim=-1)
        gate = self.global_gates[layer_idx](gate_input)
        out = x + self.learnable_dropout(gate * weighted_skip)
        return out

    # ========================================================
    #                FORWARD PRINCIPAL
    # ========================================================
    def forward(self, input_bytes: torch.Tensor, return_probabilities: bool = False) -> torch.Tensor:
        """
        Procesa la entrada aplicando análisis local y global mediante atención cruzada,
        combinando características de diferentes niveles de abstracción.

        Este método implementa el forward pass completo del modelo que:
        1. Procesa los bytes de entrada mediante embeddings
        2. Aplica análisis local y global en paralelo
        3. Combina información mediante atención cruzada
        4. Utiliza skip connections y gates adaptativos
        5. Genera probabilidades o entropía según se requiera

        Args:
            input_bytes (torch.Tensor): Tensor de entrada con forma [batch_size, seq_length]
                                    conteniendo índices de bytes.
            return_probabilities (bool, opcional): Si True, retorna distribución de probabilidades
                                                suavizada. Si False, retorna entropía.
                                                Por defecto: False.

        Returns:
            torch.Tensor: Si return_probabilities es True:
                        - Tensor de probabilidades suavizadas con forma [batch_size, seq_length, vocab_size]
                        Si return_probabilities es False:
                        - Tensor de entropía con forma [batch_size, seq_length] o [batch_size, window_size]

        Notas sobre el flujo de datos:
            1. Embeddings: input_bytes -> x [B, S, H]
            2. Análisis Local: x -> local_features [B, S, H]
            3. Análisis Global: x -> global_x -> global_features [B, S, G]
            4. Atención Cruzada: (local_features, global_features) -> (local_attn, global_attn)
            5. Skip Connections: Mejora flujo de gradientes
            6. Combinación Adaptativa: Mezcla características locales y globales
            7. Gate Final: Controla flujo de información residual
            8. Salida: Genera logits -> probabilidades/entropía

        Donde:
            B = batch_size
            S = sequence_length
            H = hidden_size
            G = global_size
        """
        # Asegurar que las dimensiones de entrada sean escalares
        batch_size, seq_length = input_bytes.shape
        seq_length = int(seq_length.item()) if isinstance(seq_length, torch.Tensor) else int(seq_length)
        device = input_bytes.device

        # ========= EMBEDDINGS INICIALES =========
        x = self.byte_embedding(input_bytes)
        x = self.embedding_dropout(x)

        # ========= MÁSCARA CAUSAL =========
        mask = self.get_causal_mask(seq_length).to(device)

        # Usamos autocast para mezclar precisión (solo si usamos GPUs con AMP)
        with torch.cuda.amp.autocast():
            # ========= ANALISIS LOCAL =========
            local_features = self.local_encoder(src=x, mask=mask, is_causal=True)

            # ========= ANALISIS GLOBAL =========
            # 1. Reducción MLP (captura no lineal)
            global_x = self.global_reduction(x)
            global_x = self.global_reduction_dropout(global_x)

            # 2. Encoder Global
            global_features = self.global_encoder(src=global_x, mask=mask, is_causal=True)

            # ========= ATENCIÓN CRUZADA (Local <-> Global) =========
            local_attn, global_attn = self.cross_attention(
                local_feats=local_features,
                global_feats=global_features,
                local_mask=mask,       # Opcional, enmascarado local
                global_mask=mask       # Opcional, enmascarado global
            )

            # ========= SKIP CONNECTIONS POR CAPA =========
            for layer_idx in range(len(self.local_skip_weights)):
                local_attn = self._apply_local_skip(local_attn, local_features, layer_idx)
                global_attn = self._apply_global_skip(global_attn, global_features, layer_idx)

            # ========= COMBINACIÓN ADAPTATIVA LOCAL-GLOBAL =========
            combined_features = torch.cat([local_attn, global_attn], dim=-1)
            weights = self.weight_network(combined_features)  # [B, S, 1]

            weighted_local = local_attn * weights
            weighted_global = global_attn * (1 - weights)
            final_features = torch.cat([weighted_local, weighted_global], dim=-1)

            # ========= CONCATENACION PARA GATE FINAL =========
            initial_state = torch.cat([x, global_x], dim=-1)
            final_gate_input = torch.cat([final_features, initial_state], dim=-1)
            final_gate = self.output_gate(final_gate_input)
            combined = final_features + self.learnable_dropout(final_gate * initial_state)

            # ========= DROPOUT Y SALIDA =========
            combined = self.output_dropout(combined)
            logits = self.output(combined)
            probabilities = F.softmax(logits, dim=-1, dtype=torch.float32)

        if return_probabilities:
            probs = F.softmax(logits, dim=-1)
            # Aplicar suavizado adaptativo
            smoothing = self._compute_adaptive_smoothing(logits)
            vocab_size = logits.size(-1)
            smoothed_probs = probs * (1 - smoothing.unsqueeze(-1)) + smoothing.unsqueeze(-1) / vocab_size
            return smoothed_probs

        return self.compute_entropy(probabilities)

    # ========================================================
    #             CÁLCULO DE ENTROPÍA
    # ========================================================
    def compute_entropy(self, probabilities: torch.Tensor, use_sliding_window: bool = True) -> torch.Tensor:
        """
        Calcula la entropía H(P) = -Σ p(x)*log2 p(x).

        Si `use_sliding_window` es True y la longitud de la secuencia es mayor o igual a `window_size`,
        utiliza ventanas deslizantes para calcular la entropía de forma local y luego promedia los resultados.
        De lo contrario, calcula la entropía de manera global sobre toda la secuencia.

        Args:
            probabilities (torch.Tensor): Tensor de probabilidades con forma [B, S, V],
                                        donde
                                        B = tamaño del batch,
                                        S = longitud de la secuencia,
                                        V = número de categorías o variables.
            use_sliding_window (bool, opcional): Indica si se debe usar una ventana deslizante para secuencias largas.
                                                Por defecto es True.

        Returns:
            torch.Tensor: Tensor de entropía con forma [B, window_size] si se usa ventana deslizante,
                        o [B, S] si se calcula globalmente.
        """
        B, S, V = probabilities.shape
        seq_length = S.item() if isinstance(S, torch.Tensor) else S

        if use_sliding_window and seq_length >= self.window_size:
            # Generar ventanas deslizantes
            # La función `unfold` crea un nuevo tensor con ventanas deslizantes a lo largo de la dimensión de la secuencia.
            # Parámetros:
            #   dimension=1: dimensión de la secuencia.
            #   size=self.window_size: tamaño de cada ventana.
            #   step=max(1, self.window_size // 2): desplazamiento entre ventanas (mitad del tamaño de la ventana o al menos 1).
            # Resultado: [B, num_windows, window_size, V]
            windows = probabilities.unfold(1, self.window_size, max(1, self.window_size // 2))

            # Reorganizar dimensiones para facilitar el cálculo vectorizado de la entropía
            # Cambiar a [B, window_size, num_windows, V]
            windows = windows.permute(0, 2, 1, 3)

            # Cálculo vectorizado de la entropía por ventana
            # Aplicamos log2 a las probabilidades, asegurando que no haya valores menores que 1e-10 para evitar log(0)
            # Multiplicamos elemento a elemento por las probabilidades y sumamos sobre la última dimensión (V)
            # Resultado: [B, window_size, num_windows]
            ent = -torch.sum(windows * torch.log2(torch.clamp(windows, min=1e-10)), dim=-1)

            # Promediar la entropía sobre el número de ventanas (dim=2)
            # Resultado final: [B, window_size]
            return ent.mean(dim=2)
        else:
            # Cálculo directo de la entropía sobre toda la secuencia
            # Aplicamos log2 a las probabilidades, asegurando que no haya valores menores que 1e-10 para evitar log(0)
            # Multiplicamos elemento a elemento por las probabilidades y sumamos sobre la última dimensión (V)
            # Resultado: [B, S]
            ent = -torch.sum(probabilities * torch.log2(torch.clamp(probabilities, min=1e-10)), dim=-1)
            return ent
# =============================================================================
#                           BLT (Byte-Level Transformer)
# =============================================================================

class BLT(nn.Module):
    """
    Byte-Level Transformer (BLT) con parcheo adaptativo optimizado,
    SIN usar cache en ningún lugar.

    Esta versión mejorada integra explícitamente las probabilidades
    generadas por EntropyLM, con el fin de refinar la lógica de
    segmentación de parches y aprovechar la información de confiabilidad.
    """

    def __init__(self, config):
        """
        Args:
            config: Objeto de configuración (BLTConfig o similar) que contiene:
                - hidden_size: Dimensión oculta de los embeddings
                - resid_dropout: Tasa de dropout residual
                - attention_dropout: Tasa de dropout en la atención
                - min_patch_size: Tamaño mínimo de parche
                - max_patch_size: Tamaño máximo de parche
                - initial_entropy_threshold: Valor inicial del umbral de entropía
                - num_heads, etc.: Otros hiperparámetros relevantes
                - (Opcional) param prob_factor: Factor para combinar entropía y prob.
        """
        super().__init__()
        self.config = config

        # =================== Submódulos Principales ===================
        self.local_encoder = LocalEncoder(config)
        self.global_transformer = GlobalTransformer(config)
        self.local_decoder = LocalDecoder(config)

        # EntropyLM mejora: Se usará para obtener entropía y/o probabilidades
        self.entropy_model = EntropyLM(
            hidden_size=config.hidden_size,
            num_layers=config.entropy_model_layers,
            num_heads=config.num_heads,
            context_size=config.entropy_context_size,
            dropout=config.attention_dropout
        )

        # Normalización y Dropout
        self.global_norm = RMSNorm(config.hidden_size)
        self.global_dropout = nn.Dropout(config.resid_dropout)

        # Configuración de Parcheo
        self.min_patch_size = config.min_patch_size
        self.max_patch_size = config.max_patch_size

        # Parámetros Aprendibles para Umbrales
        # (Se utilizan para calcular el umbral adaptativo de entropía)
        self.learnable_base_threshold = nn.Parameter(torch.tensor(config.initial_entropy_threshold))
        self.learnable_std_scale = nn.Parameter(torch.tensor(0.5))

        # Parámetro adicional para ponderar el uso de probabilidades
        # Ajusta cuánto peso se le da a la "discrepancia" de prob. frente a la entropía
        # (Podrías exponerlo como config.prob_factor si lo deseas)
        self.prob_factor = getattr(config, "prob_factor", 0.3)

        # Tamaño de la ventana para cálculos de estadística adaptativa
        self.window_size = 128
        self.stats_buffer = {}

        # Dropout para parámetros
        self.param_dropout = nn.Dropout(p=0.1)

        # Atributos para rastrear el progreso de entrenamiento
        self.current_step = 0
        self.total_steps = 1000

    # ------------------------------------------------------------------
    #         Funciones Internas para Cálculo del Umbral Adaptativo
    # ------------------------------------------------------------------
    def _compute_adaptive_threshold(self, entropies: torch.Tensor) -> torch.Tensor:
        """
        Ajusta dinámicamente el umbral de entropía basándose en estadísticos
        (media y desviación estándar). Soporta secuencias cortas.

        Args:
            entropies (torch.Tensor): Tensor [B, S] con las entropías.
        Returns:
            threshold (torch.Tensor): Umbrales por cada muestra del batch [B, 1].
        """
        batch_size = entropies.size(0)
        means = []
        stds = []

        for i in range(batch_size):
            seq_len = entropies[i].size(0)
            # Si la secuencia >= window_size, usar ventanas deslizantes
            if seq_len >= self.window_size:
                windows = entropies[i].unfold(0, self.window_size, max(1, self.window_size // 2))
                window_mean = windows.mean(dim=1, keepdim=True)
                window_std = torch.sqrt(
                    torch.var(windows, dim=1, keepdim=True, unbiased=False) + 1e-6
                )
                means.append(window_mean.mean())
                stds.append(window_std.mean())
            else:
                # Calcular media y std en toda la secuencia
                entire_mean = entropies[i].mean()
                entire_std = entropies[i].std(unbiased=False)
                means.append(entire_mean)
                stds.append(entire_std)

        mean = torch.stack(means).view(batch_size, 1)
        std = torch.stack(stds).view(batch_size, 1)

        base_threshold = self.param_dropout(self.learnable_base_threshold)
        std_scale = self.param_dropout(self.learnable_std_scale)

        # Aplicar sigmoide y limitar en [0.1, 0.9]
        threshold = torch.sigmoid(base_threshold + std_scale * std).clamp(min=0.1, max=0.9)
        return threshold

    # ------------------------------------------------------------------
    #          Función Auxiliar para Integrar Prob y Entropía
    # ------------------------------------------------------------------
    def _compute_boundary_score(self, entropies: torch.Tensor, probabilities: torch.Tensor) -> torch.Tensor:
        """
        Calcula un vector de puntajes de corte, combinando la entropía
        y la discrepancia entre probabilidades consecutivas.

        Args:
            entropies: [S] Entropías de la secuencia (una por posición).
            probabilities: [S, V=256] Probabilidades por cada posición.

        Returns:
            scores: [S-1] Puntaje para cada posición j, j in [0..S-2],
                    indicando cuán favorable es hacer un corte entre j y j+1.
        """
        seq_len = probabilities.size(0)
        if seq_len < 2:
            # Si la secuencia es muy corta, no calculamos "discrepancia"
            return entropies[:-1]  # Devolvemos algo sencillo

        # Discrepancia de prob.: Distancia L1 entre P_j e P_{j+1}
        # shape: [S-1]
        prob_diff = torch.sum(
            torch.abs(probabilities[:-1] - probabilities[1:]),
            dim=-1
        )

        # Por simplicidad, el puntaje de corte en la posición j se define como:
        # score_j = ent_j + prob_factor * prob_diff_j
        # Donde ent_j = entropía en la posición j, y prob_diff_j indica
        # el "salto" en la distribución de prob. entre j y j+1.
        # Podríamos refinar con ent_{j+1} también, etc.
        # Ajustar a gusto la mezcla.
        cut_scores = entropies[:-1] + (self.prob_factor * prob_diff)

        return cut_scores

    # ------------------------------------------------------------------
    #             Cálculo de Parques con Entropía + Probabilidades
    # ------------------------------------------------------------------
    def compute_patches(self, bytes_input: torch.Tensor) -> List[torch.Tensor]:
        """
        Calcula las fronteras (boundaries) de los parches en la secuencia de bytes de entrada
        utilizando una combinación adaptativa de entropía y discrepancia en las probabilidades.

        Esta versión mejorada integra tanto la entropía como las probabilidades generadas
        por el modelo `EntropyLM` para decidir de manera adaptativa dónde segmentar los parches.
        Además, asegura que todas las condiciones booleanas se manejen correctamente para
        evitar errores como "Boolean value of Tensor with more than one value is ambiguous".

        Args:
            bytes_input (torch.Tensor): Tensor de entrada con forma [batch_size, seq_length],
                                        conteniendo índices de bytes.

        Returns:
            List[torch.Tensor]: Lista de tensores, cada uno conteniendo las posiciones de
                                corte (boundaries) para cada muestra en el batch.
        """
        batch_size = bytes_input.size(0)
        boundaries_list = []

        # ====================== Progreso de Entrenamiento ======================
        # Calcula el progreso de entrenamiento para ajustar la mezcla de boundaries fijos y adaptativos.
        training_progress = (
            getattr(self, 'current_step', 0) / getattr(self, 'total_steps', 1000)
        ) if self.training else 1.0

        # Determina el factor de mezcla basado en el progreso de entrenamiento.
        if training_progress < 0.2:
            mix_factor = 0.0  # Fase inicial: completamente fija
        elif training_progress > 0.8:
            mix_factor = 1.0  # Fase final: completamente adaptativa
        else:
            # Fase intermedia: mezcla gradual entre fija y adaptativa
            raw_mix = (training_progress - 0.2) / 0.6
            mix_factor = 1 / (1 + torch.exp(torch.tensor(-10 * (raw_mix - 0.5))))

        # ====================== Boundaries Fijos Base ======================
        # Calcula los boundaries fijos basados en el tamaño mínimo de parche.
        seq_len = bytes_input.size(1)
        if self.min_patch_size <= seq_len:
            base_boundaries = torch.arange(
                self.min_patch_size,
                seq_len,
                self.min_patch_size,
                device=bytes_input.device
            )
        else:
            base_boundaries = torch.tensor([], device=bytes_input.device, dtype=torch.long)

        # ====================== Fase Completamente Fija ======================
        # Si el mix_factor es 0, retorna únicamente los boundaries fijos.
        if mix_factor == 0:
            return [base_boundaries for _ in range(batch_size)]

        # ====================== Cálculo de Entropía y Probabilidades ======================
        with torch.cuda.amp.autocast():
            # 1. Obtiene las probabilidades del modelo EntropyLM
            probabilities = self.entropy_model(bytes_input, return_probabilities=True)  # [B, S, V]

            # 2. Obtiene las entropías del modelo EntropyLM
            entropies = self.entropy_model(bytes_input, return_probabilities=False)  # [B, S]

            # 3. Aplica dropout a las entropías si está en modo de entrenamiento
            if self.training:
                entropies = F.dropout(entropies, p=0.1, training=True)

        # ====================== Cálculo de Umbrales Adaptativos ======================
        # Calcula umbrales de entropía adaptativos para cada posición en la secuencia.
        thresholds = self._compute_adaptive_threshold(entropies)  # [B, 1]

        # ====================== Procesamiento por Cada Muestra del Batch ======================
        for b_idx in range(batch_size):
            current_ent = entropies[b_idx]         # [S]
            current_prob = probabilities[b_idx]    # [S, V]
            threshold = thresholds[b_idx].item()    # Valor escalar

            # Combina entropía y discrepancias de probabilidad para calcular scores de corte.
            # shape: [S-1]
            boundary_scores = self._compute_boundary_score(current_ent, current_prob)

            adaptive_boundaries = []
            last_boundary = 0
            S = current_ent.size(0)

            # Itera sobre cada posición en la secuencia para decidir si cortar.
            for pos in range(S):
                current_size = pos - last_boundary + 1

                if pos < S - 1:
                    # Verifica las condiciones de corte:
                    # 1) Tamaño del parche >= max_patch_size
                    # 2) Tamaño del parche >= min_patch_size y score de corte > umbral adaptativo
                    condition1 = current_size >= self.max_patch_size
                    condition2 = (
                        (current_size >= self.min_patch_size) and
                        (boundary_scores[pos].item() > threshold * (1 + mix_factor))
                    )
                    cut_condition = condition1 or condition2
                else:
                    # Forzar el corte al final de la secuencia.
                    cut_condition = (pos == S - 1)

                # Si se cumple la condición de corte, agregar la posición como boundary.
                if cut_condition:
                    adaptive_boundaries.append(pos)
                    last_boundary = pos + 1

            # Convierte la lista de boundaries adaptativos a un tensor.
            adaptive_boundaries = torch.tensor(
                adaptive_boundaries, device=bytes_input.device, dtype=torch.long
            )

            # ====================== Mezcla de Boundaries Fijos y Adaptativos ======================
            if mix_factor == 1:
                # Si mix_factor es 1, usa solo boundaries adaptativos.
                final_boundaries = adaptive_boundaries
            else:
                # Combina boundaries fijos y adaptativos ponderados por mix_factor.
                fixed_weight = 1 - mix_factor
                adaptive_weight = mix_factor
                num_boundaries = int(
                    fixed_weight * len(base_boundaries) +
                    adaptive_weight * len(adaptive_boundaries)
                )

                if num_boundaries == 0:
                    # Si no hay boundaries, retorna un tensor vacío.
                    final_boundaries = torch.tensor([], device=bytes_input.device, dtype=torch.long)
                else:
                    # Combina los boundaries fijos y adaptativos.
                    if len(base_boundaries) > 0 and len(adaptive_boundaries) > 0:
                        combined = torch.cat([
                            base_boundaries * fixed_weight,
                            adaptive_boundaries.float() * adaptive_weight  # Convierte a float para la multiplicación
                        ])
                    elif len(base_boundaries) > 0:
                        combined = base_boundaries.float() * fixed_weight
                    else:
                        combined = adaptive_boundaries.float() * adaptive_weight

                    # Selecciona los top boundaries según num_boundaries.
                    if combined.numel() > num_boundaries:
                        _, indices = torch.topk(torch.abs(combined), num_boundaries)
                        final_boundaries = torch.sort(combined[indices])[0].long()
                    else:
                        final_boundaries = torch.sort(combined)[0].long()

            # ====================== Asegurar Límites y Limpieza ======================
            # Filtra boundaries que estén dentro de los límites permitidos.
            final_boundaries = final_boundaries[
                (final_boundaries >= self.min_patch_size) &
                (final_boundaries <= seq_len - 1)
            ]

            # Agrega los boundaries finales al listado.
            boundaries_list.append(final_boundaries)

        return boundaries_list


    # ------------------------------------------------------------------
    #                 Forward Principal del BLT
    # ------------------------------------------------------------------
    def forward(self, bytes_input: torch.Tensor,
        patch_boundaries: Optional[List[torch.Tensor]] = None) -> torch.Tensor:
        """
        Forward principal:
          1) Cálculo de byte_encodings vía LocalEncoder
          2) Segmentación en parches (adaptativa) y reducción (mean)
          3) Paso por GlobalTransformer
          4) Decodificación (LocalDecoder) para obtener logits finales

        Args:
            bytes_input: [B, S] Tensor con bytes de entrada
            patch_boundaries: Lista opcional de boundaries para forzar parches

        Returns:
            logits: [B, S, 256] Logits sobre bytes.
        """
        batch_size, seq_length = bytes_input.size()

        # -------------------- Encodings Locales --------------------
        byte_encodings = self.local_encoder(bytes_input)
        byte_encodings = self.global_dropout(byte_encodings)

        # -------------------- Boundaries Adaptativos --------------------
        if patch_boundaries is None:
            patch_boundaries = self.compute_patches(bytes_input)

        # -------------------- Calcular Patches --------------------
        patch_means = []
        max_patches = 0

        for i in range(batch_size):
            boundaries = patch_boundaries[i]
            patches = []
            start = 0

            # Asegurar que boundaries tenga un final en seq_length
            if boundaries.numel() > 0:
                if boundaries.dim() == 0:
                    boundaries = boundaries.unsqueeze(0)
                boundaries = torch.cat([
                    boundaries,
                    torch.tensor([seq_length], device=boundaries.device)
                ])
            else:
                boundaries = torch.tensor([seq_length], device=bytes_input.device)

            # Reducir cada parche
            for end in boundaries:
                if end > start:
                    patch = byte_encodings[i, start:end].mean(dim=0)
                    patches.append(patch)
                    start = end

            # Si no hay parches creados, usar toda la secuencia
            if not patches:
                patches.append(byte_encodings[i].mean(dim=0))

            patches_tensor = torch.stack(patches)
            patch_means.append(patches_tensor)
            max_patches = max(max_patches, patches_tensor.size(0))

        # -------------------- Padding de Parches --------------------
        padded_patches = torch.zeros(
            batch_size, max_patches, self.config.hidden_size,
            device=bytes_input.device,
            dtype=byte_encodings.dtype
        )

        for i, patches in enumerate(patch_means):
            num_patches = patches.size(0)
            padded_patches[i, :num_patches] = patches

        # -------------------- Global Transformer --------------------
        global_output = self.global_transformer(padded_patches)
        global_output = self.global_norm(global_output)
        global_output = self.global_dropout(global_output)

        # -------------------- Decodificación Local --------------------
        logits = self.local_decoder(
            self.global_dropout(byte_encodings),
            global_output
        )
        return logits

    # ------------------------------------------------------------------
    #                 Utilidad: Progreso de Entrenamiento
    # ------------------------------------------------------------------
    def update_training_progress(self, current_step, total_steps):
        """
        Actualiza el progreso del entrenamiento para ajustar gradualmente
        la mezcla de boundaries fijos y adaptativos.
        """
        self.current_step = current_step
        self.total_steps = total_steps

# =============================================================================
#                       CONFIGURACIÓN DEL MODELO BLT
# =============================================================================

class BLTConfig:
    """
    Configuración del Byte-Level Transformer (BLT).
    """
    def __init__(
        self,
        hidden_size=256,
        intermediate_size=1024,
        num_heads=4,
        encoder_layers=1,
        global_layers=6,
        decoder_layers=2,
        attention_dropout=0.1,
        resid_dropout=0.12,
        ngram_vocab_size=10000,
        window_size=512,
        max_position_embeddings=1024,
        entropy_model_layers=2,
        entropy_context_size=512,
        entropy_threshold=0.5,
        min_patch_size=32,
        max_patch_size=512,
        initial_entropy_threshold=0.5
    ):
        self.hidden_size = hidden_size
        self.intermediate_size = intermediate_size
        self.num_heads = num_heads
        self.encoder_layers = encoder_layers
        self.global_layers = global_layers
        self.decoder_layers = decoder_layers
        self.attention_dropout = attention_dropout
        self.resid_dropout = resid_dropout
        self.ngram_vocab_size = ngram_vocab_size
        self.window_size = window_size
        self.max_position_embeddings = max_position_embeddings
        self.entropy_model_layers = entropy_model_layers
        self.entropy_context_size = entropy_context_size
        self.entropy_threshold = entropy_threshold
        self.min_patch_size = min_patch_size
        self.max_patch_size = max_patch_size
        self.initial_entropy_threshold = initial_entropy_threshold


# =============================================================================
#                          UTILIDADES DE MÁSCARAS
# =============================================================================

def create_block_causal_mask(seq_length: int, window_size: int = None):
    """
    Crea una máscara causal con ventana opcional (sin cache).
    """
    mask = torch.ones((seq_length, seq_length), dtype=torch.bool)
    mask = torch.triu(mask, diagonal=1)

    if window_size:
        indices = torch.arange(seq_length)
        window_mask = (indices.unsqueeze(1) - indices.unsqueeze(0)).abs() <= window_size
        mask = mask | ~window_mask

    return mask

def create_patch_mask(patch_boundaries, seq_length):
    """
    Crea una máscara para parches con optimizaciones de memoria y soporte
    para parches dinámicos (sin cache).
    """
    if isinstance(patch_boundaries, list):
        patch_boundaries = torch.tensor(patch_boundaries)
    elif patch_boundaries.numel() == 0:
        return torch.zeros((seq_length, seq_length), dtype=torch.bool)

    if not torch.all(patch_boundaries[1:] > patch_boundaries[:-1]):
        patch_boundaries, _ = torch.sort(patch_boundaries)

    mask = torch.zeros((seq_length, seq_length), dtype=torch.bool)

    start = 0
    for end in patch_boundaries:
        if end > start:
            mask[start:end, start:end] = True
            start = end

    if start < seq_length:
        mask[start:, start:] = True

    return mask


# =============================================================================
#                         FUNCIONES DE ENTRENAMIENTO
# =============================================================================

def train_step(model, optimizer, batch, patch_config):
    """
    Realiza un paso de entrenamiento, calculando la pérdida y aplicando backpropagation.
    """
    optimizer.zero_grad()

    input_bytes = batch[:, :-1]
    target_bytes = batch[:, 1:]
    patch_boundaries = None

    if patch_config.scheme == 'entropy':
        with torch.no_grad():
            entropies = model.entropy_model(input_bytes)
            indices = torch.where(entropies > patch_config.entropy_threshold)
            if indices[0].numel() == 0:
                patch_boundaries = torch.tensor([], dtype=torch.long, device=entropies.device)
            else:
                patch_boundaries = indices[1]

    elif patch_config.scheme == 'space':
        indices = torch.where(input_bytes == 32)
        if indices[0].numel() == 0:
            patch_boundaries = torch.tensor([], dtype=torch.long, device=input_bytes.device)
        else:
            patch_boundaries = indices[1] + 1

    elif patch_config.scheme == 'fixed':
        stride = patch_config.stride
        seq_length = input_bytes.size(1)
        patch_boundaries = torch.arange(
            stride, seq_length, stride, device=input_bytes.device
        )

    logits = model(input_bytes, patch_boundaries)

    logits_reshaped = logits.view(-1, logits.size(-1))
    target_bytes_reshaped = target_bytes.view(-1)
    loss = F.cross_entropy(logits_reshaped, target_bytes_reshaped)

    loss.backward()
    optimizer.step()

    return loss.item()


# =============================================================================
#                           FUNCIÓN DE GENERACIÓN
# =============================================================================

def generate(model, start_bytes, max_length=1000, temperature=1.0, top_k=20, patch_config=None, device='cpu'):
    """
    Genera una secuencia de bytes a partir de un contexto inicial.
    """
    model.eval()
    generated = list(start_bytes)

    with torch.no_grad():
        while len(generated) < max_length:
            input_bytes = torch.tensor(generated, device=device).unsqueeze(0)

            if patch_config is not None:
                if patch_config.scheme == 'entropy':
                    entropies = model.entropy_model(input_bytes)
                    if patch_config.use_monotonic:
                        entropy_diff = entropies[:, 1:] - entropies[:, :-1]
                        patch_boundaries = torch.where(entropy_diff > patch_config.entropy_threshold)[1] + 1
                    else:
                        patch_boundaries = torch.where(entropies > patch_config.entropy_threshold)[1]
                else:
                    patch_boundaries = torch.arange(
                        patch_config.stride,
                        len(generated),
                        patch_config.stride,
                        device=device)
            else:
                patch_boundaries = None

            logits = model(input_bytes, patch_boundaries)

            # Forma: [1, seq_length, vocab_size]
            if logits.dim() == 3:
                logits = logits[0, -1] / temperature
            elif logits.dim() == 2:
                logits = logits[0] / temperature
            else:
                break

            top_k = min(top_k, logits.size(-1))
            topk_logits, topk_indices = torch.topk(logits, top_k)
            topk_probs = F.softmax(topk_logits, dim=-1)

            next_byte = topk_indices[torch.multinomial(topk_probs, 1).item()].item()
            generated.append(next_byte)

            if next_byte == 0:
                break

    return generated



In [ ]:
# @title ttblt/bltqwen.py
# https://github.com/ianbarber/ttblt/blob/main/ttblt/bltqwen.py
import torch
from torch import nn
from typing import Any, Dict, List, Mapping, Optional, Tuple, Union
from torchtune.modules.transformer import TransformerDecoder
from torchtune.modules import (MultiHeadAttention, RMSNorm, TransformerCrossAttentionLayer, TransformerSelfAttentionLayer,)
from torchtune.modules.model_fusion import FusionLayer
from torchtune.models.qwen2._component_builders import qwen2_mlp

PAD_ID = 256
BOS_ID = 257
EOS_ID = 258
NUM_SPECIAL_TOKENS = 3
VOCAB_SIZE = 256 + NUM_SPECIAL_TOKENS

################################################
# Local encoder/decoder (with cross-attn)
################################################

class LocalDecoder(nn.Module):
    def __init__(
        self,
        embed_dim: int,
        global_dim: int,
        vocab_size: int,
        num_layers: int = 8,
        num_cross_layers: int = 4,
        num_heads: int = 8,
        num_kv_heads: int = 8,
        hidden_dim: int = 4096,
        norm_eps: float = 1e-5,
        attn_dropout: float = 0.0,
        max_seq_len: int = 4096,
        dtype=torch.bfloat16,
    ):
        super().__init__()
        head_dim = embed_dim // num_heads

        # Self-attention layers
        layers = nn.ModuleList()
        for _ in range(num_layers):
            layer = TransformerSelfAttentionLayer(
                attn=self_attn,
                mlp=mlp,
                sa_norm=RMSNorm(embed_dim, eps=norm_eps),
                mlp_norm=RMSNorm(embed_dim, eps=norm_eps),
            )
            layers.append(layer)

        # Decoder with no token embeddings
        self.decoder = TransformerDecoder(
            tok_embeddings=nn.Identity(),  # No embedding layer needed
            layers=layers,
            max_seq_len=max_seq_len,
            num_heads=num_heads,
            head_dim=head_dim,
            norm=RMSNorm(embed_dim, eps=norm_eps),
            output=nn.Identity(),
        )

        # Cross-attention layers
        self.cross_attn_layers = nn.ModuleList()
        for _ in range(num_cross_layers):
            cross_attn = MultiHeadAttention(
                embed_dim=embed_dim,
                num_heads=num_heads,
                num_kv_heads=num_kv_heads,
                head_dim=head_dim,
                is_causal=False,
            )
            mlp = qwen2_mlp(dim=embed_dim, hidden_dim=hidden_dim)
            cross_layer = TransformerCrossAttentionLayer(
                attn=cross_attn,
                mlp=mlp,
                ca_norm=RMSNorm(embed_dim, eps=norm_eps),
                mlp_norm=RMSNorm(embed_dim, eps=norm_eps),
            )
            self.cross_attn_layers.append(cross_layer)

        self.output = nn.Linear(embed_dim, vocab_size, bias=False)
        self.to(dtype=dtype)

    def forward(self, byte_embeds, patch_embs, patch_ids):
        # Pass byte embeddings directly to the decoder
        x = self.decoder(byte_embeds, mask=None)  # Shape: [batch_size, seq_len, embed_dim]
        x = x.to(torch.bfloat16) # TODO: Fix cast because no chunking.

        # Apply cross-attention with patch embeddings
        if self.cross_attn_layers:
            num_patches = patch_embs.size(1)
            mask = (
                patch_ids.unsqueeze(2) == torch.arange(num_patches, device=byte_embeds.device).unsqueeze(0).unsqueeze(0)
            )  # Shape: [batch_size, num_patches, seq_len]
            for cross_layer in self.cross_attn_layers:
                x = cross_layer(x, encoder_input=patch_embs, encoder_mask=mask)

        # Compute logits
        logits = self.output(x)  # Shape: [batch_size, seq_len, vocab_size]
        return logits

class LocalEncoderWithPooling(nn.Module):
    def __init__(self, base_encoder, cross_attn_layers, embed_dim, global_dim):
        super().__init__()
        self.base_encoder = base_encoder
        self.cross_attn_layers = cross_attn_layers
        self.patch_projector = PatchToGlobalProjector(embed_dim, global_dim)

    def forward(self, bytes, patch_ids):
        # Get byte-level embeddings from the base encoder
        byte_embeds = self.base_encoder(bytes, mask=None)
        byte_embeds = byte_embeds.to(torch.bfloat16)

        # Apply cross-attention if layers exist
        if self.cross_attn_layers:
            # num_patches = patch_ids.max().item() + 1
            local_encoder_mask = (patch_ids.unsqueeze(1) == patch_ids.unsqueeze(2))  # Shape: [batch_size, seq_len, seq_len]
            patch_embs = byte_embeds  # Start with byte embeddings
            for cross_layer in self.cross_attn_layers:
                patch_embs = cross_layer(patch_embs, encoder_input=byte_embeds, encoder_mask=local_encoder_mask)
        else:
            patch_embs = byte_embeds

        # Reduce byte-level embeddings to patch-level embeddings
        patch_embs = patch_reduce(patch_embs, patch_ids, reduce_op="mean")  # Shape: [batch_size, num_patches, embed_dim]

        # Project to global dimension
        patch_embs = self.patch_projector(patch_embs)  # Shape: [batch_size, num_patches, global_dim]

        return byte_embeds, patch_embs  # Return both byte embeddings and reduced patch embeddings

def build_local_encoder(
    global_dim: int,
    vocab_size: int = VOCAB_SIZE,
    embed_dim: int = 2048,
    num_heads: int = 8,
    num_kv_heads: int = 8,
    hidden_dim: int = 4096,
    norm_eps: float = 1e-5,
    attn_dropout: float = 0.0,
    max_seq_len: int = 2048,
    num_layers: int = 4,
    num_cross_layers = 4,
    dtype=torch.bfloat16,
    use_hash_ngrams=True,
    max_ngram: int = 8,
    num_ngram_buckets: int = 500000,
):
    head_dim = embed_dim // num_heads

    if use_hash_ngrams:
        tok_embeddings = HashNGramEmbedder(
            embed_dim=embed_dim,
            max_n=max_ngram,
            num_buckets=num_ngram_buckets,
            vocab_size=vocab_size
        )
    else:
        tok_embeddings = nn.Embedding(vocab_size, embed_dim)

    # Build self-attention layers with Qwen MLP
    layers = nn.ModuleList()
    for _ in range(num_layers):
        # TODO: KV cache?
        self_attn = MultiHeadAttention(
            embed_dim=embed_dim,
            num_heads=num_heads,
            num_kv_heads=num_kv_heads,
            head_dim=head_dim,
            q_proj=nn.Linear(embed_dim, num_heads * head_dim, bias=True),
            k_proj=nn.Linear(embed_dim, num_kv_heads * head_dim, bias=True),
            v_proj=nn.Linear(embed_dim, num_kv_heads * head_dim, bias=True),
            output_proj=nn.Linear(embed_dim, embed_dim, bias=False),
            max_seq_len=max_seq_len,
            attn_dropout=attn_dropout,
            is_causal=True,
        )
        mlp = qwen2_mlp(dim=embed_dim, hidden_dim=hidden_dim)
        layer = TransformerSelfAttentionLayer(
            attn=self_attn,
            mlp=mlp,
            sa_norm=RMSNorm(embed_dim, eps=norm_eps),
            mlp_norm=RMSNorm(embed_dim, eps=norm_eps),
        )
        layers.append(layer)

    base_encoder = TransformerDecoder(
        tok_embeddings=tok_embeddings,
        layers=layers,
        max_seq_len=max_seq_len,
        num_heads=num_heads,
        head_dim=head_dim,
        norm=RMSNorm(embed_dim, eps=norm_eps),
        output=nn.Identity(),  # no final projection
    )

    # Cross-attention layers (optional)
    cross_attn_layers = nn.ModuleList()
    for _ in range(num_cross_layers):
        cross_attn = MultiHeadAttention(
            embed_dim=global_dim,
            num_heads=num_heads,
            num_kv_heads=num_kv_heads,
            head_dim=head_dim,
            q_proj=nn.Linear(global_dim, num_heads * head_dim, bias=True),
            k_proj=nn.Linear(global_dim, num_kv_heads * head_dim, bias=True),
            v_proj=nn.Linear(global_dim, num_kv_heads * head_dim, bias=True),
            output_proj=nn.Linear(global_dim, global_dim, bias=False),
            max_seq_len=max_seq_len,
            attn_dropout=attn_dropout,
            is_causal=False,
        )
        mlp = qwen2_mlp(dim=embed_dim, hidden_dim=hidden_dim)
        cross_layer = TransformerCrossAttentionLayer(
            attn=cross_attn,
            mlp=mlp,
            ca_norm=RMSNorm(global_dim, eps=norm_eps),
            mlp_norm=RMSNorm(global_dim, eps=norm_eps),
        )
        cross_attn_layers.append(cross_layer)

    local_encoder = LocalEncoderWithPooling(base_encoder, cross_attn_layers, embed_dim, global_dim)
    return local_encoder.to(dtype=dtype)

################################################
# dynamic patching
################################################

def compute_local_entropy(bytes_tensor, window_size=8):
    """Return a per-token "entropy" measure to guide patching

    Arguments:
        bytes_tensor: Torch.tensor[batch_size, seq_len] byttes to calc entropy on
        window_size: int size to window across

        local_entropy: torch.Tensor[batch_size, seq_len]
    """
    # bytes_tensor:
    device = bytes_tensor.device
    batch_size, seq_len = bytes_tensor.shape

    # We’ll keep a sliding frequency table. Initialize all zeros:
    freq = torch.zeros(batch_size, VOCAB_SIZE, device=device)
    local_entropy = torch.zeros(batch_size, seq_len, device=device)

    for pos in range(seq_len):
        # add current byte
        current_byte = bytes_tensor[:, pos]
        freq[torch.arange(batch_size), current_byte] += 1

        # compute distribution
        dist = freq / freq.sum(dim=1, keepdim=True).clamp_min(1e-8)
        # compute -p*log2(p)
        ent = -(dist * (dist + 1e-8).log2()).sum(dim=1)
        local_entropy[:, pos] = ent

        # remove oldest byte if we exceed window size
        if pos >= window_size:
            oldest_byte = bytes_tensor[:, pos - window_size]
            freq[torch.arange(batch_size), oldest_byte] -= 1
    return local_entropy

def dynamic_patch(
    bytes_tensor: torch.Tensor,
    threshold: float = 3.0,   # starting entropy threshold in bits
    min_threshold: float = 2.0,    # lower bound
    max_threshold: float = 5.0,    # upper bound
    threshold_step_down: float = 0.1,  # how much to decrease threshold if no patches triggered
    threshold_step_up: float = 0.1,    # how much to increase threshold if we trigger a patch
    patch_size: int = 4,
    window_size: int = 8
):
    """
    A dynamic patching approach that adjusts the entropy threshold
    upward/downward depending on whether patches are being triggered too often or not enough.

    Args:
        bytes_tensor: [batch_size, seq_len]
        threshold: initial bits threshold for local entropy
        min_threshold, max_threshold: clamp thresholds
        threshold_step_down, threshold_step_up: step sizes
        patch_size: max patch length if we haven't triggered a boundary earlier
        window_size: for computing local entropy

    Returns:
        patch_ids: [batch_size, seq_len] with patch ID for each token
        local_ent: [batch_size, seq_len] local entropy in bits
    """
    local_ent = compute_local_entropy(bytes_tensor, window_size=window_size)
    batch_size, seq_len = bytes_tensor.shape

    # Each row’s patch assignment
    patch_ids = torch.zeros(batch_size, seq_len, dtype=torch.long, device=bytes_tensor.device)

    current_patch = torch.zeros(batch_size, dtype=torch.long, device=bytes_tensor.device)
    patch_lengths = torch.zeros(batch_size, dtype=torch.long, device=bytes_tensor.device)

    # Keep track of how many consecutive tokens we have processed w/o triggering any new patch
    # so we can adjust threshold if we go too long
    consecutive_no_trigger = 0

    for pos in range(seq_len):
        patch_lengths += 1

        # Which batch elements exceed threshold or hit patch size
        trigger_new_patch = (local_ent[:, pos] > threshold) | (patch_lengths >= patch_size)
        triggered_rows = trigger_new_patch.nonzero(as_tuple=False).flatten()

        if len(triggered_rows) > 0:
            # For each row that triggered, increment patch and reset patch_lengths
            current_patch[triggered_rows] += 1
            patch_lengths[triggered_rows] = 0

            # Because at least 1 row triggered a patch, we can optionally adjust threshold upward
            # or downward. For example, *raising* threshold if we keep splitting too often:
            threshold = min(threshold + threshold_step_up, max_threshold)

            consecutive_no_trigger = 0
        else:
            # No new patch was triggered
            consecutive_no_trigger += 1
            # If we haven't triggered for a while, lower threshold so that we become more likely
            # to split in the future.
            threshold = max(threshold - threshold_step_down, min_threshold)

        patch_ids[:, pos] = current_patch

    return patch_ids, local_ent

def patch_reduce(h, patch_ids, reduce_op="mean"):
    """
    Arguments:
        h: [batch_size, seq_len, emb_dim]
        patch_ids: [batch_size, seq_len]
        reduce_op: e.g. "mean", "amin", "amax"

    returns: [batch_size, num_patches, emb_dim]
    """
    batch_size, seq_len, emb_dim = h.shape
    num_patches = patch_ids.max().item() + 1  # patch IDs go from 0..max

    # expand dims so we can scatter:
    expanded_ids = patch_ids.unsqueeze(-1).expand(-1, -1, emb_dim)
    reduced = torch.zeros(batch_size, num_patches, emb_dim, device=h.device, dtype=h.dtype)
    reduced = reduced.scatter_reduce(
        dim=1,
        index=expanded_ids,
        src=h,
        reduce=reduce_op,
        include_self=False,
    )
    return reduced

def compute_patch_size(so_far: torch.Tensor, threshold=3.0, max_patch=8):
    """
    heuristic function for deciding the patch length
    based on dynamic_patch logic or local entropy.

    so_far: [seq_len] or [1, seq_len], the current context of tokens (including newly generated ones).
    threshold: approximate bits threshold for deciding to break the patch.
    max_patch: a max patch length to avoid overly large chunks.

    Returns:
        predicted_patch_size: int
            the number of bytes to decode in the *next* patch in a single forward pass
    """
    # re-run dynamic_patch() on the entire sequence and see how big the last patch is.
    # this is probably pretty wasteful!s
    # Then we decide how big the *next* patch would be if we continued.
    # This is a simple way to reuse your dynamic_patch code.
    if so_far.dim() == 1:
        so_far = so_far.unsqueeze(0)  # [batch=1, seq_len]

    patch_ids, _ = dynamic_patch(so_far, threshold=threshold, patch_size=max_patch)

    # The ID of the last patch in that sequence:
    last_patch_id = patch_ids[0, -1].item()  # e.g. 3 means patches 0..3
    # Count how many tokens so far belong to the last patch
    # (i.e. sum of patch_ids == last_patch_id)
    count_last_patch = (patch_ids[0] == last_patch_id).sum().item()

    # guess that the next patch might be similar in size:
    # If we already used up to 'count_last_patch' tokens for the last patch,
    # we can try the same or smaller for the next patch. A simple approach is:
    predicted_size = max(1, min(count_last_patch, max_patch))

    return predicted_size

################################################
# Projection layer
################################################
class PatchToGlobalProjector(nn.Module):
    def __init__(self, local_dim, global_dim):
        super().__init__()
        self.proj = nn.Linear(local_dim, global_dim)
    def forward(self, x):
        return self.proj(x)


################################################
# ByteLatentQwen2p5Decoder with cross-attn
################################################
class ByteLatentQwen2p5Decoder(TransformerDecoder):
    def __init__(
        self,
        qwen_cfg: Dict[str, Any],
        local_encoder_cfg: Dict[str, Any],
        patch_size: int = 4,
        patching_threshold: float = 3.0,
        freeze_global_for_n_steps: int = 0,
    ):
        layers = nn.ModuleList()
        head_dim = qwen_cfg['embed_dim'] // qwen_cfg['num_heads']

        for _ in range(qwen_cfg['num_layers']):
            layer = TransformerSelfAttentionLayer(
                attn=self_attn,
                mlp=mlp,
                sa_norm=RMSNorm(dim=qwen_cfg['embed_dim'], eps=qwen_cfg['norm_eps']),
                mlp_norm=RMSNorm(dim=qwen_cfg['embed_dim'], eps=qwen_cfg['norm_eps']),
            )
            layers.append(layer)

        output = nn.Linear(
            qwen_cfg['embed_dim'],
            VOCAB_SIZE  ,  # bytes
            bias=False,
        )

        output = nn.Identity()  # Global transformer doesn’t output logits
        emb = nn.Identity()
        super().__init__(
            tok_embeddings=emb,
            layers=layers,
            max_seq_len=qwen_cfg['max_seq_len'],
            num_heads=qwen_cfg['num_heads'],
            head_dim=head_dim,
            norm=RMSNorm(qwen_cfg['embed_dim'], eps=qwen_cfg['norm_eps']),
            output=output,
        )
        self.local_encoder = build_local_encoder(**local_encoder_cfg, global_dim=qwen_cfg['embed_dim'])
        self.local_decoder = LocalDecoder(
            embed_dim=qwen_cfg['embed_dim'],
            global_dim=qwen_cfg['embed_dim'],
            vocab_size=VOCAB_SIZE,
            max_seq_len=local_encoder_cfg['max_seq_len'],
        )

         # Collect parameters for different learning rates
        self.qwen_params = []
        self.new_params = []

        # Qwen parts
        self.qwen_params.extend(self.norm.parameters())
        for layer in self.layers:
            if isinstance(layer, TransformerSelfAttentionLayer):
                self.qwen_params.extend(layer.parameters())

        # New parts
        self.new_params.extend(self.tok_embeddings.parameters())
        self.new_params.extend(self.output.parameters())
        self.new_params.extend(self.local_encoder.parameters())
        self.new_params.extend(self.local_decoder.parameters())

        self.patch_size = patch_size
        self.patching_threshold = patching_threshold
        self.freeze_global_for_n_steps = freeze_global_for_n_steps
        self.current_step = 0
        self.global_frozen = freeze_global_for_n_steps > 0
        if self.global_frozen:
            self._update_freezing()

        # We'll store how many chunks the user wants for final output
        self.num_output_chunks = 0  # default

    def _update_freezing(self):
        if self.global_frozen:
            for param in self.qwen_params:
                param.requires_grad = False
        else:
            for param in self.parameters():
                param.requires_grad = True

    def set_num_output_chunks(self, num_output_chunks: int) -> None:
        super().set_num_output_chunks(num_output_chunks)
        self.num_output_chunks = num_output_chunks

    def forward(
        self,
        tokens: torch.Tensor,
        *,
        mask: Optional[Union[torch.Tensor, float]] = None,
        encoder_input: Optional[torch.Tensor] = None,
        encoder_mask: Optional[torch.Tensor] = None,
        input_pos: Optional[torch.Tensor] = None,
        patch_ids: Optional[torch.Tensor] = None,
    ) -> Union[torch.Tensor, List[torch.Tensor]]:
        # Update freezing state if in training
        if self.training and self.global_frozen:
            self.current_step += 1
            if self.current_step >= self.freeze_global_for_n_steps:
                self.global_frozen = False
                self._update_freezing()

        if patch_ids is None:
            patch_ids, _ = dynamic_patch(tokens, threshold=self.patching_threshold, patch_size=self.patch_size)

        byte_embeds, patch_embs = self.local_encoder(tokens, patch_ids=patch_ids)
        global_out = super().forward(patch_embs, mask=mask, input_pos=input_pos)

        # Assuming the outs are chunked, take entry[0] of outputs.
        if self.num_output_chunks == 0:
            global_out = global_out.to(torch.bfloat16) # TODO: Another fix for the 0 chunk float. Need to move this dtype definiton.
            logits = self.local_decoder(byte_embeds, global_out, patch_ids)
        else:
            global_out_combined = torch.cat(global_out, dim=1) # Unchunking - it would be better not to pass this through I think.
            clogits = self.local_decoder(byte_embeds, global_out_combined, patch_ids)
            logits = [chunk for chunk in clogits.chunk(self.num_output_chunks, dim=1)]
        return logits

    def unified_generate(
        self,
        prompt: Union[torch.LongTensor, List[int]],
        max_new_tokens: int = 128,
        eos_id: int = EOS_ID,
        temperature: float = 0.7,
        top_k: int = 50,
        top_p: float = 0,
        frequency_penalty: float = 0.1,
        repetition_penalty: float = 1.5,
        greedy: bool = False,
    ) -> torch.Tensor:
        # 1) Convert prompt to a tensor [1, seq_len]
        if isinstance(prompt, list):
            prompt = torch.tensor(prompt, dtype=torch.long, device=next(self.parameters()).device)
        if prompt.dim() == 1:
            prompt = prompt.unsqueeze(0) # Add a batch dimension.

        device = prompt.device
        all_tokens = prompt.clone()

        # Track usage for freq / repetition penalty
        token_counts = torch.bincount(all_tokens[0], minlength=VOCAB_SIZE).to(device)
        recent_tokens = []

        # Start generating
        for step_i in range(max_new_tokens):

            # No patching => standard forward
            with torch.no_grad():
                logits = self.forward(all_tokens)

            if isinstance(logits, list):
                logits = torch.cat(logits, dim=1)
            step_logits = logits[0, -1, :]

            # Apply temperature
            if temperature != 1.0:
                step_logits = step_logits / temperature

            # Repetition penalty
            for tk in recent_tokens:
                step_logits[tk] /= repetition_penalty

            # Frequency penalty
            step_logits -= token_counts * frequency_penalty

            # top-k
            if top_k > 0:
                vals, idxs = torch.topk(step_logits, top_k)
                mask = torch.ones_like(step_logits, dtype=torch.bool)
                mask[idxs] = False
                step_logits[mask] = float('-inf')

            # top-p
            probs = torch.softmax(step_logits, dim=-1)
            if top_p > 0:
                sorted_logits, sorted_indices = torch.sort(step_logits, descending=True)
                cumulative_probs = torch.cumsum(probs, dim=-1)
                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[:, 1:] = sorted_indices_to_remove[:, :-1].clone()
                sorted_indices_to_remove[:, 0] = 0
                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                step_logits = step_logits.scatter(-1, indices_to_remove, -1e9)

            # Re‐normalize
            total_p = probs.sum()
            if total_p > 0:
                probs /= total_p
            else:
                probs = torch.ones_like(probs) / probs.size(-1)

            # Next token
            if greedy:
                next_token = torch.argmax(probs).item()
            else:
                next_token = torch.multinomial(probs, 1).item()

            # Add new token
            all_tokens = torch.cat(
                [all_tokens, torch.tensor([[next_token]], device=device)],
                dim=1,
            )

            # Update counters
            token_counts[next_token] += 1
            recent_tokens.append(next_token)
            if len(recent_tokens) > 5:
                recent_tokens.pop(0)

            # EOS break
            if next_token == eos_id:
                break
        return all_tokens



In [ ]:
# @title ttblt/bltqwen.py
# https://github.com/ianbarber/ttblt/blob/main/ttblt/bltqwen.py
import torch
from torch import nn
from typing import Any, Dict, List, Mapping, Optional, Tuple, Union
from torchtune.modules.transformer import TransformerDecoder
from torchtune.modules import (MultiHeadAttention, RMSNorm, TransformerCrossAttentionLayer, TransformerSelfAttentionLayer,)
from torchtune.modules.model_fusion import FusionLayer
from torchtune.models.qwen2._component_builders import qwen2_mlp

PAD_ID = 256
BOS_ID = 257
EOS_ID = 258
NUM_SPECIAL_TOKENS = 3
VOCAB_SIZE = 256 + NUM_SPECIAL_TOKENS

################################################
# Local encoder/decoder (with cross-attn)
################################################

class HashNGramEmbedder(nn.Module):
    """
    Wraps a main byte embedding plus additional hash-based
    n-gram embedding lookups. Two modes are supported:

    1. Separate tables mode (default): One embedding table per n in [3, max_n].
    2. Shared table mode (if shared_table=True): A single embedding table is used
       for all n-gram sizes.

    When using the shared table mode, a learned n-gram size embedding is added so that the model can
    distinguish among n-gram lengths.

    The final embedding at each position is:
       main_embed(byte) + sum_{n in 3..max_n} (ngram_embed [+ ngram_size_embed if shared])
    """
    def __init__(
        self,
        embed_dim: int = 2048,
        max_n: int = 8,
        num_buckets: int = 500_000,
        vocab_size: int = VOCAB_SIZE,
        hash_base: int = 0,
        hash_mod: int = 2**23,
        shared_table: bool = True  # switchable mode
    ):
        super().__init__()
        self.embed_dim = embed_dim
        self.max_n = max_n
        self.num_buckets = num_buckets
        self.hash_base = hash_base
        if (self.hash_base == 0):
            self.hash_base = vocab_size + 1
        self.hash_mod = hash_mod
        self.shared_table = shared_table

        # Main byte embedding.
        self.main_embed = nn.Embedding(vocab_size, embed_dim)

        if shared_table:
            # One shared table for all n-gram sizes.
            self.shared_ngram_table = nn.Embedding(num_buckets, embed_dim)
            nn.init.normal_(self.shared_ngram_table.weight, mean=0.0, std=0.02)
            # n-gram size embedding: one learned vector per n in [3, max_n]
            # (max_n - 2) distinct n values.
            self.ngram_size_embed = nn.Embedding(max_n - 2, embed_dim)
            nn.init.normal_(self.ngram_size_embed.weight, mean=0.0, std=0.02)
        else:
            # Separate table per n-gram size.
            self.ngram_tables = nn.ModuleDict()
            for n in range(3, max_n + 1):
                table = nn.Embedding(num_buckets, embed_dim)
                nn.init.normal_(table.weight, mean=0.0, std=0.02)
                self.ngram_tables[str(n)] = table

        # Precompute powers for rolling hash for each n in [3, max_n] using modular exponentiation.
        for n in range(3, max_n + 1):
            powers_list = [pow(hash_base, n - 1 - k, hash_mod) for k in range(n)]
            powers = torch.tensor(powers_list, dtype=torch.int32)
            self.register_buffer(f'powers_{n}', powers)

    def forward(self, tokens: torch.LongTensor) -> torch.Tensor:
        """
        tokens: [batch_size, seq_len] of byte IDs in [0..255].
        Returns final embeddings of shape [batch_size, seq_len, embed_dim].
        All hash arithmetic is done in int32.
        """
        bsz, seq_len = tokens.shape
        if seq_len == 0:
            return torch.empty(bsz, 0, self.embed_dim, device=tokens.device)

        # Main byte embedding.
        out = self.main_embed(tokens)  # [bsz, seq_len, embed_dim]

        # For each n-gram size, compute and add n-gram embeddings.
        for n in range(3, self.max_n + 1):
            if seq_len < n:
                continue

            powers = getattr(self, f'powers_{n}')  # shape: [n]
            ngrams = tokens.unfold(1, n, 1).to(torch.int32)  # [bsz, seq_len - n + 1, n]
            if ngrams.numel() == 0:
                continue

            temp = (ngrams * powers.unsqueeze(0).unsqueeze(0)) % self.hash_mod
            hashed_vals = temp.sum(dim=2) % self.hash_mod  # [bsz, seq_len - n + 1]

            hashed_idxs = torch.zeros((bsz, seq_len), dtype=torch.int32, device=tokens.device)
            hashed_idxs[:, n - 1:] = hashed_vals
            hashed_idxs = hashed_idxs % self.num_buckets

            if self.shared_table:
                ngram_embed = self.shared_ngram_table(hashed_idxs.long())  # [bsz, seq_len, embed_dim]
                # Add n-gram size embedding only in shared mode.
                size_embed = self.ngram_size_embed(torch.tensor(n - 3, device=tokens.device))
                size_embed = size_embed.unsqueeze(0).unsqueeze(0)  # [1, 1, embed_dim]
                ngram_embed = ngram_embed + size_embed
            else:
                table = self.ngram_tables[str(n)]
                ngram_embed = table(hashed_idxs.long())

            out += ngram_embed

        num_contrib = 1 + (self.max_n - 2)  # main embedding + contributions for each n from 3 to max_n
        out = out / num_contrib
        return out

class LocalDecoder(nn.Module):
    def __init__(
        self,
        embed_dim: int,
        global_dim: int,
        vocab_size: int,
        num_layers: int = 8,
        num_cross_layers: int = 4,
        num_heads: int = 8,
        num_kv_heads: int = 8,
        hidden_dim: int = 4096,
        norm_eps: float = 1e-5,
        attn_dropout: float = 0.0,
        max_seq_len: int = 4096,
        dtype=torch.bfloat16,
    ):
        super().__init__()
        head_dim = embed_dim // num_heads

        # Self-attention layers
        layers = nn.ModuleList()
        for _ in range(num_layers):
            self_attn = MultiHeadAttention(
                embed_dim=embed_dim,
                num_heads=num_heads,
                num_kv_heads=num_kv_heads,
                head_dim=head_dim,
                q_proj=nn.Linear(embed_dim, num_heads * head_dim, bias=True),
                k_proj=nn.Linear(embed_dim, num_kv_heads * head_dim, bias=True),
                v_proj=nn.Linear(embed_dim, num_kv_heads * head_dim, bias=True),
                output_proj=nn.Linear(embed_dim, embed_dim, bias=False),
                max_seq_len=max_seq_len,
                attn_dropout=attn_dropout,
                is_causal=True,
            )
            mlp = qwen2_mlp(dim=embed_dim, hidden_dim=hidden_dim)
            layer = TransformerSelfAttentionLayer(
                attn=self_attn,
                mlp=mlp,
                sa_norm=RMSNorm(embed_dim, eps=norm_eps),
                mlp_norm=RMSNorm(embed_dim, eps=norm_eps),
            )
            layers.append(layer)

        # Decoder with no token embeddings
        self.decoder = TransformerDecoder(
            tok_embeddings=nn.Identity(),  # No embedding layer needed
            layers=layers,
            max_seq_len=max_seq_len,
            num_heads=num_heads,
            head_dim=head_dim,
            norm=RMSNorm(embed_dim, eps=norm_eps),
            output=nn.Identity(),
        )

        # Cross-attention layers
        self.cross_attn_layers = nn.ModuleList()
        for _ in range(num_cross_layers):
            cross_attn = MultiHeadAttention(
                embed_dim=embed_dim,
                num_heads=num_heads,
                num_kv_heads=num_kv_heads,
                head_dim=head_dim,
                q_proj=nn.Linear(embed_dim, num_heads * head_dim, bias=True),
                k_proj=nn.Linear(global_dim, num_kv_heads * head_dim, bias=True),
                v_proj=nn.Linear(global_dim, num_kv_heads * head_dim, bias=True),
                output_proj=nn.Linear(embed_dim, embed_dim, bias=False),
                max_seq_len=max_seq_len,
                attn_dropout=attn_dropout,
                is_causal=False,
            )
            mlp = qwen2_mlp(dim=embed_dim, hidden_dim=hidden_dim)
            cross_layer = TransformerCrossAttentionLayer(
                attn=cross_attn,
                mlp=mlp,
                ca_norm=RMSNorm(embed_dim, eps=norm_eps),
                mlp_norm=RMSNorm(embed_dim, eps=norm_eps),
            )
            self.cross_attn_layers.append(cross_layer)

        self.output = nn.Linear(embed_dim, vocab_size, bias=False)
        self.to(dtype=dtype)

    def forward(self, byte_embeds, patch_embs, patch_ids):
        # Pass byte embeddings directly to the decoder
        x = self.decoder(byte_embeds, mask=None)  # Shape: [batch_size, seq_len, embed_dim]
        x = x.to(torch.bfloat16) # TODO: Fix cast because no chunking.

        # Apply cross-attention with patch embeddings
        if self.cross_attn_layers:
            num_patches = patch_embs.size(1)
            mask = (
                patch_ids.unsqueeze(2) == torch.arange(num_patches, device=byte_embeds.device).unsqueeze(0).unsqueeze(0)
            )  # Shape: [batch_size, num_patches, seq_len]
            for cross_layer in self.cross_attn_layers:
                x = cross_layer(x, encoder_input=patch_embs, encoder_mask=mask)

        # Compute logits
        logits = self.output(x)  # Shape: [batch_size, seq_len, vocab_size]
        return logits

class LocalEncoderWithPooling(nn.Module):
    def __init__(self, base_encoder, cross_attn_layers, embed_dim, global_dim):
        super().__init__()
        self.base_encoder = base_encoder
        self.cross_attn_layers = cross_attn_layers
        self.patch_projector = PatchToGlobalProjector(embed_dim, global_dim)

    def forward(self, bytes, patch_ids):
        # Get byte-level embeddings from the base encoder
        byte_embeds = self.base_encoder(bytes, mask=None)
        byte_embeds = byte_embeds.to(torch.bfloat16)

        # Apply cross-attention if layers exist
        if self.cross_attn_layers:
            # num_patches = patch_ids.max().item() + 1
            local_encoder_mask = (patch_ids.unsqueeze(1) == patch_ids.unsqueeze(2))  # Shape: [batch_size, seq_len, seq_len]
            patch_embs = byte_embeds  # Start with byte embeddings
            for cross_layer in self.cross_attn_layers:
                patch_embs = cross_layer(patch_embs, encoder_input=byte_embeds, encoder_mask=local_encoder_mask)
        else:
            patch_embs = byte_embeds

        # Reduce byte-level embeddings to patch-level embeddings
        patch_embs = patch_reduce(patch_embs, patch_ids, reduce_op="mean")  # Shape: [batch_size, num_patches, embed_dim]

        # Project to global dimension
        patch_embs = self.patch_projector(patch_embs)  # Shape: [batch_size, num_patches, global_dim]

        return byte_embeds, patch_embs  # Return both byte embeddings and reduced patch embeddings

def build_local_encoder(
    global_dim: int,
    vocab_size: int = VOCAB_SIZE,
    embed_dim: int = 2048,
    num_heads: int = 8,
    num_kv_heads: int = 8,
    hidden_dim: int = 4096,
    norm_eps: float = 1e-5,
    attn_dropout: float = 0.0,
    max_seq_len: int = 2048,
    num_layers: int = 4,
    num_cross_layers = 4,
    dtype=torch.bfloat16,
    use_hash_ngrams=True,
    max_ngram: int = 8,
    num_ngram_buckets: int = 500000,
):
    head_dim = embed_dim // num_heads

    if use_hash_ngrams:
        tok_embeddings = HashNGramEmbedder(
            embed_dim=embed_dim,
            max_n=max_ngram,
            num_buckets=num_ngram_buckets,
            vocab_size=vocab_size
        )
    else:
        tok_embeddings = nn.Embedding(vocab_size, embed_dim)

    # Build self-attention layers with Qwen MLP
    layers = nn.ModuleList()
    for _ in range(num_layers):
        # TODO: KV cache?
        self_attn = MultiHeadAttention(
            embed_dim=embed_dim,
            num_heads=num_heads,
            num_kv_heads=num_kv_heads,
            head_dim=head_dim,
            q_proj=nn.Linear(embed_dim, num_heads * head_dim, bias=True),
            k_proj=nn.Linear(embed_dim, num_kv_heads * head_dim, bias=True),
            v_proj=nn.Linear(embed_dim, num_kv_heads * head_dim, bias=True),
            output_proj=nn.Linear(embed_dim, embed_dim, bias=False),
            max_seq_len=max_seq_len,
            attn_dropout=attn_dropout,
            is_causal=True,
        )
        mlp = qwen2_mlp(dim=embed_dim, hidden_dim=hidden_dim)
        layer = TransformerSelfAttentionLayer(
            attn=self_attn,
            mlp=mlp,
            sa_norm=RMSNorm(embed_dim, eps=norm_eps),
            mlp_norm=RMSNorm(embed_dim, eps=norm_eps),
        )
        layers.append(layer)

    base_encoder = TransformerDecoder(
        tok_embeddings=tok_embeddings,
        layers=layers,
        max_seq_len=max_seq_len,
        num_heads=num_heads,
        head_dim=head_dim,
        norm=RMSNorm(embed_dim, eps=norm_eps),
        output=nn.Identity(),  # no final projection
    )

    # Cross-attention layers (optional)
    cross_attn_layers = nn.ModuleList()
    for _ in range(num_cross_layers):
        cross_attn = MultiHeadAttention(
            embed_dim=global_dim,
            num_heads=num_heads,
            num_kv_heads=num_kv_heads,
            head_dim=head_dim,
            q_proj=nn.Linear(global_dim, num_heads * head_dim, bias=True),
            k_proj=nn.Linear(global_dim, num_kv_heads * head_dim, bias=True),
            v_proj=nn.Linear(global_dim, num_kv_heads * head_dim, bias=True),
            output_proj=nn.Linear(global_dim, global_dim, bias=False),
            max_seq_len=max_seq_len,
            attn_dropout=attn_dropout,
            is_causal=False,
        )
        mlp = qwen2_mlp(dim=embed_dim, hidden_dim=hidden_dim)
        cross_layer = TransformerCrossAttentionLayer(
            attn=cross_attn,
            mlp=mlp,
            ca_norm=RMSNorm(global_dim, eps=norm_eps),
            mlp_norm=RMSNorm(global_dim, eps=norm_eps),
        )
        cross_attn_layers.append(cross_layer)

    local_encoder = LocalEncoderWithPooling(base_encoder, cross_attn_layers, embed_dim, global_dim)
    return local_encoder.to(dtype=dtype)

################################################
# dynamic patching
################################################

def compute_local_entropy(bytes_tensor, window_size=8):
    """Return a per-token "entropy" measure to guide patching

    Arguments:
        bytes_tensor: Torch.tensor[batch_size, seq_len] byttes to calc entropy on
        window_size: int size to window across

        local_entropy: torch.Tensor[batch_size, seq_len]
    """
    # bytes_tensor:
    device = bytes_tensor.device
    batch_size, seq_len = bytes_tensor.shape

    # We’ll keep a sliding frequency table. Initialize all zeros:
    freq = torch.zeros(batch_size, VOCAB_SIZE, device=device)
    local_entropy = torch.zeros(batch_size, seq_len, device=device)

    for pos in range(seq_len):
        # add current byte
        current_byte = bytes_tensor[:, pos]
        freq[torch.arange(batch_size), current_byte] += 1

        # compute distribution
        dist = freq / freq.sum(dim=1, keepdim=True).clamp_min(1e-8)
        # compute -p*log2(p)
        ent = -(dist * (dist + 1e-8).log2()).sum(dim=1)
        local_entropy[:, pos] = ent

        # remove oldest byte if we exceed window size
        if pos >= window_size:
            oldest_byte = bytes_tensor[:, pos - window_size]
            freq[torch.arange(batch_size), oldest_byte] -= 1
    return local_entropy

def dynamic_patch(
    bytes_tensor: torch.Tensor,
    threshold: float = 3.0,   # starting entropy threshold in bits
    min_threshold: float = 2.0,    # lower bound
    max_threshold: float = 5.0,    # upper bound
    threshold_step_down: float = 0.1,  # how much to decrease threshold if no patches triggered
    threshold_step_up: float = 0.1,    # how much to increase threshold if we trigger a patch
    patch_size: int = 4,
    window_size: int = 8
):
    """
    A dynamic patching approach that adjusts the entropy threshold
    upward/downward depending on whether patches are being triggered too often or not enough.

    Args:
        bytes_tensor: [batch_size, seq_len]
        threshold: initial bits threshold for local entropy
        min_threshold, max_threshold: clamp thresholds
        threshold_step_down, threshold_step_up: step sizes
        patch_size: max patch length if we haven't triggered a boundary earlier
        window_size: for computing local entropy

    Returns:
        patch_ids: [batch_size, seq_len] with patch ID for each token
        local_ent: [batch_size, seq_len] local entropy in bits
    """
    local_ent = compute_local_entropy(bytes_tensor, window_size=window_size)
    batch_size, seq_len = bytes_tensor.shape

    # Each row’s patch assignment
    patch_ids = torch.zeros(batch_size, seq_len, dtype=torch.long, device=bytes_tensor.device)

    current_patch = torch.zeros(batch_size, dtype=torch.long, device=bytes_tensor.device)
    patch_lengths = torch.zeros(batch_size, dtype=torch.long, device=bytes_tensor.device)

    # Keep track of how many consecutive tokens we have processed w/o triggering any new patch
    # so we can adjust threshold if we go too long
    consecutive_no_trigger = 0

    for pos in range(seq_len):
        patch_lengths += 1

        # Which batch elements exceed threshold or hit patch size
        trigger_new_patch = (local_ent[:, pos] > threshold) | (patch_lengths >= patch_size)
        triggered_rows = trigger_new_patch.nonzero(as_tuple=False).flatten()

        if len(triggered_rows) > 0:
            # For each row that triggered, increment patch and reset patch_lengths
            current_patch[triggered_rows] += 1
            patch_lengths[triggered_rows] = 0

            # Because at least 1 row triggered a patch, we can optionally adjust threshold upward
            # or downward. For example, *raising* threshold if we keep splitting too often:
            threshold = min(threshold + threshold_step_up, max_threshold)

            consecutive_no_trigger = 0
        else:
            # No new patch was triggered
            consecutive_no_trigger += 1
            # If we haven't triggered for a while, lower threshold so that we become more likely
            # to split in the future.
            threshold = max(threshold - threshold_step_down, min_threshold)

        patch_ids[:, pos] = current_patch

    return patch_ids, local_ent

def patch_reduce(h, patch_ids, reduce_op="mean"):
    """
    Arguments:
        h: [batch_size, seq_len, emb_dim]
        patch_ids: [batch_size, seq_len]
        reduce_op: e.g. "mean", "amin", "amax"

    returns: [batch_size, num_patches, emb_dim]
    """
    batch_size, seq_len, emb_dim = h.shape
    num_patches = patch_ids.max().item() + 1  # patch IDs go from 0..max

    # expand dims so we can scatter:
    expanded_ids = patch_ids.unsqueeze(-1).expand(-1, -1, emb_dim)
    reduced = torch.zeros(batch_size, num_patches, emb_dim, device=h.device, dtype=h.dtype)
    reduced = reduced.scatter_reduce(
        dim=1,
        index=expanded_ids,
        src=h,
        reduce=reduce_op,
        include_self=False,
    )
    return reduced

def compute_patch_size(so_far: torch.Tensor, threshold=3.0, max_patch=8):
    """
    heuristic function for deciding the patch length
    based on dynamic_patch logic or local entropy.

    so_far: [seq_len] or [1, seq_len], the current context of tokens (including newly generated ones).
    threshold: approximate bits threshold for deciding to break the patch.
    max_patch: a max patch length to avoid overly large chunks.

    Returns:
        predicted_patch_size: int
            the number of bytes to decode in the *next* patch in a single forward pass
    """
    # re-run dynamic_patch() on the entire sequence and see how big the last patch is.
    # this is probably pretty wasteful!s
    # Then we decide how big the *next* patch would be if we continued.
    # This is a simple way to reuse your dynamic_patch code.
    if so_far.dim() == 1:
        so_far = so_far.unsqueeze(0)  # [batch=1, seq_len]

    patch_ids, _ = dynamic_patch(so_far, threshold=threshold, patch_size=max_patch)

    # The ID of the last patch in that sequence:
    last_patch_id = patch_ids[0, -1].item()  # e.g. 3 means patches 0..3
    # Count how many tokens so far belong to the last patch
    # (i.e. sum of patch_ids == last_patch_id)
    count_last_patch = (patch_ids[0] == last_patch_id).sum().item()

    # guess that the next patch might be similar in size:
    # If we already used up to 'count_last_patch' tokens for the last patch,
    # we can try the same or smaller for the next patch. A simple approach is:
    predicted_size = max(1, min(count_last_patch, max_patch))

    return predicted_size

################################################
# Projection layer
################################################
class PatchToGlobalProjector(nn.Module):
    def __init__(self, local_dim, global_dim):
        super().__init__()
        self.proj = nn.Linear(local_dim, global_dim)
    def forward(self, x):
        return self.proj(x)


################################################
# ByteLatentQwen2p5Decoder with cross-attn
################################################
class ByteLatentQwen2p5Decoder(TransformerDecoder):
    def __init__(
        self,
        qwen_cfg: Dict[str, Any],
        local_encoder_cfg: Dict[str, Any],
        patch_size: int = 4,
        patching_threshold: float = 3.0,
        freeze_global_for_n_steps: int = 0,
    ):
        layers = nn.ModuleList()
        head_dim = qwen_cfg['embed_dim'] // qwen_cfg['num_heads']

        for _ in range(qwen_cfg['num_layers']):
            self_attn = MultiHeadAttention(
                embed_dim=qwen_cfg['embed_dim'],
                num_heads=qwen_cfg['num_heads'],
                num_kv_heads=qwen_cfg['num_kv_heads'],
                head_dim=head_dim,
                q_proj=nn.Linear(qwen_cfg['embed_dim'], qwen_cfg['num_heads'] * head_dim, bias=True),
                k_proj=nn.Linear(qwen_cfg['embed_dim'], qwen_cfg['num_kv_heads'] * head_dim, bias=True),
                v_proj=nn.Linear(qwen_cfg['embed_dim'], qwen_cfg['num_kv_heads'] * head_dim, bias=True),
                output_proj=nn.Linear(qwen_cfg['embed_dim'], qwen_cfg['embed_dim'], bias=False),
                kv_cache=None,
                max_seq_len=qwen_cfg['max_seq_len'],
                attn_dropout=qwen_cfg['attn_dropout'],
            )
            mlp = qwen2_mlp(dim=qwen_cfg['embed_dim'], hidden_dim=qwen_cfg['intermediate_dim'])
            layer = TransformerSelfAttentionLayer(
                attn=self_attn,
                mlp=mlp,
                sa_norm=RMSNorm(dim=qwen_cfg['embed_dim'], eps=qwen_cfg['norm_eps']),
                mlp_norm=RMSNorm(dim=qwen_cfg['embed_dim'], eps=qwen_cfg['norm_eps']),
            )
            layers.append(layer)

        output = nn.Linear(
            qwen_cfg['embed_dim'],
            VOCAB_SIZE  ,  # bytes
            bias=False,
        )
        # Initialize output bytes.
        init_std = qwen_cfg['embed_dim'] ** -0.5
        with torch.no_grad():
            nn.init.trunc_normal_(
                output.weight,
                mean=0.0,
                std=init_std,
                a=-3*init_std,
                b=3*init_std,
            )

        output = nn.Identity()  # Global transformer doesn’t output logits
        emb = nn.Identity()
        super().__init__(
            tok_embeddings=emb,
            layers=layers,
            max_seq_len=qwen_cfg['max_seq_len'],
            num_heads=qwen_cfg['num_heads'],
            head_dim=head_dim,
            norm=RMSNorm(qwen_cfg['embed_dim'], eps=qwen_cfg['norm_eps']),
            output=output,
        )
        self.local_encoder = build_local_encoder(**local_encoder_cfg, global_dim=qwen_cfg['embed_dim'])
        self.local_decoder = LocalDecoder(
            embed_dim=qwen_cfg['embed_dim'],
            global_dim=qwen_cfg['embed_dim'],
            vocab_size=VOCAB_SIZE,
            max_seq_len=local_encoder_cfg['max_seq_len'],
        )

         # Collect parameters for different learning rates
        self.qwen_params = []
        self.new_params = []

        # Qwen parts
        self.qwen_params.extend(self.norm.parameters())
        for layer in self.layers:
            if isinstance(layer, TransformerSelfAttentionLayer):
                self.qwen_params.extend(layer.parameters())

        # New parts
        self.new_params.extend(self.tok_embeddings.parameters())
        self.new_params.extend(self.output.parameters())
        self.new_params.extend(self.local_encoder.parameters())
        self.new_params.extend(self.local_decoder.parameters())

        self.patch_size = patch_size
        self.patching_threshold = patching_threshold
        self.freeze_global_for_n_steps = freeze_global_for_n_steps
        self.current_step = 0
        self.global_frozen = freeze_global_for_n_steps > 0
        if self.global_frozen:
            self._update_freezing()

        # We'll store how many chunks the user wants for final output
        self.num_output_chunks = 0  # default

    def _update_freezing(self):
        if self.global_frozen:
            for param in self.qwen_params:
                param.requires_grad = False
        else:
            for param in self.parameters():
                param.requires_grad = True

    def set_num_output_chunks(self, num_output_chunks: int) -> None:
        super().set_num_output_chunks(num_output_chunks)
        self.num_output_chunks = num_output_chunks

    def forward(
        self,
        tokens: torch.Tensor,
        *,
        mask: Optional[Union[torch.Tensor, float]] = None,
        encoder_input: Optional[torch.Tensor] = None,
        encoder_mask: Optional[torch.Tensor] = None,
        input_pos: Optional[torch.Tensor] = None,
        patch_ids: Optional[torch.Tensor] = None,
    ) -> Union[torch.Tensor, List[torch.Tensor]]:
        # Update freezing state if in training
        if self.training and self.global_frozen:
            self.current_step += 1
            if self.current_step >= self.freeze_global_for_n_steps:
                self.global_frozen = False
                self._update_freezing()

        if patch_ids is None:
            patch_ids, _ = dynamic_patch(tokens, threshold=self.patching_threshold, patch_size=self.patch_size)

        byte_embeds, patch_embs = self.local_encoder(tokens, patch_ids=patch_ids)
        global_out = super().forward(patch_embs, mask=mask, input_pos=input_pos)

        # Assuming the outs are chunked, take entry[0] of outputs.
        if self.num_output_chunks == 0:
            global_out = global_out.to(torch.bfloat16) # TODO: Another fix for the 0 chunk float. Need to move this dtype definiton.
            logits = self.local_decoder(byte_embeds, global_out, patch_ids)
        else:
            global_out_combined = torch.cat(global_out, dim=1) # Unchunking - it would be better not to pass this through I think.
            clogits = self.local_decoder(byte_embeds, global_out_combined, patch_ids)
            logits = [chunk for chunk in clogits.chunk(self.num_output_chunks, dim=1)]
        return logits

    def unified_generate(
        self,
        prompt: Union[torch.LongTensor, List[int]],
        max_new_tokens: int = 128,
        eos_id: int = EOS_ID,
        temperature: float = 0.7,
        top_k: int = 50,
        top_p: float = 0,
        frequency_penalty: float = 0.1,
        repetition_penalty: float = 1.5,
        greedy: bool = False,
    ) -> torch.Tensor:
        # 1) Convert prompt to a tensor [1, seq_len]
        if isinstance(prompt, list):
            prompt = torch.tensor(prompt, dtype=torch.long, device=next(self.parameters()).device)
        if prompt.dim() == 1:
            prompt = prompt.unsqueeze(0) # Add a batch dimension.

        device = prompt.device
        all_tokens = prompt.clone()

        # Track usage for freq / repetition penalty
        token_counts = torch.bincount(all_tokens[0], minlength=VOCAB_SIZE).to(device)
        recent_tokens = []

        # Start generating
        for step_i in range(max_new_tokens):

            # No patching => standard forward
            with torch.no_grad():
                logits = self.forward(all_tokens)

            if isinstance(logits, list):
                logits = torch.cat(logits, dim=1)
            step_logits = logits[0, -1, :]

            # Apply temperature
            if temperature != 1.0:
                step_logits = step_logits / temperature

            # Repetition penalty
            for tk in recent_tokens:
                step_logits[tk] /= repetition_penalty

            # Frequency penalty
            step_logits -= token_counts * frequency_penalty

            # top-k
            if top_k > 0:
                vals, idxs = torch.topk(step_logits, top_k)
                mask = torch.ones_like(step_logits, dtype=torch.bool)
                mask[idxs] = False
                step_logits[mask] = float('-inf')

            # top-p
            probs = torch.softmax(step_logits, dim=-1)
            if top_p > 0:
                sorted_logits, sorted_indices = torch.sort(step_logits, descending=True)
                cumulative_probs = torch.cumsum(probs, dim=-1)
                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[:, 1:] = sorted_indices_to_remove[:, :-1].clone()
                sorted_indices_to_remove[:, 0] = 0
                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                step_logits = step_logits.scatter(-1, indices_to_remove, -1e9)

            # Re‐normalize
            total_p = probs.sum()
            if total_p > 0:
                probs /= total_p
            else:
                probs = torch.ones_like(probs) / probs.size(-1)

            # Next token
            if greedy:
                next_token = torch.argmax(probs).item()
            else:
                next_token = torch.multinomial(probs, 1).item()

            # Add new token
            all_tokens = torch.cat(
                [all_tokens, torch.tensor([[next_token]], device=device)],
                dim=1,
            )

            # Update counters
            token_counts[next_token] += 1
            recent_tokens.append(next_token)
            if len(recent_tokens) > 5:
                recent_tokens.pop(0)

            # EOS break
            if next_token == eos_id:
                break

        return all_tokens

############################
# Tokenizer
############################

class ByteLatentModelTokenizer(nn.Module):
    def __init__(
        self,
        *,
        bpe_delim: bool = False,
        add_bos: bool = True,
        add_eos: bool = True,
        special_tokens: Optional[Dict[str, int]] = None,
        max_seq_len: Optional[int] = None,
        prompt_template: Optional[Any] = None,
    ):
        super().__init__()
        if special_tokens is None:
            special_tokens = {
                "<|bos|>": BOS_ID,
                "<|eos|>": EOS_ID,
                "<|pad|>": PAD_ID,
            }
        self.special_tokens = special_tokens
        self.bos_id = self.special_tokens.get("<|bos|>", BOS_ID)
        self.eos_id = self.special_tokens.get("<|eos|>", EOS_ID)
        self.pad_id = self.special_tokens.get("<|pad|>", PAD_ID)
        self.max_seq_len = max_seq_len
        self.prompt_template = prompt_template

    def encode(self, text: str, add_bos: bool = True, add_eos: bool = True) -> List[int]:
        # naive UTF-8 byte approach:
        byte_data = list(bytes(text, encoding="utf-8", errors="ignore"))
        tokens = byte_data
        if add_bos:
            tokens = [self.bos_id] + tokens
        if add_eos:
            tokens = tokens + [self.eos_id]
        return tokens

    def decode(self, tokens: List[int]) -> str:
        # naive decode - skip special tokens
        return bytes([t for t in tokens if t < 256]).decode("utf-8", errors="ignore")


    def tokenize_messages(self, messages: List[Dict[str, Any]], add_eos: bool = True) -> Tuple[List[int], List[bool]]:
        tokenized_messages = []
        mask = []
        for message in messages:
            if message.role != "ipython":
                tokenized_messages.extend(self.encode("".join([message.role, "\n"]), add_bos=False, add_eos=False))
            for item in message.content:
                if item['type'] == "text":
                    tokenized_messages.extend(
                        self.encode(item['content'], add_bos=False, add_eos=False)
                    )
        if add_eos:
            tokenized_messages.append(self.eos_id)
        mask = [False] * len(tokenized_messages)
        if self.max_seq_len:
            tokenized_messages = tokenized_messages[: self.max_seq_len]
            mask = mask[: self.max_seq_len]
        return tokenized_messages, mask

    def forward(self, sample: Mapping[str, Any], inference: bool = False) -> Mapping[str, Any]:
        messages = sample.pop("messages")
        tokens, mask = self.tokenize_messages(messages)
        sample["tokens"] = tokens
        sample["mask"] = mask
        return sample

############################
# Helper constructor
############################

def blt_tokenizer(
    special_tokens_path: Optional[str] = None,
    max_seq_len: Optional[int] = None,
    prompt_template: Optional[Any] = None,
    **kwargs,
) -> ByteLatentModelTokenizer:
    return ByteLatentModelTokenizer(
        max_seq_len=max_seq_len,
        prompt_template=prompt_template,
        **kwargs,
    )

# Define Qwen 2.5 base model with additional layers, we will expect this to be loaded
# with a pretrained Qwen checkpoint which should match.
def qwen2_5_blt(
    # Warm up decoder/encoder. Tried it with both 100 and 0 and it seemed to work better without
    # the warmup, but never tested a longer warmup.
    freeze_global_for_n_steps=500,
    use_hash_ngrams=True,
    patch_size=4,
    max_seq_len=4096
) -> ByteLatentQwen2p5Decoder:
    qwen_cfg = dict(
        vocab_size=151936, # Kinda irrelevant
        embed_dim=2048,
        num_layers=36,
        num_heads=16,
        num_kv_heads=2,
        max_seq_len=max_seq_len,
        intermediate_dim=11008,
        attn_dropout=0.0,
        norm_eps=1e-6,
    )

    local_enc_cfg = dict(
        vocab_size=VOCAB_SIZE,
        embed_dim=2048, # Keeping same as Qwen
        num_layers=4,
        num_heads=8,
        num_kv_heads=8,
        max_seq_len=max_seq_len,
        hidden_dim=4096,
        norm_eps=1e-5,
        num_cross_layers = 4,
        use_hash_ngrams=use_hash_ngrams,
        max_ngram= 8,
        num_ngram_buckets=500000,
    )

    return ByteLatentQwen2p5Decoder(
        qwen_cfg=qwen_cfg,
        local_encoder_cfg=local_enc_cfg,
        patch_size=patch_size,
        patching_threshold=3.0,
        freeze_global_for_n_steps=freeze_global_for_n_steps
    )


In [ ]:
# https://github.com/facebookresearch/blt/blob/main/bytelatent/model/local_models.py
# perceiver cross attn

# https://github.com/facebookresearch/blt/blob/main/bytelatent/model/latent_transformer.py#L30


In [ ]:
# @title facebookresearch patcher.py entropy monotonicity

# https://github.com/facebookresearch/blt/blob/main/bytelatent/data/patcher.py

def entropy(scores):
    """
    scores: [bs, seq_len, vocab]
    returns [bs, seq_len]
    Computes the entropy for each token in the batch.
    Note: uses natural log.
    """
    log_probs = F.log_softmax(scores, dim=-1)
    probs = torch.exp(log_probs)
    p_log_p = log_probs * probs
    entropy = -p_log_p.sum(dim=-1)
    return entropy


def patch_start_mask_global_and_monotonicity(entropies, t, t_add=0):
    """
    entropies: [bs, seq_len] torch tensor of entropies
    t: threshold
    returns [bs, seq_len] mask where True indicates the start of a patch
    """
    bs, seq_len = entropies.shape
    if seq_len == 0: return entropies > t
    mask = torch.zeros_like(entropies, dtype=torch.bool)
    mask[:, 0] = True
    # Calculate differences between consecutive elements along the sequence length
    differences = entropies[:, 1:] - entropies[:, :-1]
    # Calculate conditions for all elements except the first one in each sequence
    # condition = (differences > t_add) & (entropies[:, 1:] > t) & (~mask[:, :-1])
    condition = differences > t_add
    # Update the mask based on the condition
    mask[:, 1:] = condition
    return mask



In [ ]:
# @title facebookresearch bytelatent/model/blt.py
# https://github.com/facebookresearch/blt/blob/main/bytelatent/model/blt.py
from enum import Enum, auto
from typing import Any, Optional

import torch
from pydantic import ConfigDict, model_validator
from torch import nn
from torch.nn.attention.flex_attention import create_block_mask
from typing_extensions import Self

from bytelatent.base_transformer import (
    BaseTransformerArgs,
    InitStdFactor,
    SequenceModelWithOutput,
    TransformerBlock,
)
from bytelatent.data.patcher import Patcher, PatcherArgs
from bytelatent.model.latent_transformer import GlobalTransformer
from bytelatent.model.local_models import LocalDecoder, LocalEncoder, LocalModelArgs
from bytelatent.model.utils import downsample
from bytelatent.tokenizers.constants import BOE_ID, BOS_ID, EOS_ID, OFFSET, PAD_ID



def causal_mask(b, h, q_idx, kv_idx):
    return q_idx >= kv_idx


def setattrs(_self, **kwargs):
    for k, v in kwargs.items():
        setattr(_self, k, v)


def get_encoder_dim_token_emb(args):
    if args.dim_token is not None:
        dim_token_emb = args.dim_token
    elif args.use_local_encoder_transformer:
        dim_token_emb = args.dim_local_encoder
    else:
        dim_token_emb = args.dim_global // args.patch_size
    return dim_token_emb


def get_encoder_dim_patch_emb(args):
    dim_patch_emb = None
    if args.cross_attn_encoder:
        if args.cross_attn_init_by_pooling:
            dim_patch_emb = args.dim_local_encoder
        else:
            dim_patch_emb = args.dim_global
    return dim_patch_emb


def get_global_dim_patch_emb(args):
    dim_token_emb = get_encoder_dim_token_emb(args)
    if args.cross_attn_encoder:
        dim_patch_emb = dim_token_emb * args.cross_attn_k
    elif (
        args.downsampling_by_pooling is None
        or not args.downsampling_by_pooling
        or len(args.downsampling_by_pooling) == 0
    ):
        dim_patch_emb = dim_token_emb * args.patch_size
    else:
        dim_patch_emb = dim_token_emb * sum([pooling in args.downsampling_by_pooling for pooling in ["avg", "min", "max"]])
    return dim_patch_emb


def get_decoder_dim_token_emb(args):
    if args.share_encoder_decoder_emb:
        dim_token_emb = get_encoder_dim_token_emb(args)
    elif args.dim_token is not None:
        dim_token_emb = args.dim_token
    else:
        dim_token_emb = args.dim_local_decoder
    return dim_token_emb


def parse_ngram_to_size(ngram_to_size_str: str | None) -> dict[int, int]:
    if ngram_to_size_str is None:
        return None
    ngram_to_size = {}
    for entry in ngram_to_size_str.split(","):
        ngram, size = entry.split(":")
        ngram = int(ngram)
        size = int(size)
        ngram_to_size[ngram] = size
    return ngram_to_size


def fill_tokens(tokens, patch_size, fill_id):
    batch_size, seq_len = tokens.shape
    if seq_len % patch_size == 0:
        return tokens
    else:
        remaining = patch_size - seq_len % patch_size
        final_padding = tokens.new(batch_size, remaining).fill_(fill_id)
        return torch.cat((tokens, final_padding), dim=1)


def decoder_patch_ids_from_lengths(patch_lengths, nb_boe, seq_len):
    first_patch_length = patch_lengths[0, 0]
    assert torch.all(
        first_patch_length == patch_lengths[:, 0]
    ), "first patch should always be the same size (1 for dynamic, patch_size for static)."
    assert (
        first_patch_length - nb_boe == 1
    ), f"First patch (patch length: {first_patch_length}) should have one non-boe token (boe toks: {nb_boe})"
    # Remove first patch from patch_ids for local decoder inputs and shift the last patch.
    # decoder_patch_lengths = patch_lengths[:, 1:].clone()
    # decoder_patch_lengths = add_to_last_nonzero_patch(decoder_patch_lengths, 1)
    decoder_patch_lengths = patch_lengths[:, 1:]
    assert (
        decoder_patch_lengths.sum() + (nb_boe + 1) * patch_lengths.shape[0]
        == patch_lengths.sum()
    ), f"{decoder_patch_lengths.sum() + (nb_boe + 1) * patch_lengths.shape[0]} != {patch_lengths.sum()}"
    assert torch.all(decoder_patch_lengths >= 0), f"{decoder_patch_lengths}"
    decoder_patch_ids = patch_ids_from_lengths(
        patch_lengths=decoder_patch_lengths, seq_len=seq_len
    )
    return decoder_patch_ids


primes = [
    1000000007,
    5915587277,
    1500450271,
    3267000013,
    5754853343,
    4093082899,
    9576890767,
    3628273133,
    2860486313,
    5463458053,
    3367900313,
]


def rolling_polynomial_hash(t, hash_func_nb: int = 0):
    prime = torch.tensor(primes[hash_func_nb], dtype=torch.int64, device=t.device)
    prime_powers = torch.stack([prime**i for i in range(t.shape[-1])])
    return torch.sum(t * prime_powers, dim=-1)


def get_rolling_polynomial_hash_fn(hash_func_nb: int = 0, group_size: int = 2):
    prime = torch.tensor(primes[hash_func_nb], dtype=torch.int64)
    prime_powers = torch.stack([prime**i for i in range(group_size)])

    def rolling_polynomial_hash_fn(t):
        return torch.sum(t * prime_powers, dim=-1)

    return rolling_polynomial_hash_fn


def byte_group_hash_function(
    x: torch.Tensor, group_size: int = 2, hash_func_nb: int = 0, max_hash: int = 30000
):
    """
    Returns a hash of the input x and maps it to a value in the range [0, max_hash].

    expects: x of shape (batch_size, seq_len) with values as ids in the token vocab.
    returns a tensor  of shape (batch_size, seq_len) with values in the range [0, max_hash].

    Note: max hash can make a big difference on the number of collisions.
    """
    with torch.no_grad():
        bs, seq_len = x.shape
        # x_numpy = x.numpy()
        # hash_values = torch.zeros(bs, seq_len, dtype=torch.int64, requires_grad=False)
        # for i in range(bs):
        #     for j in range(seq_len):
        #         start = max(j, j-group_size+1)
        #         end = j+1
        #         hash_values[i, j] = hash_array(x_numpy[i, start:end], max_hash)

        prefix = torch.zeros(bs, group_size - 1, dtype=torch.int64, device=x.device)
        x = torch.cat([prefix, x], dim=1)
        windows = x.unfold(1, group_size, 1)
        # hashes = get_rolling_polynomial_hash_fn(hash_func_nb, group_size)(windows)
        hashes = rolling_polynomial_hash(windows, hash_func_nb)
        hash_values_range = hashes % max_hash
    hash_values_range.requires_grad = False
    return hash_values_range


def create_patch_mask_from_ids(
    patch_ids, num_patches, window=None, patches_as_queries=False
):
    """
    Creates a tensor of shape [bs, seq_len, num_patches] where each element at position (i, j, k)
    is True if the patch id at position (i, j) is less than or equal to k.
    Args:
        patch_ids (torch.Tensor): Tensor of shape [bs, seq_len] containing patch ids.
        num_patches (int): Total number of patches.
        window (int): If not None, only considers patches within a window of size window.
        patches_as_queries (bool): If True, the patches are used as queries
    Returns:
        torch.Tensor: Tensor of shape [bs, q_len, kv_len] with the desired mask.
    """
    bs, seq_len = patch_ids.shape
    if not patches_as_queries:
        q_ids = patch_ids.unsqueeze(-1).expand(bs, seq_len, num_patches)
        kv_ids = (
            torch.arange(num_patches, device=patch_ids.device)
            .unsqueeze(0)
            .unsqueeze(0)
            .expand(bs, seq_len, num_patches)
        )
    else:
        kv_ids = patch_ids.unsqueeze(1).expand(bs, num_patches, seq_len)
        q_ids = (
            torch.arange(num_patches, device=patch_ids.device)
            .unsqueeze(0)
            .unsqueeze(-1)
            .expand(bs, num_patches, seq_len)
        )
    if window is None:
        mask = q_ids == kv_ids
    else:
        mask = (kv_ids <= q_ids) & (q_ids < kv_ids + window)
    return mask


def cross_attn_mask(
    patch_ids,
    patch_lengths,
    N,
    patches_as_queries=False,
    cross_attn_k=1,
    window=None,
    block_mask=True,
):
    bs = patch_ids.shape[0]
    with torch.no_grad():
        # Create the patch mask
        cross_mask = create_patch_mask_from_ids(
            patch_ids,
            patch_lengths.shape[1],
            window=window,
            patches_as_queries=patches_as_queries,
        ).repeat_interleave(cross_attn_k, dim=1 if patches_as_queries else -1)
        q_len = patch_lengths.shape[1] * cross_attn_k if patches_as_queries else N
        kv_len = N if patches_as_queries else patch_lengths.shape[1] * cross_attn_k
        assert cross_mask.shape == (
            bs,
            q_len,
            kv_len,
        ), f"{cross_mask.shape} != {(bs, q_len, kv_len)}"
        if block_mask:

            def patch_mask(b, h, q_idx, kv_idx):
                return cross_mask[b, q_idx, kv_idx]

            block_mask = create_block_mask(
                patch_mask,
                B=bs,
                H=None,
                Q_LEN=q_len,
                KV_LEN=kv_len,
                _compile=True,
            )
            return block_mask
        else:
            return torch.where(
                cross_mask, torch.tensor(0.0), torch.tensor(float("-inf"))
            ).unsqueeze(
                1
            )  # [bs, 1, q_len, kv_len]


def get_blt_input(
    tokens: torch.Tensor,
    enforce_patch_size_multiple: bool,
    nb_boe: torch.Tensor,
    patch_size: int,
    boe_id: int,
):
    """
        This function returns X_et, X_gt and X_dt, the encoder, global, and decoder
    tokens respectively.

    Consider the input and target sequences:
    X=[3,4,5,6,7,eos,bos,8,9,10,eos,bos,11,12,13]
    Y=[4,5,6,7,eos,bos,8,9,10,eos,bos,11,12,13,14]
    with patch_size=4

    Note 1: that there will be no special tokens introduced at the patch level.
    Note 2: X_e needs to be trimmed to be passed to Global

    Current without boe:
    X_et = [[boe,boe,boe,boe] [3,4,5,6],      [7,eos,bos,8],    [9,10,eos,bos] [11,12,13, pad]]
    X_g =  [[boe,boe,boe,boe] [3,4,5,6],      [7,eos,bos,8],    [9,10,eos,bos] [11,12,13, pad]] # remove last glob patch
    X_dt = [[3,4,5,6]         [7,eos,bos,8],  [9,10,eos,bos],   [11,12,13]]
    Y =    [[4,5,6,7]         [eos,bos,8,9],  [10,eos,bos,11],  [12,13,14]]

    --> lag fix:
    X_et = [[boe,boe,boe,3]   [4,5,6,7],      [eos,bos,8,9],    [10,eos,bos,11] [12,13,pad,pad]]
    X_g =  [[boe,boe,boe,3]   [4,5,6,7],      [eos,bos,8,9],    [10,eos,bos,11]]
    X_dt = [[3,4,5,6]         [7,eos,bos,8],  [9,10,eos,bos],   [11,12,13]]
    Y =    [[4,5,6,7]    	  [eos,bos,8,9],  [10,eos,bos,11],  [12,13,14]]

    Dynamic (current):
    X = [3,4,5,6,7,eos,bos,8,9,10,eos,bos]
    Y = [4,5,6,7,eos,bos,8,9,10,eos,bos,11]

    entropy patching:
    input: 7, bos, 9, 10
    pred (high entropy): eos, 8, 10, eos

    X_et = [[boe,3,4,5,6,7,eos,bos,8,9,10,eos,bos]
    X_g =  [[boe],      [3,4,5,6], [7,eos],[bos,8],[9],     [10,eos]]
    X_dt = [[3,4,5,6],  [7,eos],   [bos,8],[9],    [10,eos],[bos]]
    Y =    [4,5,6,7,eos,bos,8,9,10,eos,bos,11]

    --> lag fix no boe (force single byte first patch):
    X_et = [[3,4,5,6,7,eos,bos,8,9,10,eos,bos,11,12]
    X_g =  [[3],        [4,5,6,7], [eos,bos],[8,9], [10],       [eos,bos],      [11,12]] # remove last global patch
    X_dt = [[3,4,5,6],  [7,eos],   [bos,8], [9],    [10,eos],   [bos,11,12]]
    Y =    [4,5,6,7,    eos,bos,    8,9,    10,     eos,bos,    11,12,13]

    input: 4, 7, bos, 9, 10
    pred (high entropy): 5, eos, 8, 10, eos

    X_et = [[3,4,5,6,7,eos,bos,8,9,10,eos,bos,11,12]
    X_g =  [[3],        [4]   ,   [5,6,7], [eos,bos],[8,9], [10],       [eos,bos],      [11,12]] # remove last global patch
    X_dt = [[3]         [4,5,6],  [7,eos],   [bos,8], [9],    [10,eos],   [bos,11,12]]
    Y =    [4,]         [5,6,7,    eos,bos,    8,9,    10,     eos,bos,    11,12,13]

    Handle the last byte properly.
    patch_lengths = [1, 1,         3,      2,         2      1           2               2         1]
    X_et = [[3,4,5,6,7,eos,bos,8,9,10,eos,bos,11,12]
    X_g =  [[3],        [4]   ,   [5,6,7], [eos,bos],[8,9], [10],       [eos,bos],      [11,12]] # do not remove last global patch
    X_dt = [[3]         [4,5,6],  [7,eos],   [bos,8], [9],    [10,eos],   [bos,11]       [12]]
    Y =    [4,]         [5,6,7,    eos,bos,    8,9,    10,     eos,bos,    11,12,        13]]


    bpe delim
    X_et = [[3,4,5,6,7,<d>,eos,bos,<d>,8,9,<d>,10,<d>,eos,bos,11,12]
    X_g =  [[3],          [4,5,6,7,<d>],     [eos,bos,<d>], ..
    X_dt = [[3,4,5,6,7],  [<d>,eos,bos],     [<d>,bos,8], ..
    Y =    [4,5,6,7,<d>,    eos,bos,<d>       8,9,<d>, ..


    Note 1: that there will be no special tokens introduced at the patch level.
    Note 2: X_e needs to be trimmed to be passed to Global
    """
    batch_size, seq_len = tokens.shape
    local_encoder_tokens = tokens
    local_decoder_tokens = tokens

    if nb_boe > 0:
        padded_patch = tokens.new(batch_size, nb_boe).fill_(boe_id)
        local_encoder_tokens = torch.cat((padded_patch, local_encoder_tokens), dim=1)
    # global_tokens = tokens.new(batch_size, ((seq_len-1) // patch_size)+1).fill_(boe_id)

    # create global tokens, contains boe tokens and eos
    # padded_local_encoder_tokens = fill_tokens(local_encoder_tokens, patch_size, boe_id)
    # patches = padded_local_encoder_tokens.view(batch_size, -1, patch_size)
    # global_tokens = (patches.eq(eos_id).any(dim=2).int() * eos_id)[:, 1:]
    # global_tokens += global_tokens.eq(0).int() * boe_id
    # TODO: fix this when we want to use block causal in the global.

    if enforce_patch_size_multiple and local_encoder_tokens.shape[-1] % patch_size != 0:
        local_encoder_tokens = fill_tokens(local_encoder_tokens, patch_size, boe_id)

    return local_encoder_tokens, None, local_decoder_tokens


def patch_ids_from_lengths(patch_lengths, seq_len):
    bs, num_patches = patch_lengths.shape
    # Create a tensor of cumulative sums of the patch lengths
    cum_d = torch.cat(
        [
            torch.zeros(bs, 1, dtype=patch_lengths.dtype, device=patch_lengths.device),
            patch_lengths.cumsum(dim=-1),
        ],
        dim=-1,
    )
    patch_ids = (cum_d.unsqueeze(-1) <= torch.arange(seq_len, device=cum_d.device)).sum(
        dim=-2
    ) - 1
    assert not (
        torch.max(patch_ids) > patch_lengths.shape[-1] or torch.min(patch_ids) < 0
    ), f"{torch.max(patch_ids)} > {patch_lengths.shape[-1]} or {torch.min(patch_ids)} < 0"
    return patch_ids


class ByteLatentTransformerArgs(BaseTransformerArgs):
    # Basic model configuration
    seed: int = 42
    vocab_size: int = -1
    dim: int = 512
    n_layers: int = 8
    n_heads: int = 8
    # TODO: What is the purpose of this parameter?
    weight_tying: bool = False
    patch_in_forward: bool = False

    # Architecture and dimensions
    dim_token: int | None = None
    dim_global: int = 512
    dim_local_decoder: int = 512
    dim_local_encoder: int = 512
    n_layers_global: int = 8
    n_layers_local_decoder: int = 8
    n_layers_local_encoder: int = 8

    # Tokenization and patching
    patch_size: float | None = None
    patching_mode: str | None = None
    patching_threshold: float | None = None
    patching_threshold_add: float | None = None
    monotonicity: bool = False
    patching_batch_size: int = 1
    patching_device: str = "cuda"
    max_patch_length: int | None = None

    # Encoder/Decoder configuration
    tie_local_encoder_decoder_logits: bool = False
    use_local_encoder_transformer: bool = False
    encoder_lm_loss: bool = False
    max_encoder_seq_length: int | None = None
    pad_to_max_length: bool = False
    encoder_enable_byte_ngrams: bool = False
    encoder_enable_byte_group_hash: bool = False
    ngram_vocab_sizes: int | None = None

    # Cross attention configurations
    cross_attn_encoder: bool = False
    cross_attn_decoder: bool = False
    cross_attn_window_encoder: int | None = None
    cross_attn_window_decoder: int | None = None
    cross_attn_k: int | None = None
    cross_attn_nheads: int | None = None
    cross_attn_all_layers_decoder: bool = False
    cross_attn_all_layers_encoder: bool = False
    cross_attn_use_flex_attention: bool = True
    cross_attn_init_by_pooling: bool = False

    # Encoder hash configurations
    encoder_hash_byte_group_size: Any | None = None
    encoder_hash_byte_group_vocab: int = 30000
    encoder_hash_byte_group_nb_functions: int = 3

    # Model behavior and optimization
    log_patch_lengths: bool = False
    non_linearity: str = "swiglu"
    use_rope: bool = True
    recompute_fc1_out: bool = False
    recompute_fc3_out: bool = False
    recompute_attn: bool = True
    custom_bwd: bool = False
    layer_ckpt: str = "all"

    # Initialization and attention
    init_use_gaussian: bool = True
    init_use_depth: str = "current"
    attn_bias_type: str = "causal"
    alpha_depth: str = "disabled"
    max_length: int = 2048

    # Norm configuration
    norm_eps: float = 1e-5
    norm_affine: bool = True
    pre_norm: bool = True
    norm_type: str = "rmsnorm"

    # Additional configurations
    multiple_of: int = 256
    ffn_dim_multiplier: float = 1.0
    dropout: float = 0
    output_size: int = -1

    # Additional parameters from ModelArgs
    architecture: str = "vanilla"
    share_encoder_decoder_emb: bool = True
    global_local_decoder_residual_layer: str | None = None

    tokenize_with_bpe_delimiter: bool = False
    patching_thresholds_str: str | None = None
    tie_local_encoder_decoder: bool = False
    encoder_preds_low_entropy_toks: float | None = None
    encoder_preds_random_toks: float | None = None
    dim_token_emb: int | None = None
    dim_patch_emb: int | None = None

    encoder_ngram_table_dir: str | None = None
    encoder_ngram_to_size_str: str | None = None

    # Model architecture params
    entropy_model_checkpoint_dir: str | None = None
    entropy_model_is_ngram_model: bool = False
    downsampling_by_pooling: str | None = None
    n_heads_global: int = 8
    n_heads_local_decoder: int = 8
    n_heads_local_encoder: int = 8
    n_kv_heads: int | None = None
    n_kv_heads_global: int | None = None
    conv_kernel_size: int | None = None
    local_attention_window_len: int | None = None

    # Performance optimization
    sequence_parallel: bool = False
    loss_parallel: bool = False
    fuse_sequence_parallel: bool = False
    use_fsdp: bool = True
    attn_to_keep: str = "all"

    # Parameter mixing
    pm_size: int = 0

    # Logging
    full_logging_n_layers: int = 4

    @model_validator(mode="after")
    def check_hash_byte_sizes(self) -> Self:
        if (
            self.encoder_hash_byte_group_size is not None
            and type(self.encoder_hash_byte_group_size) == str
        ):
            self.encoder_hash_byte_group_size = [
                int(x)
                for x in self.encoder_hash_byte_group_size.split(",")
                if len(x) > 0
            ]
        return self


class GlobalTransformerArgs(ByteLatentTransformerArgs):
    # Global encoder specific dimensions
    dim_token_emb: int | None = None
    dim_patch_emb: int | None = None

    def __post_init__(self):
        # Override base args with global encoder specific values
        self.dim = self.dim_global
        self.n_layers = self.n_layers_global
        self.n_heads = self.n_heads_global
        self.n_kv_heads = self.n_kv_heads_global
        self.local_attention_window_len = None
        self.cross_attn_encoder = False
        self.cross_attn_decoder = False


class LocalDecoderArgs(ByteLatentTransformerArgs):
    # Local decoder specific dimensions
    dim_token_emb: int | None = None
    dim_patch_emb: int | None = None

    def __post_init__(self):
        # Override base args with local decoder specific values
        self.dim = self.dim_local_decoder
        self.n_layers = self.n_layers_local_decoder
        self.n_heads = self.n_heads_local_decoder
        self.cross_attn_encoder = False
        self.cross_attn_init_by_pooling = False
        self.attn_bias_type = "local_block_causal"


def create_global_transformer(args: ByteLatentTransformerArgs) -> GlobalTransformer:
    global_args = args.model_copy(
        deep=True,
        update=dict(
            dim=args.dim_global,
            n_layers=args.n_layers_global,
            n_heads=args.n_heads_global,
            n_kv_heads=args.n_kv_heads_global,
            local_attention_window_len=None,
            dim_token_emb=get_global_dim_patch_emb(args),
            dim_patch_emb=None,
            cross_attn_encoder=False,
            cross_attn_decoder=False,
        ),
    )

    return GlobalTransformer(global_args)


def create_local_encoder(args: ByteLatentTransformerArgs) -> LocalEncoder:
    local_encoder_args = LocalModelArgs(
        # Updated args
        dim=args.dim_local_encoder,
        n_layers=args.n_layers_local_encoder,
        n_heads=args.n_heads_local_encoder,
        dim_token_emb=get_encoder_dim_token_emb(args),
        dim_patch_emb=get_encoder_dim_patch_emb(args),
        cross_attn_encoder=args.cross_attn_encoder,
        cross_attn_decoder=False,
        cross_attn_k=args.cross_attn_k if args.cross_attn_encoder else None,
        cross_attn_init_by_pooling=args.cross_attn_init_by_pooling,
        # Defaults
        head_dim=args.head_dim,
        max_seqlen=args.max_encoder_seq_length,
        dropout=args.dropout,
        vocab_size=args.vocab_size + args.pm_size,
        norm_eps=args.norm_eps,
        patch_size=args.patch_size,
        sliding_window=args.local_attention_window_len,
        use_rope=args.use_rope,
        rope_theta=args.rope_theta,
        rope_use_fp32_in_outer_product=args.rope_use_fp32_in_outer_product,
        init_base_std=args.init_base_std,
        init_std_factor=args.init_std_factor,
        n_kv_heads=args.n_kv_heads,
        attn_impl=args.attn_impl,
        attn_bias_type="local_block_causal",
        multiple_of=args.multiple_of,
        ffn_dim_multiplier=args.ffn_dim_multiplier,
        patching_mode=args.patching_mode,
        use_local_encoder_transformer=args.use_local_encoder_transformer,
        downsampling_by_pooling=args.downsampling_by_pooling,
        encoder_hash_byte_group_size=args.encoder_hash_byte_group_size,
        cross_attn_all_layers_encoder=args.cross_attn_all_layers_encoder,
        cross_attn_all_layers_decoder=args.cross_attn_all_layers_decoder,
        cross_attn_nheads=args.cross_attn_nheads,
        eos_id=args.eos_id,
    )

    return LocalEncoder(local_encoder_args)


def create_local_decoder(args: ByteLatentTransformerArgs) -> LocalDecoder:
    # First deep copy the original args
    local_decoder_args = LocalModelArgs(
        dim=args.dim_local_decoder,
        n_layers=args.n_layers_local_decoder,
        n_heads=args.n_heads_local_decoder,
        dim_token_emb=get_decoder_dim_token_emb(args),
        dim_patch_emb=args.dim_global,
        cross_attn_encoder=False,
        cross_attn_decoder=args.cross_attn_decoder,
        cross_attn_init_by_pooling=False,  # states are already defined
        cross_attn_k=args.cross_attn_k if args.cross_attn_decoder else None,
        # Defaults
        head_dim=args.head_dim,
        max_seqlen=args.max_encoder_seq_length,
        dropout=args.dropout,
        vocab_size=args.vocab_size + args.pm_size,
        norm_eps=args.norm_eps,
        patch_size=args.patch_size,
        sliding_window=args.local_attention_window_len,
        use_rope=args.use_rope,
        rope_theta=args.rope_theta,
        rope_use_fp32_in_outer_product=args.rope_use_fp32_in_outer_product,
        init_base_std=args.init_base_std,
        init_std_factor=args.init_std_factor,
        n_kv_heads=args.n_kv_heads,
        attn_impl=args.attn_impl,
        attn_bias_type="local_block_causal",
        multiple_of=args.multiple_of,
        ffn_dim_multiplier=args.ffn_dim_multiplier,
        patching_mode=args.patching_mode,
        use_local_encoder_transformer=args.use_local_encoder_transformer,
        downsampling_by_pooling=args.downsampling_by_pooling,
        encoder_hash_byte_group_size=args.encoder_hash_byte_group_size,
        cross_attn_all_layers_encoder=args.cross_attn_all_layers_encoder,
        cross_attn_all_layers_decoder=args.cross_attn_all_layers_decoder,
        cross_attn_nheads=args.cross_attn_nheads,
        eos_id=args.eos_id,
    )

    return LocalDecoder(local_decoder_args)


class EmbeddingType(Enum):
    HASH_TOK = auto()
    NGRAM = auto()


def init_embeddings(
    args,
    embedding_type: EmbeddingType,
    local_encoder_dim: int,
    encoder_hash_byte_group_size: list = None,
):
    if (
        embedding_type == EmbeddingType.HASH_TOK
        and args.encoder_hash_byte_group_size is None
    ):
        return None
    if embedding_type == EmbeddingType.NGRAM and args.encoder_ngram_to_size_str is None:
        return None

    embeddings = []

    if embedding_type == EmbeddingType.HASH_TOK:
        emb_dim = local_encoder_dim
        encoder_hash_byte_group_vocab = args.encoder_hash_byte_group_vocab
        for _ in range(args.encoder_hash_byte_group_nb_functions):
            for _ in encoder_hash_byte_group_size:
                embeddings.append(
                    nn.Embedding(encoder_hash_byte_group_vocab, emb_dim))

    elif embedding_type == EmbeddingType.NGRAM:
        encoder_ngram_to_size = parse_ngram_to_size(args.encoder_ngram_to_size_str)
        emb_dim = local_encoder_dim
        OFFSET = 4  # This should be passed as parameter if it's variable
        for ngram_vocab_size in encoder_ngram_to_size.values():
            embeddings.append(nn.Embedding(ngram_vocab_size + OFFSET, emb_dim))

    return nn.ModuleList(embeddings)


def compute_hash_embeddings(
    local_encoder_tokens: torch.Tensor,
    local_encoder,
    encoder_hash_tok_embedding: nn.ModuleList,
    encoder_hash_byte_group_nb_functions: int,
    encoder_hash_byte_group_size: list,
    encoder_hash_byte_group_vocab: int,
) -> torch.Tensor:
    """
    Compute embeddings using hash token embeddings.

    Args:
        local_encoder_tokens: Input tokens tensor
        local_encoder: Encoder object with tok_embeddings method
        encoder_hash_tok_embedding: ModuleList of hash token embeddings
        encoder_hash_byte_group_nb_functions: Number of hash functions
        encoder_hash_byte_group_size: List of byte group sizes
        encoder_hash_byte_group_vocab: Vocabulary size for hash embeddings

    Returns:
        torch.Tensor: Combined embeddings
    """
    if encoder_hash_tok_embedding is None:
        return None

    local_encoder_embeds = local_encoder.tok_embeddings(local_encoder_tokens)

    i = 0
    for func_nb in range(encoder_hash_byte_group_nb_functions):
        for byte_group_size in encoder_hash_byte_group_size:
            hash_ids = byte_group_hash_function(
                local_encoder_tokens,
                byte_group_size,
                hash_func_nb=func_nb,
                max_hash=encoder_hash_byte_group_vocab,
            )
            hash_tok_embedding = encoder_hash_tok_embedding[i]
            local_encoder_embeds = local_encoder_embeds + hash_tok_embedding(hash_ids)
            i += 1

    assert i == len(encoder_hash_tok_embedding)
    return local_encoder_embeds


class ByteLatentTransformer(nn.Module, SequenceModelWithOutput):
    """
    The ByteLatentTransformer (BLT) is a byte-level language model architecture that processes byte sequences
    by dynamically segmenting them into patches. It uses a combination of local encoders, global transformers,
    and local decoders to efficiently encode and decode byte sequences, leveraging patch-based processing for
    improved performance and inference efficiency.
    """

    def __init__(self, args: ByteLatentTransformerArgs):
        super().__init__()

        # General configuration
        self.weight_tying = args.weight_tying
        self.patch_size = args.patch_size
        self.patching_mode = args.patching_mode
        self.boe_id, self.bos_id, self.pad_id, self.eos_id = (
            BOE_ID,
            BOS_ID,
            PAD_ID,
            EOS_ID,
        )
        self.downsampling_by_pooling = args.downsampling_by_pooling
        self.patching_threshold = args.patching_threshold
        self.dim = args.dim
        self.init_base_std = args.init_base_std
        self.init_std_factor = InitStdFactor(args.init_std_factor)
        self.max_seqlen = args.max_seqlen

        # Cross attention configuration
        self.cross_attn_encoder = args.cross_attn_encoder
        self.cross_attn_decoder = args.cross_attn_decoder
        self.cross_attn_k = args.cross_attn_k
        self.cross_attn_window_encoder = args.cross_attn_window_encoder
        self.cross_attn_window_decoder = args.cross_attn_window_decoder
        self.cross_attn_use_flex_attention = args.cross_attn_use_flex_attention

        # Encoder hash configuration
        self.encoder_hash_byte_group_size = args.encoder_hash_byte_group_size
        self.encoder_hash_byte_group_vocab = args.encoder_hash_byte_group_vocab
        self.encoder_hash_byte_group_nb_functions = (
            args.encoder_hash_byte_group_nb_functions
        )

        # ByteLatent modules
        self.local_encoder = create_local_encoder(args)
        self.global_transformer = create_global_transformer(args)
        self.local_decoder = create_local_decoder(args)
        self.encoder_hash_tok_embedding = init_embeddings(
            args,
            EmbeddingType.HASH_TOK,
            local_encoder_dim=self.local_encoder.dim,
            encoder_hash_byte_group_size=self.encoder_hash_byte_group_size,
        )
        self.encoder_ngram_embedding = init_embeddings(
            args,
            EmbeddingType.NGRAM,
            local_encoder_dim=self.local_encoder.dim,
            encoder_hash_byte_group_size=None,
        )

        # Encoder ngram embedding tables
        self.encoder_ngram_embedding = None
        if args.encoder_enable_byte_ngrams:
            self.encoder_ngram_embedding = nn.ModuleList()
            assert args.ngram_vocab_sizes is not None
            self.encoder_ngram_to_size = parse_ngram_to_size(
                args.encoder_ngram_to_size_str
            )
            ngram_emb_dim = self.local_encoder.dim
            for ngram_vocab_size in self.encoder_ngram_to_size.values():
                self.encoder_ngram_embedding.append(
                    nn.Embedding(ngram_vocab_size + OFFSET, ngram_emb_dim)
                )

        # Output layer
        assert args.vocab_size > 0, "vocab_size must be greater than 0"

        # Patcher module
        if args.patch_in_forward:
            self.patcher = Patcher(
                PatcherArgs(
                    patch_size=args.patch_size,
                    patching_mode=args.patching_mode,
                    patching_threshold=args.patching_threshold,
                    patching_threshold_add=args.patching_threshold_add,
                    monotonicity=args.monotonicity,
                    max_patch_length=args.max_patch_length,
                )
            )

    def get_output_seq_len(self):
        return self.max_seqlen

    def forward(
        self,
        tokens: torch.Tensor,
        patch_lengths: Optional[torch.Tensor] = None,
        ngram_ids: Optional[torch.Tensor] = None,
    ):
        # Ensure ngram_ids is either a tensor or None
        assert (
            isinstance(ngram_ids, torch.Tensor) or ngram_ids is None
        ), f"ngram_ids must be a tensor or None, but was: {type(ngram_ids)}"

        bs, N = tokens.shape  # Batch size and sequence length

        # Get megabyte inputs
        nb_boe = int(0 if self.patching_mode != "" else self.patch_size - 1)
        local_encoder_tokens, _, local_decoder_tokens = get_blt_input(
            tokens=tokens,
            enforce_patch_size_multiple=False,
            nb_boe=nb_boe,
            patch_size=self.patch_size,
            boe_id=self.boe_id,
        )

        # Patching
        if patch_lengths is None:
            assert (
                getattr(self, "patcher", None) is not None
            ), "Patcher not defined and no patch_lengths passed."
            patch_lengths, tok_scores = self.patcher.patch(
                local_encoder_tokens,
                include_next_token=True,
                threshold=self.patcher.threshold,
            )
        else:
            if nb_boe > 0:
                patch_lengths[:, 0] += nb_boe

        assert torch.min(patch_lengths) >= 0

        # Generate patch IDs from patch_lengths
        patch_ids = patch_ids_from_lengths(
            patch_lengths, local_encoder_tokens.shape[-1]
        )
        assert torch.max(patch_ids) + 1 <= torch.max(
            (patch_lengths != 0).sum(dim=-1)
        ), f"{torch.max(patch_ids) + 1} > {torch.max((patch_lengths != 0).sum(dim=-1))}"

        cross_attn_mask_enc = None
        # Cross-attention encoder
        if self.cross_attn_encoder:
            cross_attn_mask_enc = cross_attn_mask(
                patch_ids,
                patch_lengths,
                N,
                patches_as_queries=True,
                cross_attn_k=self.cross_attn_k,
                window=self.cross_attn_window_encoder,
                block_mask=self.cross_attn_use_flex_attention,
            )

        # Hashing and embedding
        local_encoder_embeds = compute_hash_embeddings(
            local_encoder_tokens=local_encoder_tokens,
            local_encoder=self.local_encoder,
            encoder_hash_tok_embedding=self.encoder_hash_tok_embedding,
            encoder_hash_byte_group_nb_functions=self.encoder_hash_byte_group_nb_functions,
            encoder_hash_byte_group_size=self.encoder_hash_byte_group_size,
            encoder_hash_byte_group_vocab=self.encoder_hash_byte_group_vocab,
        )

        # N-gram table embeddings
        if self.encoder_ngram_embedding is not None:
            assert ngram_ids is not None, "ngram_ids must be provided"
            if local_encoder_embeds is None:
                local_encoder_embeds = self.local_encoder.tok_embeddings(
                    local_encoder_tokens
                )
            assert len(ngram_ids) == len(
                self.encoder_ngram_embedding
            ), f"ngram_ids.shape[0]={ngram_ids.shape[0]} versus len(encoder_ngram_embedding)={len(self.encoder_ngram_embedding)}, ngram_ids.shape={ngram_ids.shape}"
            for i in range(ngram_ids.shape[0]):
                ngram_embedding = self.encoder_ngram_embedding[i]
                ngram_embeds = ngram_embedding(ngram_ids[i])
                assert (
                    local_encoder_embeds.shape == ngram_embeds.shape
                ), f"Shape mismatch: {local_encoder_embeds.shape} vs {ngram_embeds.shape}, ngram_ids.shape={ngram_ids.shape}"
                local_encoder_embeds = local_encoder_embeds + ngram_embeds

        # Local encoder
        (h_encoder, h_cross), cache_encoder = self.local_encoder(
            tokens=local_encoder_tokens,
            embeds=local_encoder_embeds,
            patch_embeds=None,
            cross_mask=cross_attn_mask_enc,
            num_patches=patch_lengths.shape[1],
            patch_ids=patch_ids,
        )

        # Downsampling
        if not self.cross_attn_encoder:
            assert (
                patch_ids.shape[1] == h_encoder.shape[1]
            ), f"{patch_ids.shape[1]} != {h_encoder.shape[1]}"
            h = downsample(
                h_encoder,
                patch_lengths.shape[1],
                patch_lengths,
                patch_ids,
                downsampling_by_pooling=self.downsampling_by_pooling,
                patch_size=self.patch_size,
            )
        else:
            # Reshape h_cross
            h = h_cross.view(bs, patch_lengths.shape[1], -1)

        # Global transformer
        global_tokens = tokens.new(h.shape[0], h.shape[1]).fill_(self.boe_id)
        rows, cols = torch.where(local_encoder_tokens == self.eos_id)
        eos_patch_ids = patch_ids[rows, cols]
        global_tokens[rows, eos_patch_ids] = self.eos_id

        h, _ = self.global_transformer(
            embeds=h,
            tokens=global_tokens,
        )

        # Unpatching
        dec_embeds = h_encoder[:, nb_boe : nb_boe + N, :]

        # Generate decoder patch IDs
        decoder_patch_ids = decoder_patch_ids_from_lengths(
            patch_lengths, nb_boe, local_decoder_tokens.shape[-1]
        )
        assert (
            torch.max(decoder_patch_ids) + 1 <= h.shape[1]
        ), f"{torch.max(decoder_patch_ids) + 1} > {h.shape[1]}"
        assert (
            decoder_patch_ids.shape[1] == dec_embeds.shape[1]
        ), f"{decoder_patch_ids.shape[1]} != {dec_embeds.shape[1]}"

        # Cross-attention decoder
        if not self.cross_attn_decoder:
            h = torch.gather(
                h, 1, decoder_patch_ids.unsqueeze(-1).expand(-1, -1, h.shape[-1])
            )
            cross_attn_mask_dec = None
            assert local_decoder_tokens.shape == h.shape[:-1]
        else:
            cross_attn_mask_dec = cross_attn_mask(
                decoder_patch_ids,
                patch_lengths,
                N,
                patches_as_queries=False,
                cross_attn_k=self.cross_attn_k,
                window=self.cross_attn_window_decoder,
                block_mask=self.cross_attn_use_flex_attention,
            )

        # Local decoder
        output, _ = self.local_decoder(
            embeds=dec_embeds,
            patch_embeds=h,
            tokens=local_decoder_tokens,
            cross_mask=cross_attn_mask_dec,
        )
        return output

    def init_weights(self):
        self.local_encoder.init_weights()
        self.global_transformer.init_weights()
        self.local_decoder.init_weights()

        emb_std = self.local_encoder.dim ** (-0.5)
        for emb in self.encoder_hash_tok_embedding:
            nn.init.trunc_normal_(
                emb.weight,
                mean=0.0,
                std=emb_std,
                a=-3 * emb_std,
                b=3 * emb_std,
            )


## trash

In [ ]:
# @title big char dataloader fail



# !pip install -qU datasets # restart?
from datasets import load_dataset
import torch
from torch.utils.data import Dataset

# dataset = load_dataset("stas/openwebtext-10k", split="train", streaming=True, cache_dir="/content/hf")
dataset = load_dataset("stas/openwebtext-10k", split="train", cache_dir="/content/hf")
data = dataset['text']
chars = sorted(list(set(data)))
print(chars)


# chr()
# ord()





data = dataset['text']
data = ''.join(data)
chars = sorted(list(set(data)))
print(chars[:200])

print(len(data))
# print(ord('\n'))
print([ord(x) for x in chars])
print([chr(i) for i in [10, *list(range(32,127))]])
# 10, 32-


# @title hf stream dataset me
!pip install -qU datasets # restart?
from datasets import load_dataset
import torch
from torch.utils.data import Dataset
import tiktoken # https://github.com/openai/tiktoken/tree/main

class StreamDataset(Dataset):
    def __init__(self, dataset, seq_len, buffer_size):
        self.enc = tiktoken.get_encoding("gpt2") # https://github.com/openai/tiktoken/blob/main/tiktoken/core.py
        self.vocab_size = self.enc.n_vocab # gpt2:50257



        self.dataset = dataset
        self.data = iter(dataset)
        self.seq_len = seq_len
        self.buffer_size = buffer_size  # must be ≥ seq_len
        self.buffer = []  # token buffer
        self.fill_buffer()

    def fill_buffer(self):
        while len(self.buffer) < self.buffer_size:
            x = next(self.data)
            tokens = self.enc.encode(x["text"]) # tiktoken
            self.buffer.extend(tokens)

    def __len__(self):
        # /4.5/(4/3)
        return 128000000
        # return self.length

    def __getitem__(self, idx):
        # print('get', idx)
        if idx == 0: self.data = iter(self.dataset)
        if len(self.buffer) < self.seq_len: self.fill_buffer()
        if len(self.buffer) < self.seq_len:
            raise StopIteration
        x = self.buffer[:self.seq_len]
        self.buffer = self.buffer[self.seq_len:]
        return torch.tensor(x)

def collate_fn(batch):
    # print(batch)
    return torch.stack(batch)

# dataset = load_dataset("stas/openwebtext-10k", split="train", streaming=True, cache_dir="/content/hf")
# dataset = load_dataset("maxtli/OpenWebText-2M", split="train", streaming=True, cache_dir="/content/hf") # 8.7,3.8
dataset = load_dataset("Skylion007/openwebtext", trust_remote_code=True, split="train", streaming=True, cache_dir="/content/hf") # 8.7,3.8

seq_len = 128
buffer_size = seq_len*1
train_data = StreamDataset(dataset, seq_len, buffer_size) # train_data = StreamDataset(dataset["train"], seq_len, buffer_size)
del dataset

from torch.utils.data.dataloader import DataLoader
batch_size = 64 #512
train_loader = DataLoader(train_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=True, pin_memory=True, num_workers=2)
del train_data


import torch
import torch.nn as nn
from torch.utils.data import Dataset

class CharDataset(Dataset): # https://github.com/karpathy/minGPT
    def __init__(self, raw_data, seq_len):
        data = ''.join(raw_data)
        chars = sorted(list(set(data)))
        self.vocab_size = len(chars) # 283
        self.stoi = {ch:i for i,ch in enumerate(chars)}
        self.itos = {i:ch for i,ch in enumerate(chars)}
        self.data = self.data_process(data) # list of int
        self.seq_len = seq_len

    def data_process(self, data): # str 10780437
        return torch.tensor([self.stoi.get(c) for c in data]) # list of int 4570571 # stoi.get(c,UNK_IDX)

    def __len__(self):
        # return len(self.data) - self.seq_len
        return len(self.data)//(self.seq_len+1)

    def __getitem__(self, idx):
        # dix = self.data[idx:idx + self.seq_len + 1]
        dix = self.data[idx*(self.seq_len+1) : (idx+1)*(self.seq_len+1)]
        x, y = dix[:-1], dix[1:]
        return x, y


seq_len = 100 # 128
train_data = CharDataset(text, seq_len) # one line of poem is roughly 50 characters
test_data = CharDataset(test_text, seq_len) # one line of poem is roughly 50 characters
from torch.utils.data.dataloader import DataLoader
batch_size = 64 #512
train_loader = DataLoader(train_data, shuffle = True, pin_memory = True, batch_size = batch_size, num_workers = 2) # num_workers = 4
test_loader = DataLoader(test_data, shuffle = True, pin_memory = True, batch_size = batch_size, num_workers = 0)


def encode(context): return torch.tensor([train_data.stoi.get(c) for c in context], device=device).unsqueeze(0)
def decode(x): return ''.join([train_data.itos[int(i)] for i in x])
# for x,y in train_loader:
#     break
# print(train_data.vocab_size)
